In [1]:
import multiprocessing
import os
from re import A
from typing import ItemsView

from py import log
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
from xtquant import xttrader
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant
import pandas_market_calendars as mcal
import pandas as pd
import akshare as ak

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
def plot(df, column):
    
    # 设置中文字体
    mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定中文字体为 SimHei
    mpl.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 假设这是你的DataFrame，其中包含收益

    # 绘制收益曲线
    # 将日期列转换为日期类型
    # df['date'] = pd.to_datetime(df['date'])

    # # 设置日期列为索引
    # df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    # 绘制收益曲线
    plt.figure(figsize=(10, 6))  # 设置图表大小
    plt.plot(df.index, df[column], label='Return')
    plt.title('收益曲线')
    plt.xlabel('日期')
    plt.ylabel('收益率')
    plt.legend()  # 显示图例
    plt.grid(True)  # 显示网格线

    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))

    plt.show()

d:\anaconda3\envs\python3.8\lib\site-packages\exchange_calendars\exchange_calendar.py:2345: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),


In [2]:
from xtquant import xtdatacenter as xtdc
xtdc.set_token("26e6009f4de3bfb2ae4b89763f255300e96d6912")

print('xtdc.init')
xtdc.init() # 初始化行情模块，加载合约数据，会需要大约十几秒的时间
print('done')
# print('xtdc.listen')
    # addr_list = [
    # '115.231.218.73:55310', 
    # '115.231.218.79:55310', 
    # '42.228.16.211:55300',
    # '42.228.16.210:55300',
    # '36.99.48.20:55300',
    # '36.99.48.21:55300'
    # ]
    # xtdc.set_allow_optmize_address(addr_list)

    # xtdc.set_kline_mirror_enabled(True) 
    
listen_addr = xtdc.listen(port = 58611)
print(f'done, listen_addr:{listen_addr}')

from xtquant import xtdata
xtdata.connect(port=58611)

print('-----连接上了------')
print(xtdata.data_dir)

xtdc.init
done
done, listen_addr:('0.0.0.0', 58611)
***** xtdata连接成功 *****
服务信息: {'tag': 'xtquant', 'version': '1.0'}
服务地址: 127.0.0.1:58611
数据路径: d:\workspace\TradeX\notebook\data\datadir
设置xtdata.enable_hello = False可隐藏此消息

-----连接上了------
None


In [3]:
sl = xtdata.get_stock_list_in_sector('沪深A股')  
len(sl)

5154

In [4]:
def get_first_tick_trade_amount(stock_code, datekey):
    import datetime
    import pandas as pd

    today = datetime.datetime.strptime(datekey, '%Y-%m-%d').date()

    time_0930 = datetime.time(9, 20, 0)

    dt_0930 = datetime.datetime.combine(today, time_0930)

    timestamp_0930 = dt_0930.timestamp()

    time_09305 = datetime.time(9, 26, 0)

    dt_09305 = datetime.datetime.combine(today, time_09305)

    timestamp_09305 = dt_09305.timestamp()

    tims = int(timestamp_0930*1000)

    tims5 = int(timestamp_09305*1000)
    import numpy as np
    n_data_key = datekey.replace('-', '')
    xtdata.download_history_data(stock_code, 'tick', n_data_key, n_data_key)
    all_tick_data = xtdata.get_market_data(stock_list=[stock_code], period='tick', start_time=n_data_key, end_time=n_data_key)

    # 假设 all_tick_data['000759.SZ'] 是 numpy.void 数组
    if isinstance(all_tick_data[stock_code], np.ndarray) and all_tick_data[stock_code].dtype.type is np.void:
        df = pd.DataFrame(all_tick_data[stock_code].tolist(), columns=all_tick_data[stock_code].dtype.names)
    else:
        raise

    filtered_df = df[(df['time'] >= tims) & (df['time'] <= tims5)]

    # 按 time 列升序排序
    sorted_df = filtered_df.sort_values(by='time')

    # 取 time 最小的行
    min_time_row = sorted_df.tail(1)

    amount = min_time_row['amount']

    if len(amount) == 1:
        real_amount = amount.item()
    else:
        print(f"{stock_code}-{datekey}")
        real_amount = 0

    return real_amount

In [7]:
datekey = '20250701'
n_datekey = '20250621'
stock_code = '003029.SZ'
import datetime
import pandas as pd

import numpy as np


xtdata.download_history_data(stock_code, 'announcement', n_datekey, datekey)

ac = xtdata.get_market_data([],[stock_code],period="announcement",start_time = n_datekey, end_time = datekey)
df = pd.DataFrame.from_dict(ac[stock_code])
df


,time,1,2,3,4,5,6,7
0,1750869194865,,吉大正元：关于公司股票交易异常波动的公告,,TXT,http://disc.static.szse.cn/disc/disk03/finalpa...,0,0
1,1750955489956,,吉大正元：关于股东减持股份预披露公告,,TXT,http://disc.static.szse.cn/disc/disk03/finalpa...,0,0
2,1751301169753,,吉大正元：关于首次公开发行前部分已发行股份上市流通的提示性公告,,TXT,http://disc.static.szse.cn/disc/disk03/finalpa...,0,0
3,1751301169873,,吉大正元：招商证券股份有限公司关于吉大正元首次公开发行前部分已发行股份上市流通的核查意见,,TXT,http://disc.static.szse.cn/disc/disk03/finalpa...,0,0


In [34]:
datekey = '20250629'
n_datekey = '20250615'
stock_code = '003029.SZ'
import datetime
import pandas as pd

import numpy as np


xtdata.download_history_data(stock_code, '1d', n_datekey, datekey)

ac = xtdata.get_market_data([],[stock_code],period="1d",start_time = n_datekey, end_time = datekey)
ac['open']

,20250616,20250617,20250618,20250619,20250620,20250623,20250624,20250625,20250626,20250627
003029.SZ,24.77,25.1,23.81,24.5,24.79,24.31,28.78,31.66,34.83,34.68


In [35]:
def convert_dict(ac):
    result = {}
    # 排除 'time' 键
    keys_to_process = [key for key in ac.keys()]
    for key in keys_to_process:
        df = ac[key]
        for stock_code in df.index:
            if stock_code not in result:
                result[stock_code] = pd.DataFrame(index=df.columns).sort_index(ascending=False)
            result[stock_code][key] = df.loc[stock_code]
    return result

convert_dict(ac)['003029.SZ']


,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250627,1750953600000,34.68,36.50,31.49,32.16,661721,2.291216e+09,0.0,15,34.83,0
20250626,1750867200000,34.83,34.83,34.83,34.83,22592,7.868794e+07,0.0,15,31.66,0
20250625,1750780800000,31.66,31.66,31.66,31.66,35188,1.114053e+08,0.0,15,28.78,0
20250624,1750694400000,28.78,28.78,28.78,28.78,25962,7.471864e+07,0.0,15,26.16,0
20250623,1750608000000,24.31,26.16,24.31,26.16,142681,3.687682e+08,0.0,15,23.78,0
20250620,1750348800000,24.79,24.93,23.30,23.78,278297,6.669278e+08,0.0,15,25.69,0
20250619,1750262400000,24.50,27.04,23.24,25.69,416801,1.053573e+09,0.0,15,24.96,0
20250618,1750176000000,23.81,25.37,23.58,24.96,308104,7.612944e+08,0.0,15,24.59,0
20250617,1750089600000,25.10,25.85,24.31,24.59,389253,9.771846e+08,0.0,15,26.43,0
20250616,1750003200000,24.77,27.54,24.11,26.43,534835,1.411739e+09,0.0,15,25.04,0


In [ ]:
from tkinter.font import BOLD


def is_previous_limit_up_and_volume_breakout(df):
    """
    判断是否前一天一字涨停（包含 T 型板、高开封板且低成交量），昨天放量断板的股票
    :param df: 股票数据 DataFrame
    :return: 是否满足条件
    """
    valid_days = df[df['suspendFlag'] == 0].head(7)
    if len(valid_days) < 2:
        return False
    prev_day = valid_days.iloc[1]
    last_day = valid_days.iloc[0]
    
    # 计算涨停价格
    limit_up_price = prev_day['preClose'] * 1.095
    #一字涨停
    prev_limit_up = prev_day['open'] == prev_day['close'] == prev_day['high'] == prev_day['low'] and \
                    (prev_day['close'] / prev_day['preClose'] - 1) >= 0.095
    
    prev_limit_up_no_volume = prev_day['open'] == prev_day['close'] == prev_day['high'] and prev_day['close'] >= limit_up_price

    # 前一天满足 T 型板、高开封板且低成交量条件
    # 高开七八个点以上直接封板且成交量低
    high_open_limit_up = (prev_day['open'] / prev_day['preClose'] - 1 >= 0.07) and \
                         prev_day['close'] >= limit_up_price and \
                         prev_day['close'] == prev_day['high'] and \
                         prev_day['volume'] < df['volume'].mean() * 0.5
    # T 型板且成交量低
    t_shape_limit_up = prev_day['close'] >= limit_up_price and \
                       prev_day['open'] >= limit_up_price and \
                       prev_day['open'] == prev_day['close'] == prev_day['high'] and \
                       prev_day['volume'] < df['volume'].mean() * 0.5
    prev_limit_up = high_open_limit_up or t_shape_limit_up or prev_limit_up
    
    # 昨天放量断板
    volume_breakout = last_day['close'] < last_day['open'] and last_day['close'] <= prev_day['close'] and last_day['volume'] > prev_day['volume'] * 3

    
    volume_breakout2 = last_day['close'] < last_day['open'] and last_day['close'] <= prev_day['close'] and last_day['volume'] > prev_day['volume'] * 8
    res =  (prev_limit_up and volume_breakout) or (volume_breakout2 and prev_limit_up_no_volume)
    return bool(res)

In [ ]:
def is_last_day_limit_up_and_break(df):
    """
    判断昨天是否涨停后炸板的股票，且最后放巨量绿盘收尾，同时最近 7 日内涨幅超过 40%
    :param df: 股票数据 DataFrame
    :return: 是否满足条件
    """
    valid_days = df[df['suspendFlag'] == 0].head(7)
    if len(valid_days) < 7:
        return False
    last_day = valid_days.iloc[0]

    seven_day_lowest = valid_days['low'].min()

    seven_day_growth = (last_day['high'] / seven_day_lowest - 1)
    
    # 涨停后炸板
    limit_up_and_break = last_day['high'] / last_day['preClose'] - 1 >= 0.095 and last_day['close'] < last_day['high']
    # 放巨量绿盘收尾
    huge_volume_green = last_day['volume'] > df['volume'].mean() * 1.5 and last_day['close'] < last_day['open']

    limit_up_price = last_day['preClose'] * 1.095
    # 开盘一字涨停
    open_limit_up = last_day['open'] >= limit_up_price
    # 盘中炸板，收盘未封死涨停
    break_and_not_closed = last_day['low'] < limit_up_price and last_day['close'] < limit_up_price * 0.09

    # 最近 7 日内涨幅超过 40%
    high_growth = seven_day_growth >= 0.35
    
    res =  (limit_up_and_break and huge_volume_green and high_growth) or (high_growth and break_and_not_closed and open_limit_up)
    return bool(res)

In [37]:
is_previous_limit_up_and_volume_breakout(convert_dict(ac)['003029.SZ'])

True

In [ ]:
is_last_day_limit_up_and_break(convert_dict(ac)['003029.SZ'])

False

: 

In [4]:
data = xtdata.get_instrument_detail("000001.SZ")
data

{'ExchangeID': 'SZ',
 'InstrumentID': '000001',
 'InstrumentName': '平安银行',
 'ProductID': '',
 'ProductName': '',
 'ProductType': -1,
 'ExchangeCode': '000001',
 'UniCode': '000001',
 'CreateDate': '0',
 'OpenDate': '19910403',
 'ExpireDate': '99999999',
 'PreClose': 11.370000000000001,
 'SettlementPrice': 11.370000000000001,
 'UpStopPrice': 12.51,
 'DownStopPrice': 10.23,
 'FloatVolume': 19405571850.0,
 'TotalVolume': 19405918198.0,
 'LongMarginRatio': 1.7976931348623157e+308,
 'ShortMarginRatio': 1.7976931348623157e+308,
 'PriceTick': 0.01,
 'VolumeMultiple': 1,
 'MainContract': 2147483647,
 'LastVolume': 2147483647,
 'InstrumentStatus': 0,
 'IsTrading': False,
 'IsRecent': False,
 'ProductTradeQuota': 0,
 'ContractTradeQuota': 0,
 'ProductOpenInterestQuota': 892487424,
 'ContractOpenInterestQuota': 960705840}

In [8]:
import time
xtdata.download_his_st_data()

# 由于download_his_st_data是异步函数，需要确保下载完成

time.sleep(3)

# 获取000004.SZ历史ST情况
xtdata.get_his_st_data('000004.SZ')

{'ST': [['19990427', '20010306'],
  ['20070525', '20090421'],
  ['20100531', '20110608'],
  ['20220506', '20230628']],
 '*ST': [['20060421', '20070525'], ['20090421', '20100531']]}

In [28]:
from xtquant import xtdata

def f(data):
    print(data)

# 获取前需要先订阅，该数据通过指定period = 'snapshotindex'获取
xtdata.subscribe_quote("000001.SZ",period = 'snapshotindex', count = -1,callback = f)

data = xtdata.get_market_data_ex([],["000001.SZ"],period="snapshotindex",start_time = "", end_time = "20240921103100",count = 10)
print(data)

{'000001.SZ':                          time    量比  1分钟涨速(%)  5分钟涨速(%)   3日涨幅(%)   5日涨幅(%)  \
20240920145100  1726815060000  1.08       0.0       0.1  1.855670  2.383420   
20240920145200  1726815120000  1.08      -0.1       0.0  1.752577  2.279793   
20240920145300  1726815180000  1.09       0.1       0.1  1.855670  2.383420   
20240920145400  1726815240000  1.10       0.0       0.0  1.855670  2.383420   
20240920145500  1726815300000  1.12       0.0       0.0  1.855670  2.383420   
20240920145600  1726815360000  1.13       0.0       0.0  1.855670  2.383420   
20240920145700  1726815420000  1.13       0.1       0.2  1.958763  2.487047   
20240920145800  1726815480000  1.13      -0.2      -0.1  1.752577  2.279793   
20240920145900  1726815540000  1.12       0.0      -0.1  1.752577  2.279793   
20240920150000  1726815600000  1.18       0.3       0.2  2.061856  2.590674   

                10日涨幅(%)   3日换手(%)   5日换手(%)  10日换手(%)  
20240920145100 -1.397206  0.944608  1.536342  4.107184  
20

In [29]:
data['000001.SZ']

,time,量比,1分钟涨速(%),5分钟涨速(%),3日涨幅(%),5日涨幅(%),10日涨幅(%),3日换手(%),5日换手(%),10日换手(%)
20240920145100,1726815060000,1.08,0.0,0.1,1.855670,2.383420,-1.397206,0.944608,1.536342,4.107184
20240920145200,1726815120000,1.08,-0.1,0.0,1.752577,2.279793,-1.497006,0.946978,1.538712,4.109554
20240920145300,1726815180000,1.09,0.1,0.1,1.855670,2.383420,-1.397206,0.950241,1.541975,4.112817
20240920145400,1726815240000,1.10,0.0,0.0,1.855670,2.383420,-1.397206,0.956488,1.548222,4.119063
20240920145500,1726815300000,1.12,0.0,0.0,1.855670,2.383420,-1.397206,0.963078,1.554812,4.125653
20240920145600,1726815360000,1.13,0.0,0.0,1.855670,2.383420,-1.397206,0.968243,1.559977,4.130818
20240920145700,1726815420000,1.13,0.1,0.2,1.958763,2.487047,-1.297405,0.970379,1.562113,4.132955
20240920145800,1726815480000,1.13,-0.2,-0.1,1.752577,2.279793,-1.497006,0.970884,1.562618,4.133460
20240920145900,1726815540000,1.12,0.0,-0.1,1.752577,2.279793,-1.497006,0.970884,1.562618,4.133460
20240920150000,1726815600000,1.18,0.3,0.2,2.061856,2.590674,-1.197605,0.992084,1.583818,4.154660


In [30]:
from xtquant import xtdata


def f(data):
    print(data)

period = "limitupperformance"
stock = "000001.SZ"
ls = [stock]
xtdata.download_history_data2(ls,period = period) # 下载历史数据

# 获取前需要先订阅，该数据通过指定period = 'limitupperformance'获取
xtdata.subscribe_quote(stock,period = 'limitupperformance', count = -1,callback = f)

data = xtdata.get_market_data_ex([],[stock],period="limitupperformance",start_time = "", end_time = "",count = 10)
print(data)

{'000001.SZ': [{'time': 1743350400000, 'openVol': 11338, 'closeVol': 9395, 'finalVol': 0, 'startUp': 0, 'endUp': 0, 'breakUp': 0, 'upAmount': 0.0, 'startDn': 0, 'endDn': 0, 'breakDn': 0, 'dnAmount': 0.0, 'direct': 0, 'sealVolRatio': 0.0, 'sealFreeRatio': 0.0, 'bidPreRatio': 1.757977, 'sealCount': 0, 'sealDays': 0, 'sealBreak': 0}]}
{'000001.SZ':                          time  openVol  closeVol  finalVol  startUp  endUp  \
20250318000000  1742227200000    27279     22219         0        0      0   
20250319000000  1742313600000    13188      9286         0        0      0   
20250320000000  1742400000000     7911     16320         0        0      0   
20250321000000  1742486400000     7237     28724         0        0      0   
20250324000000  1742745600000     4854      8543         0        0      0   
20250325000000  1742832000000     7123      6066         0        0      0   
20250326000000  1742918400000     4194     11681         0        0      0   
20250327000000  174300480000

In [31]:
data['000001.SZ']

,time,openVol,closeVol,finalVol,startUp,endUp,breakUp,upAmount,startDn,endDn,breakDn,dnAmount,direct,sealVolRatio,sealFreeRatio,bidPreRatio,sealCount,sealDays,sealBreak
20250318000000,1742227200000,27279,22219,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.592556,0,0,0
20250319000000,1742313600000,13188,9286,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.821534,0,0,0
20250320000000,1742400000000,7911,16320,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.580643,0,0,0
20250321000000,1742486400000,7237,28724,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.657218,0,0,0
20250324000000,1742745600000,4854,8543,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.352662,0,0,0
20250325000000,1742832000000,7123,6066,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.611638,0,0,0
20250326000000,1742918400000,4194,11681,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.570140,0,0,0
20250327000000,1743004800000,2960,10693,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.399534,0,0,0
20250328000000,1743091200000,2092,14331,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,0.378062,0,0,0
20250331000000,1743350400000,11338,9395,0,0,0,0,0.0,0,0,0,0.0,0,0.0,0.0,1.757977,0,0,0


In [33]:
from xtquant import xtdata

# 获取历史数据前，请确保已经下载历史数据
xtdata.download_history_data("000001.SZ",period="transactioncount1d")
xtdata.download_history_data("000582.SZ",period="transactioncount1d")

# 获取一只股票在一个时间段内的资金流量数据
data1 = xtdata.get_market_data_ex([],["000001.SZ"],period="transactioncount1d",start_time = "20230101", end_time = "20231009")

# 获取多只股票在一个时间段内的资金流向数据
data2 = xtdata.get_market_data_ex([],["000001.SZ","000582.SZ"],period="transactioncount1d",start_time = "20230101", end_time = "20231009")
# 获取多只股票在某一天的资金流向数据
data3 = xtdata.get_market_data_ex([],["000001.SZ","000582.SZ"],period="transactioncount1d",start_time = "20231009", end_time = "20231009")

In [35]:
data1['000001.SZ']

,time,bidNumber,bidMostVolume,bidBigVolume,bidMediumVolume,bidSmallVolume,offNumber,offMostVolume,offBigVolume,offMediumVolume,...,unactiveOffSmallVolumeDx,unactiveBidMostAmountDx,unactiveBidBigAmountDx,unactiveBidMediumAmountDx,unactiveBidSmallAmountDx,unactiveOffMostAmountDx,unactiveOffBigAmountDx,unactiveOffMediumAmountDx,unactiveOffSmallAmountDx,transactionNumber
20230103000000,1672675200000,2483,751916,414495,62421,2695,2269,542998,355429,61497,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230104000000,1672761600000,2072,814330,303116,51841,2097,2666,572405,368774,73485,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230105000000,1672848000000,2322,490688,261504,65825,4168,2392,510222,262012,66348,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230106000000,1672934400000,2654,317055,278677,92047,3932,2069,195899,236979,67547,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230109000000,1673193600000,2614,236244,260462,88034,5164,2103,184584,208733,70353,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230925000000,1695571200000,1623,231467,74114,43191,10924,2505,187342,122762,72830,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230926000000,1695657600000,2062,67169,55677,51364,17352,2249,116657,60107,56529,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230927000000,1695744000000,2387,80378,88149,68916,16988,1625,125014,55978,43881,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20230928000000,1695830400000,2290,84649,93402,77252,15067,1817,106822,74914,56200,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [36]:
from xtquant import xtdata
# 订单流数据仅提供1m周期数据下载，其他周期的订单流数据都是通过1m周期合成的
period = "orderflow1m"
# 下载000001.SZ的1m订单流数据
xtdata.download_history_data("000001.SZ",period=period)
# 获取000001.SZ的1m订单流数据
xtdata.get_market_data_ex([],["000001.SZ"],period=period)["000001.SZ"]

,time,price,buyNum,sellNum
20220329093000,1648517400000,[14.88],[0],[2752]
20220329093100,1648517460000,"[14.85, 14.870000000000001, 14.88, 14.89, 14.9...","[0, 154, 126, 231, 1263, 1118, 344, 200]","[590, 2368, 0, 644, 0, 1174, 0, 165]"
20220329093200,1648517520000,"[14.82, 14.83, 14.84, 14.85, 14.86]","[0, 245, 1710, 0, 44]","[444, 201, 1624, 171, 94]"
20220329093300,1648517580000,"[14.8, 14.81, 14.82, 14.83, 14.84]","[0, 0, 1537, 62, 74]","[986, 1026, 563, 0, 0]"
20220329093400,1648517640000,"[14.81, 14.82, 14.83, 14.85, 14.88, 14.89, 14....","[0, 769, 46, 223, 441, 206, 0, 1033, 671]","[2866, 940, 0, 0, 0, 0, 1581, 0, 0]"
...,...,...,...,...
20250331145600,1743404160000,"[11.27, 11.28]","[0, 1807]","[1529, 0]"
20250331145700,1743404220000,"[11.27, 11.28]","[818, 2959]","[5831, 0]"
20250331145800,1743404280000,[11.28],[36],[0]
20250331145900,1743404340000,[11.28],[0],[0]


In [42]:
# 下载000001.SZ的orderflow1m，以获取历史数据
# orderflow仅提供1m周期进行下载，其他周期皆在系统底层通过1m订单流数据进行合成给出
xtdata.download_history_data("002520.SZ",period="orderflow1m")


# 获取000001.SZ，1m订单流数据
period = "orderflow1m"
data1 = xtdata.get_market_data_ex([],["002520.SZ"],period=period)["002520.SZ"]

# 获取000001.SZ, 5m订单流数据
period = "orderflow5m"
data2 = xtdata.get_market_data_ex([],["002520.SZ"],period=period)["002520.SZ"]

# 获取000001.SZ 1d订单流数据
period = "orderflow1d"
data3 = xtdata.get_market_data_ex([],["002520.SZ"],period=period)["002520.SZ"]

# 订阅实时000001.SZ 1m订单流数据
period = "orderflow1m"

# 进行数据订阅
xtdata.subscribe_quote("002520.SZ", period = period)
# 获取订阅后的实时数据
data4 = xtdata.get_market_data_ex([],["002520.SZ"],period=period)["002520.SZ"]

print(data1)
print(data2)
print(data3)

print(data4)

                         time  \
20220329093000  1648517400000   
20220329093100  1648517460000   
20220329093200  1648517520000   
20220329093300  1648517580000   
20220329093400  1648517640000   
...                       ...   
20250331145600  1743404160000   
20250331145700  1743404220000   
20250331145800  1743404280000   
20250331145900  1743404340000   
20250331150000  1743404400000   

                                                            price  \
20220329093000                                             [6.53]   
20220329093100  [6.54, 6.55, 6.5600000000000005, 6.57, 6.58, 6...   
20220329093200             [6.58, 6.59, 6.6000000000000005, 6.61]   
20220329093300  [6.5600000000000005, 6.57, 6.58, 6.59, 6.60000...   
20220329093400             [6.54, 6.55, 6.5600000000000005, 6.57]   
...                                                           ...   
20250331145600                                 [7.43, 7.44, 7.45]   
20250331145700    [7.43, 7.44, 7.45, 7.46, 7.47, 7.

In [54]:
from xtquant import xtdata
xtdata.download_history_data("000001.SZ",period="interactiveqa")
data = xtdata.get_market_data_ex([],["000001.SZ"],period="interactiveqa")
print(data)

{'000001.SZ':               time                 问答编号           问题时间  \
0    1688572800000  1477967097550430208  1686794016000   
1    1688572800001  1481018439885479936  1687149238000   
2    1688572800002  1486238955455750144  1687756986000   
3    1688572800003  1492863495831379968  1688528184000   
4    1688572800004  1480984724391051264  1687145313000   
..             ...                  ...            ...   
294  1739894400004  1930823578698133504  1739513448000   
295  1739894400005  1931092864557658112  1739544798000   
296  1739894400006  1934362940987682816  1739925484000   
297  1739894400007  1932629981813288960  1739723742000   
298  1739894400008  1930860051560411136  1739517695000   

                                                  问题内容           回答时间  \
0                                    公司认为经营银行的长期主义是什么？  1688605558000   
1                                         请问现在的股东人数是多少  1688605821000   
2                              贵公司分红率为什么这么低，是否可以加大分红比率  1688606076000 

In [55]:
data["000001.SZ"]

,time,问答编号,问题时间,问题内容,回答时间,回答内容
0,1688572800000,1477967097550430208,1686794016000,公司认为经营银行的长期主义是什么？,1688605558000,本行以“中国最卓越、全球领先的智能化零售银行”为战略目标，坚持“科技引领、零售突破、对公做精...
1,1688572800001,1481018439885479936,1687149238000,请问现在的股东人数是多少,1688605821000,"您好，截至2023年一季度末，本行股东总户数为506,867户。感谢您对我行的关注。"
2,1688572800002,1486238955455750144,1687756986000,贵公司分红率为什么这么低，是否可以加大分红比率,1688606076000,您好！本行于2021年4月8日召开的2020年年度股东大会审议通过了《平安银行股份有限公司2...
3,1688572800003,1492863495831379968,1688528184000,建议平安私有化平安银行，这样的估值没有必要留在资本市场，平安也受益。,1688606100000,感谢您对我行的关注和建议。
4,1688572800004,1480984724391051264,1687145313000,公司手续费佣金收入年年下降，有没有什么办法改善？,1688606510000,2022年，本集团手续费及佣金净收入302.08亿元，主要受宏观环境等因素影响，未来，本行将...
...,...,...,...,...,...,...
294,1739894400004,1930823578698133504,1739513448000,请问董秘2024年的业绩快报何时发布？,1739952356000,您好。本行2024年年度报告计划于2025年3月15日发布，感谢您的关注。
295,1739894400005,1931092864557658112,1739544798000,贵公司有没有投资参与重组？方信证券,1739953379000,您好。本行无应披露而未披露的信息。感谢您的关注。
296,1739894400006,1934362940987682816,1739925484000,请问股东人数，为啥不回复,1739953401000,"您好。本行在定期报告中披露具体股东数。截至2024年9月末，本行普通股股东总户数为517,6..."
297,1739894400007,1932629981813288960,1739723742000,目前渠道了解，平安银行很多员工离职，其中有一个情况是总行那边缩小放贷额度，员工帮客户申请贷款...,1739953429000,您好。在零售业务方面，本行顺应外部经营环境变化，持续主动调整优化贷款业务结构，为中长期业务健...


In [57]:
from xtquant import xtdata
# 取数据前请确保已下载所需要的财务数据
xtdata.download_financial_data(["000001.SZ","600519.SH","430017.BJ"], table_list=["Balance","Income"])
x = xtdata.get_financial_data(["000001.SZ","600519.SH","430017.BJ"],["Balance","Income"])

In [60]:
x['000001.SZ']['Income']

,m_timetag,m_anntime,revenue_inc,earned_premium,real_estate_sales_income,total_operating_cost,real_estate_sales_cost,research_expenses,surrender_value,net_payments,...,m_paymentsInsuranceClaims,m_amortizedCompensationExpenses,m_netReserveInsuranceLiability,m_policyReserve,m_amortizeInsuranceReserve,m_nsuranceFeesCommissionExpenses,m_operationAdministrativeExpense,m_amortizedReinsuranceExpenditure,m_netProfitLossdisposalNonassets,m_otherItemsAffectingNetProfit
0,19911231,19920430,3.346900e+08,0.0,NaN,1.882300e+08,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19921231,19930226,4.755108e+08,0.0,NaN,2.546128e+08,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19930630,19930805,2.664955e+08,0.0,NaN,0.000000e+00,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19931231,19940329,NaN,0.0,NaN,1.146220e+08,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19940630,19940630,NaN,0.0,NaN,9.915458e+07,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,20231231,20240315,1.646990e+11,NaN,NaN,1.067710e+11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,20240331,20240420,3.877000e+10,NaN,NaN,2.021600e+10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,20240630,20240816,7.713200e+10,NaN,NaN,4.504500e+10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,20240930,20241019,1.115820e+11,NaN,NaN,6.371300e+10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from xtquant import xtdata
xtdata.download_sector_data()

***** xtdata连接成功 *****
服务信息: {'tag': 'xtquant', 'version': '1.0'}
服务地址: 127.0.0.1:58609
数据路径: d:\workspace\TradeX\notebook\data\datadir
设置xtdata.enable_hello = False可隐藏此消息



In [62]:
xtdata.download_history_contracts()

In [5]:
hy_list = xtdata.get_sector_list()

In [8]:
thy_list = []
for hy in hy_list:
    if 'THY' in hy:
        thy_list.append(hy)

In [11]:
thy_list

['THY1交通运输',
 'THY1传媒',
 'THY1公用事业',
 'THY1农林牧渔',
 'THY1医药生物',
 'THY1商贸零售',
 'THY1国防军工',
 'THY1基础化工',
 'THY1家用电器',
 'THY1建筑材料',
 'THY1建筑装饰',
 'THY1房地产',
 'THY1有色金属',
 'THY1机械设备',
 'THY1汽车',
 'THY1煤炭',
 'THY1环保',
 'THY1电力设备',
 'THY1电子',
 'THY1石油石化',
 'THY1社会服务',
 'THY1纺织服饰',
 'THY1综合',
 'THY1美容护理',
 'THY1计算机',
 'THY1轻工制造',
 'THY1通信',
 'THY1钢铁',
 'THY1银行',
 'THY1非银金融',
 'THY1食品饮料',
 'THY2IT服务',
 'THY2专用设备',
 'THY2中药',
 'THY2互联网电商',
 'THY2仪器仪表',
 'THY2保险',
 'THY2元件',
 'THY2光伏设备',
 'THY2光学光电子',
 'THY2公路铁路运输',
 'THY2其他电子',
 'THY2其他电源设备',
 'THY2其他社会服务',
 'THY2养殖业',
 'THY2军工电子',
 'THY2军工装备',
 'THY2农业服务',
 'THY2农产品加工',
 'THY2农化制品',
 'THY2包装印刷',
 'THY2化学制品',
 'THY2化学制药',
 'THY2化学原料',
 'THY2化学纤维',
 'THY2化工合成材料',
 'THY2医疗器械',
 'THY2医疗服务',
 'THY2医药商业',
 'THY2半导体',
 'THY2厨卫电器',
 'THY2塑料制品',
 'THY2多元金融',
 'THY2家居用品',
 'THY2小家电',
 'THY2小金属',
 'THY2工业金属',
 'THY2工程机械',
 'THY2建筑材料',
 'THY2建筑装饰',
 'THY2影视院线',
 'THY2房地产',
 'THY2房地产服务',
 'THY2教育',
 'THY2文化传媒',
 'THY2旅游及酒店',
 'THY2服装家纺',
 'THY2机场航运',
 'THY2

In [13]:
for x in thy_list:
    if '房地产' in x:
        print(x)

THY1房地产
THY2房地产
THY2房地产服务
THY3房地产服务


In [16]:
xt_sector_index_list = xtdata.get_stock_list_in_sector("THY2房地产")

In [17]:
xt_sector_index_list

['000002.SZ',
 '000006.SZ',
 '000011.SZ',
 '000014.SZ',
 '000029.SZ',
 '000031.SZ',
 '000036.SZ',
 '000042.SZ',
 '000090.SZ',
 '000402.SZ',
 '000514.SZ',
 '000517.SZ',
 '000560.SZ',
 '000573.SZ',
 '000608.SZ',
 '000609.SZ',
 '000620.SZ',
 '000656.SZ',
 '000668.SZ',
 '000718.SZ',
 '000736.SZ',
 '000797.SZ',
 '000838.SZ',
 '000863.SZ',
 '000886.SZ',
 '000897.SZ',
 '000909.SZ',
 '000926.SZ',
 '000965.SZ',
 '001914.SZ',
 '001979.SZ',
 '002016.SZ',
 '002133.SZ',
 '002146.SZ',
 '002188.SZ',
 '002208.SZ',
 '002244.SZ',
 '002285.SZ',
 '002305.SZ',
 '002314.SZ',
 '002968.SZ',
 '300917.SZ',
 '600007.SH',
 '600048.SH',
 '600064.SH',
 '600082.SH',
 '600094.SH',
 '600159.SH',
 '600162.SH',
 '600173.SH',
 '600208.SH',
 '600246.SH',
 '600266.SH',
 '600322.SH',
 '600325.SH',
 '600340.SH',
 '600376.SH',
 '600383.SH',
 '600463.SH',
 '600503.SH',
 '600510.SH',
 '600533.SH',
 '600603.SH',
 '600604.SH',
 '600606.SH',
 '600622.SH',
 '600638.SH',
 '600639.SH',
 '600641.SH',
 '600648.SH',
 '600649.SH',
 '6006

In [19]:
xt_sector_index_info =xtdata.get_instrument_detail("THY2房地产")

In [ ]:
from xtquant import xtdata
# 下载权重相关信息
xtdata.download_index_weight()
# 获取权重相关信息
ret_weight_data = xtdata.get_index_weight('000300.SH')
print(ret_weight_data)

In [23]:
xt_sector_index_list = xtdata.get_stock_list_in_sector("迅投一级行业板块加权指数")
xt_sector_index_info = xtdata.get_instrument_detail(xt_sector_index_list[0])
xt_sector_index_info

{'ExchangeID': 'BKZS',
 'InstrumentID': '260992',
 'InstrumentName': 'SW1农林牧渔加权',
 'ProductID': '',
 'ProductName': '',
 'ProductType': -1,
 'ExchangeCode': '260992',
 'UniCode': '260992',
 'CreateDate': '0',
 'OpenDate': '0',
 'ExpireDate': '0',
 'PreClose': 24190.13,
 'SettlementPrice': 1.7976931348623157e+308,
 'UpStopPrice': 1.7976931348623157e+308,
 'DownStopPrice': 1.7976931348623157e+308,
 'FloatVolume': 1.7976931348623157e+308,
 'TotalVolume': 1.7976931348623157e+308,
 'LongMarginRatio': 1.7976931348623157e+308,
 'ShortMarginRatio': 1.7976931348623157e+308,
 'PriceTick': 0.01,
 'VolumeMultiple': 1,
 'MainContract': 2147483647,
 'LastVolume': 2147483647,
 'InstrumentStatus': 2147483647,
 'IsTrading': False,
 'IsRecent': False,
 'ProductTradeQuota': None,
 'ContractTradeQuota': None,
 'ProductOpenInterestQuota': None,
 'ContractOpenInterestQuota': None}

In [25]:
xt_sector_index_list[0]

'260992.BKZS'

In [84]:
# coding=utf-8
from xtquant import xtdata
# 获取迅投板块指数代码列表
xt_sector_index_list = xtdata.get_stock_list_in_sector("迅投一级行业板块加权指数")
# 获取迅投板块指数合约信息
xt_sector_index_info = xtdata.get_instrument_detail(xt_sector_index_list[0])
xt_sector_index = xt_sector_index_list[0]
print(xt_sector_index_info)
# 订阅合约数据
xtdata.subscribe_quote(xt_sector_index, period='1d', start_time='', end_time='20231026', count=1, callback=None)
# 下载指定合约历史行情
xtdata.download_history_data(xt_sector_index, '1d', '20231020', '20231026')
# 获取指定合约历史行情
day_data = xtdata.get_market_data_ex(field_list=[], stock_list=[xt_sector_index], period='1d', start_time='',end_time='20231026',  count=5, dividend_type='none', fill_data=True)
print(day_data)

{'ExchangeID': 'BKZS', 'InstrumentID': '260992', 'InstrumentName': 'SW1农林牧渔加权', 'ProductID': '', 'ProductName': '', 'ProductType': -1, 'ExchangeCode': '260992', 'UniCode': '260992', 'CreateDate': '0', 'OpenDate': '0', 'ExpireDate': '0', 'PreClose': 24242.59, 'SettlementPrice': 1.7976931348623157e+308, 'UpStopPrice': 1.7976931348623157e+308, 'DownStopPrice': 1.7976931348623157e+308, 'FloatVolume': 1.7976931348623157e+308, 'TotalVolume': 1.7976931348623157e+308, 'LongMarginRatio': 1.7976931348623157e+308, 'ShortMarginRatio': 1.7976931348623157e+308, 'PriceTick': 0.01, 'VolumeMultiple': 1, 'MainContract': 2147483647, 'LastVolume': 2147483647, 'InstrumentStatus': 2147483647, 'IsTrading': False, 'IsRecent': False, 'ProductTradeQuota': None, 'ContractTradeQuota': None, 'ProductOpenInterestQuota': None, 'ContractOpenInterestQuota': None}
{'260992.BKZS':                    time      open      high       low     close    volume  \
20231020  1697731200000  24602.81  24703.36  24401.10  24472.14 

In [85]:
xt_sector_index_list = xtdata.get_stock_list_in_sector("迅投一级行业板块加权指数")

In [ ]:
len(xt_sector_index_list)

31

: 

In [28]:
# coding=utf-8
from xtquant import xtdata
# 获取迅投板块指数代码列表
xt_sector_index_list = xtdata.get_stock_list_in_sector("迅投一级行业板块加权指数")
# 获取迅投板块指数合约信息
xt_sector_index_info = xtdata.get_instrument_detail(xt_sector_index_list[0])
xt_sector_index = xt_sector_index_list[0]
print(xt_sector_index_info)
# 订阅合约数据
xtdata.subscribe_quote(xt_sector_index, period='1m', start_time='', end_time='20250403', count=1, callback=None)
# 下载指定合约历史行情
xtdata.download_history_data(xt_sector_index, '1m', '20250403', '20250403')
# 获取指定合约历史行情
day_data = xtdata.get_market_data_ex(field_list=[], stock_list=[xt_sector_index], period='1m', start_time='',end_time='20250403',  count=10, dividend_type='none', fill_data=True)
print(day_data)

{'ExchangeID': 'BKZS', 'InstrumentID': '260992', 'InstrumentName': 'SW1农林牧渔加权', 'ProductID': '', 'ProductName': '', 'ProductType': -1, 'ExchangeCode': '260992', 'UniCode': '260992', 'CreateDate': '0', 'OpenDate': '0', 'ExpireDate': '0', 'PreClose': 24190.13, 'SettlementPrice': 1.7976931348623157e+308, 'UpStopPrice': 1.7976931348623157e+308, 'DownStopPrice': 1.7976931348623157e+308, 'FloatVolume': 1.7976931348623157e+308, 'TotalVolume': 1.7976931348623157e+308, 'LongMarginRatio': 1.7976931348623157e+308, 'ShortMarginRatio': 1.7976931348623157e+308, 'PriceTick': 0.01, 'VolumeMultiple': 1, 'MainContract': 2147483647, 'LastVolume': 2147483647, 'InstrumentStatus': 2147483647, 'IsTrading': False, 'IsRecent': False, 'ProductTradeQuota': None, 'ContractTradeQuota': None, 'ProductOpenInterestQuota': None, 'ContractOpenInterestQuota': None}
{'260992.BKZS':                          time      open      high       low     close  volume  \
20250403145100  1743663060000  24596.24  24596.90  24587.18 

In [29]:
day_data['260992.BKZS']

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250403145100,1743663060000,24596.24,24596.90,24587.18,24591.30,114100,9.939985e+07,0.0,13,24597.36,0
20250403145200,1743663120000,24590.91,24595.43,24585.34,24586.87,115499,8.422923e+07,0.0,13,24591.30,0
20250403145300,1743663180000,24589.78,24600.03,24589.41,24600.03,118162,8.925733e+07,0.0,13,24586.87,0
20250403145400,1743663240000,24599.09,24611.00,24596.79,24608.22,129436,1.026944e+08,0.0,13,24600.03,0
20250403145500,1743663300000,24605.04,24615.69,24605.04,24609.71,156309,1.259561e+08,0.0,13,24608.22,0
20250403145600,1743663360000,24609.05,24617.73,24608.82,24612.91,189835,1.400905e+08,0.0,13,24609.71,0
20250403145700,1743663420000,24618.47,24624.08,24614.43,24621.64,207233,1.671847e+08,0.0,13,24612.91,0
20250403145800,1743663480000,24620.22,24621.44,24620.22,24621.44,11169,8.559598e+06,0.0,13,24621.64,0
20250403145900,1743663540000,24621.44,24621.44,24621.44,24621.44,0,0.000000e+00,0.0,13,24621.44,0
20250403150000,1743663600000,24631.48,24633.86,24631.48,24632.93,310379,2.456740e+08,0.0,15,24621.44,0


In [31]:
names = []
for x in xt_sector_index_list:
    xt_sector_index_info = xtdata.get_instrument_detail(x)
    print(xt_sector_index_info['InstrumentName'])
    names.append(xt_sector_index_info['InstrumentName'])

SW1农林牧渔加权
SW1基础化工加权
SW1钢铁加权
SW1有色金属加权
SW1电子加权
SW1汽车加权
SW1家用电器加权
SW1食品饮料加权
SW1纺织服饰加权
SW1轻工制造加权
SW1医药生物加权
SW1公用事业加权
SW1交通运输加权
SW1房地产加权
SW1商贸零售加权
SW1社会服务加权
SW1银行加权
SW1非银金融加权
SW1综合加权
SW1建筑材料加权
SW1建筑装饰加权
SW1电力设备加权
SW1机械设备加权
SW1国防军工加权
SW1计算机加权
SW1传媒加权
SW1通信加权
SW1煤炭加权
SW1石油石化加权
SW1环保加权
SW1美容护理加权


In [32]:
names

['SW1农林牧渔加权',
 'SW1基础化工加权',
 'SW1钢铁加权',
 'SW1有色金属加权',
 'SW1电子加权',
 'SW1汽车加权',
 'SW1家用电器加权',
 'SW1食品饮料加权',
 'SW1纺织服饰加权',
 'SW1轻工制造加权',
 'SW1医药生物加权',
 'SW1公用事业加权',
 'SW1交通运输加权',
 'SW1房地产加权',
 'SW1商贸零售加权',
 'SW1社会服务加权',
 'SW1银行加权',
 'SW1非银金融加权',
 'SW1综合加权',
 'SW1建筑材料加权',
 'SW1建筑装饰加权',
 'SW1电力设备加权',
 'SW1机械设备加权',
 'SW1国防军工加权',
 'SW1计算机加权',
 'SW1传媒加权',
 'SW1通信加权',
 'SW1煤炭加权',
 'SW1石油石化加权',
 'SW1环保加权',
 'SW1美容护理加权']

In [34]:
names[:20]

['SW1农林牧渔加权',
 'SW1基础化工加权',
 'SW1钢铁加权',
 'SW1有色金属加权',
 'SW1电子加权',
 'SW1汽车加权',
 'SW1家用电器加权',
 'SW1食品饮料加权',
 'SW1纺织服饰加权',
 'SW1轻工制造加权',
 'SW1医药生物加权',
 'SW1公用事业加权',
 'SW1交通运输加权',
 'SW1房地产加权',
 'SW1商贸零售加权',
 'SW1社会服务加权',
 'SW1银行加权',
 'SW1非银金融加权',
 'SW1综合加权',
 'SW1建筑材料加权']

In [10]:
import time
def my_download(stock_list:list,period:str,start_date = '', end_date = ''):
    '''
    用于显示下载进度
    '''
    import string
    
    if [i for i in ["d","w","mon","q","y",] if i in period]:
        period = "1d"
    elif "m" in period:
        numb = period.translate(str.maketrans("", "", string.ascii_letters))
        if int(numb) < 5:
            period = "1m"
        else:
            period = "5m"
    elif "tick" == period:
        pass
    else:
        raise KeyboardInterrupt("周期传入错误")


    n = 1
    num = len(stock_list)
    for i in stock_list:
        print(f"当前正在下载 {period} {n}/{num}")
        
        xtdata.download_history_data(i,period,start_date, end_date)
        n += 1
    print("下载任务结束")

def do_subscribe_quote(stock_list:list, period:str):
  for i in stock_list:
    xtdata.subscribe_quote(i,period = period)
  time.sleep(1) # 等待订阅完成



start_date = '20250401'# 格式"YYYYMMDD"，开始下载的日期，date = ""时全量下载
end_date = "" 
period = "1d" 

need_download = 1  # 取数据是空值时，将need_download赋值为1，确保正确下载了历史数据

code_list = ["000001.SZ", "600519.SH"] # 股票列表

if need_download: # 判断要不要下载数据, gmd系列函数都是从本地读取历史数据,从服务器订阅获取最新数据
    my_download(code_list, period, start_date, end_date)

############ 仅获取历史行情 #####################
count = -1 # 设置count参数，使gmd_ex返回全部数据
data1 = xtdata.get_market_data_ex([],code_list,period = period, start_time = start_date, end_time = end_date)

############ 仅获取最新行情 #####################
do_subscribe_quote(code_list,period)# 设置订阅参数，使gmd_ex取到最新行情
count = 1 # 设置count参数，使gmd_ex仅返回最新行情数据
data2 = xtdata.get_market_data_ex([],code_list,period = period, start_time = start_date, end_time = end_date, count = 1) # count 设置为1，使返回值只包含最新行情

############ 获取历史行情+最新行情 #####################
do_subscribe_quote(code_list,period) # 设置订阅参数，使gmd_ex取到最新行情
count = -1 # 设置count参数，使gmd_ex返回全部数据
data3 = xtdata.get_market_data_ex([],code_list,period = period, start_time = start_date, end_time = end_date, count = -1) # count 设置为1，使返回值只包含最新行情


print(data1[code_list[0]].tail())# 行情数据查看
print(data2[code_list[0]].tail())
print(data3[code_list[0]].tail())

当前正在下载 1d 1/2
当前正在下载 1d 2/2
下载任务结束
                   time   open   high    low  close  volume        amount  \
20250401  1743436800000  11.27  11.30  11.22  11.27  681470  7.674241e+08   
20250402  1743523200000  11.25  11.41  11.25  11.37  932642  1.058747e+09   
20250403  1743609600000  11.31  11.39  11.30  11.34  643914  7.303758e+08   

          settelementPrice  openInterest  preClose  suspendFlag  
20250401               0.0            15     11.26            0  
20250402               0.0            15     11.27            0  
20250403               0.0            15     11.37            0  
                   time   open   high   low  close  volume        amount  \
20250403  1743609600000  11.31  11.39  11.3  11.34  643914  7.303758e+08   

          settelementPrice  openInterest  preClose  suspendFlag  
20250403               0.0            15     11.37            0  
                   time   open   high    low  close  volume        amount  \
20250401  1743436800000  11.27

In [13]:
data2[
'000001.SZ'
]

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250403,1743609600000,11.31,11.39,11.3,11.34,643914,7.303758e+08,0.0,15,11.37,0


In [15]:
a = data2[
'000001.SZ'
]

In [27]:
open = float(a.loc['20250403']['open'])
close = float(a.loc['20250403']['close'])
preClose = float(a.loc['20250403']['preClose'])
high = float(a.loc['20250403']['high'])

In [28]:
close/preClose - 1

-0.0026385224274407815

In [29]:
from xtquant import xtdata
stock = '000001.SZ'

tick = xtdata.get_full_tick([stock])[stock]
info = xtdata.get_instrument_detail(stock)

vol = tick["volume"]
total_volume = info["TotalVolume"]
turnover_rate = vol* 100/total_volume

In [30]:
turnover_rate

0.0033181320947048135

In [31]:
tick

{'time': 1743663600000,
 'timetag': '20250403 15:00:00',
 'lastPrice': 11.34,
 'open': 11.31,
 'high': 11.39,
 'low': 11.3,
 'lastClose': 11.37,
 'amount': 730375800,
 'volume': 643914,
 'pvolume': 64391391,
 'tickvol': 6596,
 'stockStatus': 5,
 'openInt': 15,
 'transactionNum': 31803,
 'settlementPrice': 0,
 'lastSettlementPrice': 0,
 'pe': 0,
 'askPrice': [11.35, 11.36, 11.37, 11.38, 11.39],
 'bidPrice': [11.34, 11.33, 11.32, 11.31, 11.3],
 'askVol': [4110, 9849, 11893, 7426, 13122],
 'bidVol': [9808, 13687, 13784, 13328, 19227]}

In [32]:
info

{'ExchangeID': 'SZ',
 'InstrumentID': '000001',
 'InstrumentName': '平安银行',
 'ProductID': '',
 'ProductName': '',
 'ProductType': -1,
 'ExchangeCode': '000001',
 'UniCode': '000001',
 'CreateDate': '0',
 'OpenDate': '19910403',
 'ExpireDate': '99999999',
 'PreClose': 11.370000000000001,
 'SettlementPrice': 11.370000000000001,
 'UpStopPrice': 12.51,
 'DownStopPrice': 10.23,
 'FloatVolume': 19405571850.0,
 'TotalVolume': 19405918198.0,
 'LongMarginRatio': 1.7976931348623157e+308,
 'ShortMarginRatio': 1.7976931348623157e+308,
 'PriceTick': 0.01,
 'VolumeMultiple': 1,
 'MainContract': 2147483647,
 'LastVolume': 2147483647,
 'InstrumentStatus': 0,
 'IsTrading': False,
 'IsRecent': False,
 'ProductTradeQuota': 0,
 'ContractTradeQuota': 0,
 'ProductOpenInterestQuota': 892487424,
 'ContractOpenInterestQuota': 960705840}

In [33]:
from jqdatasdk import *

In [34]:
auth('18801110136','Wanghw@0309') #账号是申请时所填写的手机号；密码为聚宽官网登录密码


auth success 


In [37]:
from jqdatasdk.technical_analysis import *

In [ ]:
# 定义股票池列表
security_list1 = '000001.XSHE'
security_list2 = ['000001.XSHE','000002.XSHE','601211.XSHG','603177.XSHG']


# 计算并输出 security_list1 的 GDX 值，分别返回：济安线、压力线和支撑线的值。
gdx_jax, gdx_ylx, gdx_zcx = GDX(security_list1,check_date='2017-01-04', N = 30, M = 9)
print (gdx_jax[security_list1])
print (gdx_ylx[security_list1])
print (gdx_zcx[security_list1])

# 输出 security_list2 的 GDX 值
gdx_jax, gdx_ylx, gdx_zcx = GDX(security_list2,check_date='2017-01-04', N = 30, M = 9)
for stock in security_list2:
    print (gdx_jax[stock])
    print (gdx_ylx[stock])
    print (gdx_zcx[stock])

Exception: 您的账号权限仅能获取2023-12-26至2025-01-01的数据，请调整时间参数后重试。如需更长时间的数据范围，可联系运营咨询采购

: 

In [5]:
buy_volume = 400
buy_total_coef = 0.6

In [ ]:
int(round(buy_volume // 100 * buy_total_coef))

2

: 

In [2]:
a[1] =12

In [28]:
current_max_increase = 0
dynamic_hc_stop_profit_thres = 0.6
current_max_price = 16

In [29]:
dynamic_zs_line = (1 - ((10 - current_max_increase * 100) * dynamic_hc_stop_profit_thres) / 100) * current_max_price
dynamic_zs_line

15.04

In [30]:
15.04/16

0.94

In [31]:
import datetime

In [32]:
# 获取当前时间
now = datetime.datetime.now().time()
# 定义目标时间
target_time = datetime.time(11, 25)
# 判断当前时间是否超过 11:25
if now > target_time:
    print(now)

20:09:06.937681


In [ ]:
10 % 5

0

: 

In [ ]:
import requests
import brotli

url = "https://p-xcapi.topxlc.com/stock/get_system_time"
headers = {
    "authority": "p-xcapi.topxlc.com",
    "method": "POST",
    "scheme": "https",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "zh-CN,zh;q=0.9",
    "Content-Length": "0",
    "Origin": "https://www.topxlc.com",
    "Referer": "https://www.topxlc.com/",
    "Sec-Ch-Ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
data = {}

import requests
import brotli
import json

# ... existing code ...

try:
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        if response.headers.get('Content-Encoding') == 'br':
            decompressed_data = brotli.decompress(response.content)
            decoded_data = decompressed_data.decode('utf-8')
            try:
                # Replace eval with json.loads
                result = json.loads(decoded_data)
                print("d: ", result)
            except json.JSONDecodeError:
                print(decoded_data)
        else:
            try:
                result = response.json()
                print(result)
            except ValueError:
                print(response.text)
    else:
        print(f"Request failed with status code: {response.status_code}")
except requests.RequestException as e:
    print(f"An error occurred during the request: {e}")

d:  {'version': '1.0', 'requestId': None, 'code': 8200, 'msg': None, 'errcode': None, 'errmsg': None, 'result': 1745656613860, 'ok': True}


: 

In [3]:
from collections import deque

class SmoothFilter:
    def __init__(self, window_size=10):
        self.window = deque(maxlen=window_size)  # 滑动窗口
        self.smoothed_value = 0
        
    def update(self, new_value):
        self.window.append(new_value)
        self.smoothed_value = sum(self.window)/len(self.window)
        return self.smoothed_value

# 使用示例
filter = SmoothFilter(window_size=3)  # 20个tick的窗口

def get_smoothed_volume(x):
    raw_vol = x # 原始tick数据
    return filter.update(raw_vol)

In [4]:
get_smoothed_volume(1)

1.0

In [ ]:
get_smoothed_volume(4)

3.0

: 

In [6]:
x

{'000038.BKDL': {'categoryCode': '000038.BKDL',
  'categoryName': '**中市值股票',
  'num': 230.14,
  'prePctChangeRate': -1.03,
  'numChange': 59.97,
  'blocks': ['885694.DDBK'],
  'industry': 0,
  'rank': 1,
  'block_dict': {'885694.DDBK': {'blockCode': '885694.DDBK',
    'num': 80.06,
    'prePctChangeRate': -1.23,
    'numChange': 20.0,
    'rank': 2}}},
 '000019.BKDL': {'categoryCode': '000019.BKDL',
  'categoryName': '其他小概念',
  'num': 160.12,
  'prePctChangeRate': -1.19,
  'numChange': 90.05,
  'blocks': ['886012.DDBK', '885522.DDBK'],
  'industry': 0,
  'rank': 2,
  'block_dict': {'886012.DDBK': {'blockCode': '886012.DDBK',
    'num': 20.01,
    'prePctChangeRate': -1.95,
    'numChange': 20.01,
    'rank': 67},
   '885522.DDBK': {'blockCode': '885522.DDBK',
    'num': 10.01,
    'prePctChangeRate': -1.08,
    'numChange': 0.0,
    'rank': 89}}},
 '000033.BKDL': {'categoryCode': '000033.BKDL',
  'categoryName': '智造机器人',
  'num': 160.11,
  'prePctChangeRate': -0.99,
  'numChange': 90.0

In [9]:
a = list(x.keys())

In [10]:
a

['000038.BKDL',
 '000019.BKDL',
 '000033.BKDL',
 '000023.BKDL',
 '000004.BKDL',
 '000035.BKDL',
 '000009.BKDL',
 '000027.BKDL',
 '000007.BKDL',
 '000010.BKDL',
 '000034.BKDL',
 '980365.ZHBK',
 '000012.BKDL',
 '000025.BKDL',
 '980364.ZHBK',
 '000018.BKDL',
 '000036.BKDL',
 '000017.BKDL',
 '000031.BKDL',
 '980378.ZHBK',
 '000014.BKDL',
 '000011.BKDL',
 '980372.ZHBK',
 '000006.BKDL',
 '000026.BKDL',
 '000003.BKDL',
 '000015.BKDL',
 '000022.BKDL',
 '000013.BKDL',
 '000032.BKDL',
 '000021.BKDL',
 '000029.BKDL',
 '980371.ZHBK',
 '000024.BKDL',
 '000037.BKDL',
 '980475.ZHBK',
 '000005.BKDL',
 '980362.ZHBK',
 '000008.BKDL',
 '980357.ZHBK',
 '000001.BKDL',
 '000016.BKDL',
 '980345.ZHBK',
 '980343.ZHBK',
 '980474.ZHBK',
 '980338.ZHBK',
 '980373.ZHBK',
 '980346.ZHBK',
 '980355.ZHBK',
 '980750.ZHBK',
 '980753.ZHBK',
 '980476.ZHBK',
 '980756.ZHBK',
 '980348.ZHBK',
 '980366.ZHBK',
 '000028.BKDL',
 '980368.ZHBK',
 '980363.ZHBK',
 '980335.ZHBK',
 '980358.ZHBK']

In [68]:
b = list(y.keys())

In [69]:
b

['000023.BKDL',
 '000038.BKDL',
 '000012.BKDL',
 '000027.BKDL',
 '000017.BKDL',
 '000010.BKDL',
 '000019.BKDL',
 '000004.BKDL',
 '000034.BKDL',
 '000015.BKDL',
 '000006.BKDL',
 '000026.BKDL',
 '000025.BKDL',
 '000033.BKDL',
 '000018.BKDL',
 '000003.BKDL',
 '000009.BKDL',
 '000035.BKDL',
 '000011.BKDL',
 '000007.BKDL',
 '000013.BKDL',
 '000008.BKDL',
 '000016.BKDL',
 '000014.BKDL',
 'T17.ZHBK',
 '000031.BKDL',
 'T07.ZHBK',
 '000036.BKDL',
 '000029.BKDL',
 '000028.BKDL',
 'T08.ZHBK',
 '000032.BKDL',
 'T10.ZHBK',
 '000037.BKDL',
 'T22.ZHBK',
 'T28.ZHBK',
 'T25.ZHBK',
 '000024.BKDL',
 '000022.BKDL',
 'T03.ZHBK',
 '000001.BKDL',
 'T24.ZHBK',
 'T15.ZHBK',
 '000021.BKDL',
 'T23.ZHBK',
 'T12.ZHBK',
 '000005.BKDL',
 'T01.ZHBK',
 'T29.ZHBK',
 'T18.ZHBK',
 'T26.ZHBK',
 'T09.ZHBK',
 'T14.ZHBK',
 'T11.ZHBK',
 'T13.ZHBK']

In [14]:
a

['000038.BKDL',
 '000019.BKDL',
 '000033.BKDL',
 '000023.BKDL',
 '000004.BKDL',
 '000035.BKDL',
 '000009.BKDL',
 '000027.BKDL',
 '000007.BKDL',
 '000010.BKDL',
 '000034.BKDL',
 '980365.ZHBK',
 '000012.BKDL',
 '000025.BKDL',
 '980364.ZHBK',
 '000018.BKDL',
 '000036.BKDL',
 '000017.BKDL',
 '000031.BKDL',
 '980378.ZHBK',
 '000014.BKDL',
 '000011.BKDL',
 '980372.ZHBK',
 '000006.BKDL',
 '000026.BKDL',
 '000003.BKDL',
 '000015.BKDL',
 '000022.BKDL',
 '000013.BKDL',
 '000032.BKDL',
 '000021.BKDL',
 '000029.BKDL',
 '980371.ZHBK',
 '000024.BKDL',
 '000037.BKDL',
 '980475.ZHBK',
 '000005.BKDL',
 '980362.ZHBK',
 '000008.BKDL',
 '980357.ZHBK',
 '000001.BKDL',
 '000016.BKDL',
 '980345.ZHBK',
 '980343.ZHBK',
 '980474.ZHBK',
 '980338.ZHBK',
 '980373.ZHBK',
 '980346.ZHBK',
 '980355.ZHBK',
 '980750.ZHBK',
 '980753.ZHBK',
 '980476.ZHBK',
 '980756.ZHBK',
 '980348.ZHBK',
 '980366.ZHBK',
 '000028.BKDL',
 '980368.ZHBK',
 '980363.ZHBK',
 '980335.ZHBK',
 '980358.ZHBK']

In [19]:
a

['000038.BKDL',
 '000019.BKDL',
 '000033.BKDL',
 '000023.BKDL',
 '000004.BKDL',
 '000035.BKDL',
 '000009.BKDL',
 '000027.BKDL',
 '000007.BKDL',
 '000010.BKDL',
 '000034.BKDL',
 '980365.ZHBK',
 '000012.BKDL',
 '000025.BKDL',
 '980364.ZHBK',
 '000018.BKDL',
 '000036.BKDL',
 '000017.BKDL',
 '000031.BKDL',
 '980378.ZHBK',
 '000014.BKDL',
 '000011.BKDL',
 '980372.ZHBK',
 '000006.BKDL',
 '000026.BKDL',
 '000003.BKDL',
 '000015.BKDL',
 '000022.BKDL',
 '000013.BKDL',
 '000032.BKDL',
 '000021.BKDL',
 '000029.BKDL',
 '980371.ZHBK',
 '000024.BKDL',
 '000037.BKDL',
 '980475.ZHBK',
 '000005.BKDL',
 '980362.ZHBK',
 '000008.BKDL',
 '980357.ZHBK',
 '000001.BKDL',
 '000016.BKDL',
 '980345.ZHBK',
 '980343.ZHBK',
 '980474.ZHBK',
 '980338.ZHBK',
 '980373.ZHBK',
 '980346.ZHBK',
 '980355.ZHBK',
 '980750.ZHBK',
 '980753.ZHBK',
 '980476.ZHBK',
 '980756.ZHBK',
 '980348.ZHBK',
 '980366.ZHBK',
 '000028.BKDL',
 '980368.ZHBK',
 '980363.ZHBK',
 '980335.ZHBK',
 '980358.ZHBK']

In [20]:
b

['000023.BKDL',
 '000038.BKDL',
 '000012.BKDL',
 '000027.BKDL',
 '000017.BKDL',
 '000010.BKDL',
 '000019.BKDL',
 '000004.BKDL',
 '000034.BKDL',
 '000015.BKDL',
 '000006.BKDL',
 '000026.BKDL',
 '000025.BKDL',
 '000033.BKDL',
 '000018.BKDL',
 '000003.BKDL',
 '000009.BKDL',
 '000035.BKDL',
 '000011.BKDL',
 '000007.BKDL',
 '000013.BKDL',
 '000008.BKDL',
 '000016.BKDL',
 '000014.BKDL',
 'T17.ZHBK',
 '000031.BKDL',
 'T07.ZHBK',
 '000036.BKDL',
 '000029.BKDL',
 '000028.BKDL',
 'T08.ZHBK',
 '000032.BKDL',
 'T10.ZHBK',
 '000037.BKDL',
 'T22.ZHBK',
 'T28.ZHBK',
 'T25.ZHBK',
 '000024.BKDL',
 '000022.BKDL',
 'T03.ZHBK',
 '000001.BKDL',
 'T24.ZHBK',
 'T15.ZHBK',
 '000021.BKDL',
 'T23.ZHBK',
 'T12.ZHBK',
 '000005.BKDL',
 'T01.ZHBK',
 'T29.ZHBK',
 'T18.ZHBK',
 'T26.ZHBK',
 'T09.ZHBK',
 'T14.ZHBK',
 'T11.ZHBK',
 'T13.ZHBK']

In [70]:
abs = []
for c,d in y.items():
    blocks = d['blocks']
    abs.extend(blocks)

In [71]:
abs

['885802.DDBK',
 '886079.DDBK',
 '885742.DDBK',
 '885869.DDBK',
 '886097.DDBK',
 '885729.DDBK',
 '885867.DDBK',
 '886089.DDBK',
 '885787.DDBK',
 '885932.DDBK',
 '885740.DDBK',
 '885739.DDBK',
 '883437.DDBK',
 '885338.DDBK',
 '885694.DDBK',
 '885497.DDBK',
 '885457.DDBK',
 '885788.DDBK',
 '885480.DDBK',
 '885709.DDBK',
 '885418.DDBK',
 '885586.DDBK',
 '885934.DDBK',
 '885816.DDBK',
 '885797.DDBK',
 '886098.DDBK',
 '885792.DDBK',
 '885876.DDBK',
 '885890.DDBK',
 '885791.DDBK',
 '885779.DDBK',
 '886060.DDBK',
 '885918.DDBK',
 '885603.DDBK',
 '885820.DDBK',
 '885772.DDBK',
 '885647.DDBK',
 '885615.DDBK',
 '885431.DDBK',
 '885997.DDBK',
 '885662.DDBK',
 '885461.DDBK',
 '885564.DDBK',
 '886078.DDBK',
 '885945.DDBK',
 '885736.DDBK',
 '885545.DDBK',
 '886035.DDBK',
 '885766.DDBK',
 '886067.DDBK',
 '885775.DDBK',
 '886061.DDBK',
 '886021.DDBK',
 '885595.DDBK',
 '885420.DDBK',
 '885988.DDBK',
 '885784.DDBK',
 '885899.DDBK',
 '885910.DDBK',
 '885705.DDBK',
 '883436.DDBK',
 '885963.DDBK',
 '885522

In [72]:
'885338.DDBK' in abs

True

In [27]:
z = {"000023.BKDL": {"categoryCode": "000023.BKDL", "categoryName": "业绩股权类", "num": 190.17, "prePctChangeRate": 0.22, "numChange": 10.03, "blocks": ["885802.DDBK", "886079.DDBK", "885742.DDBK", "885869.DDBK", "886097.DDBK", "885729.DDBK", "885867.DDBK", "886089.DDBK", "885787.DDBK", "885932.DDBK", "885740.DDBK", "885739.DDBK", "883437.DDBK"], "industry": 0, "rank": 1, "block_dict": {"885802.DDBK": {"blockCode": "885802.DDBK", "num": 60.04, "prePctChangeRate": 0.16, "numChange": 20.0, "rank": 12}, "886079.DDBK": {"blockCode": "886079.DDBK", "num": 50.05, "prePctChangeRate": 0.24, "numChange": 0.0, "rank": 15}, "885742.DDBK": {"blockCode": "885742.DDBK", "num": 30.03, "prePctChangeRate": 1.02, "numChange": 20.02, "rank": 29}, "885869.DDBK": {"blockCode": "885869.DDBK", "num": 30.03, "prePctChangeRate": 0.0, "numChange": 0.01, "rank": 30}, "886097.DDBK": {"blockCode": "886097.DDBK", "num": 20.02, "prePctChangeRate": 0.33, "numChange": -20.01, "rank": 55}, "885729.DDBK": {"blockCode": "885729.DDBK", "num": 20.02, "prePctChangeRate": 0.37, "numChange": 0.0, "rank": 56}, "885867.DDBK": {"blockCode": "885867.DDBK", "num": 20.02, "prePctChangeRate": 0.37, "numChange": 10.01, "rank": 57}, "886089.DDBK": {"blockCode": "886089.DDBK", "num": 20.02, "prePctChangeRate": 0.18, "numChange": -20.01, "rank": 58}, "885787.DDBK": {"blockCode": "885787.DDBK", "num": 10.01, "prePctChangeRate": 0.24, "numChange": -10.01, "rank": 104}, "885932.DDBK": {"blockCode": "885932.DDBK", "num": 10.01, "prePctChangeRate": 1.05, "numChange": 10.01, "rank": 105}, "885740.DDBK": {"blockCode": "885740.DDBK", "num": 10.01, "prePctChangeRate": 1.03, "numChange": 10.01, "rank": 106}, "885739.DDBK": {"blockCode": "885739.DDBK", "num": 10.01, "prePctChangeRate": -0.15, "numChange": 10.01, "rank": 107}, "883437.DDBK": {"blockCode": "883437.DDBK", "num": 10.01, "prePctChangeRate": 0.55, "numChange": 0.0, "rank": 108}}}, "000038.BKDL": {"categoryCode": "000038.BKDL", "categoryName": "**中市值股票", "num": 170.14, "prePctChangeRate": 0.22, "numChange": -110.07, "blocks": ["885338.DDBK", "885694.DDBK"], "industry": 0, "rank": 2, "block_dict": {"885338.DDBK": {"blockCode": "885338.DDBK", "num": 150.12, "prePctChangeRate": 0.21, "numChange": -110.07, "rank": 1}, "885694.DDBK": {"blockCode": "885694.DDBK", "num": 60.05, "prePctChangeRate": 0.18, "numChange": -30.01, "rank": 8}}}, "000012.BKDL": {"categoryCode": "000012.BKDL", "categoryName": "传媒游戏娱乐", "num": 150.12, "prePctChangeRate": 0.2, "numChange": 100.1, "blocks": ["885497.DDBK", "885457.DDBK", "885788.DDBK", "885480.DDBK", "885709.DDBK", "885418.DDBK", "885586.DDBK", "885934.DDBK", "885816.DDBK", "885797.DDBK", "886098.DDBK", "885792.DDBK", "885876.DDBK", "885890.DDBK", "885791.DDBK", "885779.DDBK", "886060.DDBK", "885918.DDBK", "885603.DDBK", "885820.DDBK", "885772.DDBK", "885647.DDBK", "885615.DDBK"], "industry": 0, "rank": 3, "block_dict": {"885497.DDBK": {"blockCode": "885497.DDBK", "num": 30.03, "prePctChangeRate": 0.32, "numChange": 30.03, "rank": 31}, "885457.DDBK": {"blockCode": "885457.DDBK", "num": 30.03, "prePctChangeRate": 0.32, "numChange": 20.02, "rank": 32}, "885788.DDBK": {"blockCode": "885788.DDBK", "num": 30.02, "prePctChangeRate": 0.17, "numChange": 20.01, "rank": 44}, "885480.DDBK": {"blockCode": "885480.DDBK", "num": 30.02, "prePctChangeRate": 0.08, "numChange": 30.02, "rank": 45}, "885709.DDBK": {"blockCode": "885709.DDBK", "num": 30.02, "prePctChangeRate": 0.13, "numChange": 30.02, "rank": 46}, "885418.DDBK": {"blockCode": "885418.DDBK", "num": 20.02, "prePctChangeRate": 0.31, "numChange": 10.02, "rank": 59}, "885586.DDBK": {"blockCode": "885586.DDBK", "num": 20.02, "prePctChangeRate": 0.14, "numChange": 20.02, "rank": 60}, "885934.DDBK": {"blockCode": "885934.DDBK", "num": 20.02, "prePctChangeRate": 0.27, "numChange": 10.02, "rank": 61}, "885816.DDBK": {"blockCode": "885816.DDBK", "num": 20.02, "prePctChangeRate": -0.13, "numChange": 20.02, "rank": 62}, "885797.DDBK": {"blockCode": "885797.DDBK", "num": 20.02, "prePctChangeRate": 0.74, "numChange": 10.01, "rank": 63}, "886098.DDBK": {"blockCode": "886098.DDBK", "num": 20.01, "prePctChangeRate": 1.33, "numChange": 20.01, "rank": 86}, "885792.DDBK": {"blockCode": "885792.DDBK", "num": 20.01, "prePctChangeRate": 0.93, "numChange": 20.01, "rank": 87}, "885876.DDBK": {"blockCode": "885876.DDBK", "num": 20.01, "prePctChangeRate": 0.5, "numChange": 20.01, "rank": 88}, "885890.DDBK": {"blockCode": "885890.DDBK", "num": 10.01, "prePctChangeRate": 0.48, "numChange": 0.0, "rank": 109}, "885791.DDBK": {"blockCode": "885791.DDBK", "num": 10.01, "prePctChangeRate": 0.55, "numChange": 10.01, "rank": 110}, "885779.DDBK": {"blockCode": "885779.DDBK", "num": 10.01, "prePctChangeRate": 0.41, "numChange": 10.01, "rank": 111}, "886060.DDBK": {"blockCode": "886060.DDBK", "num": 10.01, "prePctChangeRate": 0.03, "numChange": 10.01, "rank": 112}, "885918.DDBK": {"blockCode": "885918.DDBK", "num": 10.01, "prePctChangeRate": 0.63, "numChange": 10.01, "rank": 113}, "885603.DDBK": {"blockCode": "885603.DDBK", "num": 10.01, "prePctChangeRate": 0.11, "numChange": 10.01, "rank": 114}, "885820.DDBK": {"blockCode": "885820.DDBK", "num": 10.0, "prePctChangeRate": 0.31, "numChange": 10.0, "rank": 209}, "885772.DDBK": {"blockCode": "885772.DDBK", "num": 10.0, "prePctChangeRate": -0.27, "numChange": 10.0, "rank": 210}, "885647.DDBK": {"blockCode": "885647.DDBK", "num": 10.0, "prePctChangeRate": 1.17, "numChange": 0.0, "rank": 211}, "885615.DDBK": {"blockCode": "885615.DDBK", "num": 10.0, "prePctChangeRate": -0.12, "numChange": -10.0, "rank": 212}}}, "000027.BKDL": {"categoryCode": "000027.BKDL", "categoryName": "汽车低空", "num": 140.11, "prePctChangeRate": -0.16, "numChange": -0.01, "blocks": ["885431.DDBK", "885997.DDBK", "885662.DDBK", "885461.DDBK", "885564.DDBK", "886078.DDBK", "885945.DDBK", "885736.DDBK", "885545.DDBK", "886035.DDBK", "885766.DDBK", "886067.DDBK", "885775.DDBK", "886061.DDBK"], "industry": 0, "rank": 4, "block_dict": {"885431.DDBK": {"blockCode": "885431.DDBK", "num": 40.03, "prePctChangeRate": -0.27, "numChange": -10.02, "rank": 23}, "885997.DDBK": {"blockCode": "885997.DDBK", "num": 30.03, "prePctChangeRate": -0.33, "numChange": -10.01, "rank": 33}, "885662.DDBK": {"blockCode": "885662.DDBK", "num": 20.02, "prePctChangeRate": 0.24, "numChange": 20.02, "rank": 64}, "885461.DDBK": {"blockCode": "885461.DDBK", "num": 20.01, "prePctChangeRate": -0.29, "numChange": 0.0, "rank": 89}, "885564.DDBK": {"blockCode": "885564.DDBK", "num": 20.01, "prePctChangeRate": -0.44, "numChange": -40.03, "rank": 90}, "886078.DDBK": {"blockCode": "886078.DDBK", "num": 20.01, "prePctChangeRate": -0.61, "numChange": -30.03, "rank": 91}, "885945.DDBK": {"blockCode": "885945.DDBK", "num": 10.01, "prePctChangeRate": 0.05, "numChange": -10.01, "rank": 115}, "885736.DDBK": {"blockCode": "885736.DDBK", "num": 10.01, "prePctChangeRate": -0.31, "numChange": 0.0, "rank": 116}, "885545.DDBK": {"blockCode": "885545.DDBK", "num": 10.01, "prePctChangeRate": -0.29, "numChange": 0.0, "rank": 117}, "886035.DDBK": {"blockCode": "886035.DDBK", "num": 10.01, "prePctChangeRate": -0.62, "numChange": 0.0, "rank": 118}, "885766.DDBK": {"blockCode": "885766.DDBK", "num": 10.01, "prePctChangeRate": 0.41, "numChange": 10.01, "rank": 119}, "886067.DDBK": {"blockCode": "886067.DDBK", "num": 10.01, "prePctChangeRate": -0.2, "numChange": 0.0, "rank": 120}, "885775.DDBK": {"blockCode": "885775.DDBK", "num": 10.01, "prePctChangeRate": -0.42, "numChange": 10.01, "rank": 121}, "886061.DDBK": {"blockCode": "886061.DDBK", "num": 10.01, "prePctChangeRate": -0.43, "numChange": -10.01, "rank": 122}}}, "000017.BKDL": {"categoryCode": "000017.BKDL", "categoryName": "**中字头国企", "num": 130.12, "prePctChangeRate": 0.56, "numChange": 90.08, "blocks": ["886021.DDBK", "885595.DDBK"], "industry": 0, "rank": 5, "block_dict": {"886021.DDBK": {"blockCode": "886021.DDBK", "num": 130.12, "prePctChangeRate": 0.55, "numChange": 90.08, "rank": 2}, "885595.DDBK": {"blockCode": "885595.DDBK", "num": 10.0, "prePctChangeRate": 0.45, "numChange": -0.01, "rank": 213}}}, "000010.BKDL": {"categoryCode": "000010.BKDL", "categoryName": "**大消费类", "num": 130.1, "prePctChangeRate": 0.45, "numChange": 80.07, "blocks": ["885420.DDBK", "885988.DDBK", "885784.DDBK", "885899.DDBK", "885910.DDBK", "885705.DDBK"], "industry": 0, "rank": 6, "block_dict": {"885420.DDBK": {"blockCode": "885420.DDBK", "num": 60.05, "prePctChangeRate": 0.38, "numChange": 30.03, "rank": 9}, "885988.DDBK": {"blockCode": "885988.DDBK", "num": 40.03, "prePctChangeRate": 3.3, "numChange": 40.03, "rank": 24}, "885784.DDBK": {"blockCode": "885784.DDBK", "num": 20.02, "prePctChangeRate": 0.17, "numChange": 20.02, "rank": 65}, "885899.DDBK": {"blockCode": "885899.DDBK", "num": 20.01, "prePctChangeRate": -0.06, "numChange": 20.01, "rank": 92}, "885910.DDBK": {"blockCode": "885910.DDBK", "num": 10.01, "prePctChangeRate": 1.06, "numChange": 0.0, "rank": 123}, "885705.DDBK": {"blockCode": "885705.DDBK", "num": 10.01, "prePctChangeRate": 0.56, "numChange": 10.01, "rank": 124}}}, "000019.BKDL": {"categoryCode": "000019.BKDL", "categoryName": "其他小概念", "num": 120.11, "prePctChangeRate": 0.46, "numChange": 0.01, "blocks": ["883436.DDBK", "885963.DDBK", "885522.DDBK"], "industry": 0, "rank": 7, "block_dict": {"883436.DDBK": {"blockCode": "883436.DDBK", "num": 100.09, "prePctChangeRate": 0.48, "numChange": 0.0, "rank": 3}, "885963.DDBK": {"blockCode": "885963.DDBK", "num": 10.01, "prePctChangeRate": 0.0, "numChange": 10.01, "rank": 125}, "885522.DDBK": {"blockCode": "885522.DDBK", "num": 10.01, "prePctChangeRate": 0.45, "numChange": -10.0, "rank": 126}}}, "000004.BKDL": {"categoryCode": "000004.BKDL", "categoryName": "**区域经济", "num": 120.09, "prePctChangeRate": 0.43, "numChange": -50.04, "blocks": ["885521.DDBK", "886086.DDBK", "885832.DDBK", "885827.DDBK", "885744.DDBK", "885924.DDBK", "885894.DDBK", "885514.DDBK", "885617.DDBK", "885939.DDBK", "885345.DDBK"], "industry": 0, "rank": 8, "block_dict": {"885521.DDBK": {"blockCode": "885521.DDBK", "num": 30.03, "prePctChangeRate": 0.53, "numChange": 0.01, "rank": 34}, "886086.DDBK": {"blockCode": "886086.DDBK", "num": 20.01, "prePctChangeRate": 0.26, "numChange": -50.05, "rank": 93}, "885832.DDBK": {"blockCode": "885832.DDBK", "num": 10.01, "prePctChangeRate": 0.51, "numChange": -20.01, "rank": 127}, "885827.DDBK": {"blockCode": "885827.DDBK", "num": 10.01, "prePctChangeRate": 0.38, "numChange": 0.0, "rank": 128}, "885744.DDBK": {"blockCode": "885744.DDBK", "num": 10.01, "prePctChangeRate": 0.26, "numChange": -10.01, "rank": 129}, "885924.DDBK": {"blockCode": "885924.DDBK", "num": 10.01, "prePctChangeRate": 0.25, "numChange": -10.0, "rank": 130}, "885894.DDBK": {"blockCode": "885894.DDBK", "num": 10.01, "prePctChangeRate": -0.09, "numChange": 10.01, "rank": 131}, "885514.DDBK": {"blockCode": "885514.DDBK", "num": 10.01, "prePctChangeRate": 1.32, "numChange": -10.01, "rank": 132}, "885617.DDBK": {"blockCode": "885617.DDBK", "num": 10.0, "prePctChangeRate": 0.36, "numChange": 0.0, "rank": 214}, "885939.DDBK": {"blockCode": "885939.DDBK", "num": 10.0, "prePctChangeRate": 1.87, "numChange": 10.0, "rank": 215}, "885345.DDBK": {"blockCode": "885345.DDBK", "num": 10.0, "prePctChangeRate": 0.58, "numChange": 10.0, "rank": 216}}}, "000034.BKDL": {"categoryCode": "000034.BKDL", "categoryName": "人工智能", "num": 110.09, "prePctChangeRate": 0.32, "numChange": 50.06, "blocks": ["885728.DDBK", "886099.DDBK", "886031.DDBK", "886100.DDBK", "886074.DDBK", "886070.DDBK", "886071.DDBK", "886019.DDBK", "886068.DDBK"], "industry": 0, "rank": 9, "block_dict": {"885728.DDBK": {"blockCode": "885728.DDBK", "num": 60.05, "prePctChangeRate": 0.32, "numChange": 10.03, "rank": 10}, "886099.DDBK": {"blockCode": "886099.DDBK", "num": 20.02, "prePctChangeRate": 0.92, "numChange": -9.99, "rank": 66}, "886031.DDBK": {"blockCode": "886031.DDBK", "num": 20.02, "prePctChangeRate": 0.87, "numChange": 10.02, "rank": 67}, "886100.DDBK": {"blockCode": "886100.DDBK", "num": 20.01, "prePctChangeRate": 0.47, "numChange": 10.01, "rank": 94}, "886074.DDBK": {"blockCode": "886074.DDBK", "num": 10.01, "prePctChangeRate": 0.72, "numChange": 10.01, "rank": 133}, "886070.DDBK": {"blockCode": "886070.DDBK", "num": 10.01, "prePctChangeRate": 0.06, "numChange": 10.01, "rank": 134}, "886071.DDBK": {"blockCode": "886071.DDBK", "num": 10.01, "prePctChangeRate": 0.42, "numChange": 10.01, "rank": 135}, "886019.DDBK": {"blockCode": "886019.DDBK", "num": 10.01, "prePctChangeRate": 0.59, "numChange": -19.99, "rank": 136}, "886068.DDBK": {"blockCode": "886068.DDBK", "num": 10.01, "prePctChangeRate": 0.46, "numChange": 10.01, "rank": 137}}}, "000015.BKDL": {"categoryCode": "000015.BKDL", "categoryName": "**非银金融", "num": 100.09, "prePctChangeRate": 0.9, "numChange": 70.07, "blocks": ["885413.DDBK", "885456.DDBK", "885767.DDBK", "885657.DDBK"], "industry": 0, "rank": 10, "block_dict": {"885413.DDBK": {"blockCode": "885413.DDBK", "num": 80.08, "prePctChangeRate": 0.48, "numChange": 60.06, "rank": 6}, "885456.DDBK": {"blockCode": "885456.DDBK", "num": 20.02, "prePctChangeRate": 2.24, "numChange": 20.02, "rank": 68}, "885767.DDBK": {"blockCode": "885767.DDBK", "num": 10.01, "prePctChangeRate": 1.95, "numChange": 10.01, "rank": 138}, "885657.DDBK": {"blockCode": "885657.DDBK", "num": 10.0, "prePctChangeRate": 0.99, "numChange": 10.0, "rank": 217}}}, "000006.BKDL": {"categoryCode": "000006.BKDL", "categoryName": "华为大科技", "num": 100.09, "prePctChangeRate": 0.27, "numChange": 30.03, "blocks": ["885806.DDBK", "885502.DDBK", "885923.DDBK", "885376.DDBK"], "industry": 0, "rank": 11, "block_dict": {"885806.DDBK": {"blockCode": "885806.DDBK", "num": 90.08, "prePctChangeRate": 0.18, "numChange": 20.02, "rank": 4}, "885502.DDBK": {"blockCode": "885502.DDBK", "num": 20.02, "prePctChangeRate": 1.2, "numChange": 20.02, "rank": 69}, "885923.DDBK": {"blockCode": "885923.DDBK", "num": 10.01, "prePctChangeRate": 0.72, "numChange": 10.01, "rank": 139}, "885376.DDBK": {"blockCode": "885376.DDBK", "num": 10.0, "prePctChangeRate": -0.28, "numChange": -0.01, "rank": 218}}}, "000026.BKDL": {"categoryCode": "000026.BKDL", "categoryName": "**外贸出口", "num": 100.08, "prePctChangeRate": 0.54, "numChange": -30.01, "blocks": ["885642.DDBK", "885965.DDBK", "885840.DDBK", "885611.DDBK"], "industry": 0, "rank": 12, "block_dict": {"885642.DDBK": {"blockCode": "885642.DDBK", "num": 40.04, "prePctChangeRate": 1.61, "numChange": -20.01, "rank": 18}, "885965.DDBK": {"blockCode": "885965.DDBK", "num": 40.03, "prePctChangeRate": 0.98, "numChange": 20.02, "rank": 25}, "885840.DDBK": {"blockCode": "885840.DDBK", "num": 20.02, "prePctChangeRate": 0.16, "numChange": -60.03, "rank": 70}, "885611.DDBK": {"blockCode": "885611.DDBK", "num": 20.01, "prePctChangeRate": 0.8, "numChange": 10.0, "rank": 95}}}, "000025.BKDL": {"categoryCode": "000025.BKDL", "categoryName": "通信类", "num": 100.08, "prePctChangeRate": 0.05, "numChange": 20.02, "blocks": ["885556.DDBK", "885312.DDBK", "885783.DDBK", "885574.DDBK", "885998.DDBK", "885868.DDBK", "886084.DDBK", "885940.DDBK"], "industry": 0, "rank": 13, "block_dict": {"885556.DDBK": {"blockCode": "885556.DDBK", "num": 50.05, "prePctChangeRate": -0.03, "numChange": 20.03, "rank": 16}, "885312.DDBK": {"blockCode": "885312.DDBK", "num": 40.03, "prePctChangeRate": 0.35, "numChange": 30.02, "rank": 26}, "885783.DDBK": {"blockCode": "885783.DDBK", "num": 30.01, "prePctChangeRate": -0.02, "numChange": 10.0, "rank": 52}, "885574.DDBK": {"blockCode": "885574.DDBK", "num": 20.02, "prePctChangeRate": -0.45, "numChange": 0.0, "rank": 71}, "885998.DDBK": {"blockCode": "885998.DDBK", "num": 10.01, "prePctChangeRate": 0.66, "numChange": 0.0, "rank": 140}, "885868.DDBK": {"blockCode": "885868.DDBK", "num": 10.01, "prePctChangeRate": 0.01, "numChange": 10.01, "rank": 141}, "886084.DDBK": {"blockCode": "886084.DDBK", "num": 10.01, "prePctChangeRate": 0.59, "numChange": 0.0, "rank": 142}, "885940.DDBK": {"blockCode": "885940.DDBK", "num": 10.0, "prePctChangeRate": 0.12, "numChange": 10.0, "rank": 219}}}, "000033.BKDL": {"categoryCode": "000033.BKDL", "categoryName": "智造机器人", "num": 100.06, "prePctChangeRate": -0.14, "numChange": -80.06, "blocks": ["885929.DDBK", "885517.DDBK", "885941.DDBK", "886057.DDBK", "886002.DDBK", "885946.DDBK", "885427.DDBK", "885786.DDBK", "885619.DDBK"], "industry": 0, "rank": 14, "block_dict": {"885929.DDBK": {"blockCode": "885929.DDBK", "num": 40.03, "prePctChangeRate": -0.19, "numChange": -90.05, "rank": 27}, "885517.DDBK": {"blockCode": "885517.DDBK", "num": 30.01, "prePctChangeRate": -0.3, "numChange": -10.02, "rank": 53}, "885941.DDBK": {"blockCode": "885941.DDBK", "num": 30.0, "prePctChangeRate": -0.27, "numChange": -10.02, "rank": 54}, "886057.DDBK": {"blockCode": "886057.DDBK", "num": 20.0, "prePctChangeRate": -0.45, "numChange": -10.02, "rank": 102}, "886002.DDBK": {"blockCode": "886002.DDBK", "num": 20.0, "prePctChangeRate": -0.42, "numChange": -0.02, "rank": 103}, "885946.DDBK": {"blockCode": "885946.DDBK", "num": 10.01, "prePctChangeRate": -0.33, "numChange": -10.01, "rank": 143}, "885427.DDBK": {"blockCode": "885427.DDBK", "num": 10.01, "prePctChangeRate": -0.27, "numChange": -30.03, "rank": 144}, "885786.DDBK": {"blockCode": "885786.DDBK", "num": 10.01, "prePctChangeRate": 0.02, "numChange": 10.01, "rank": 145}, "885619.DDBK": {"blockCode": "885619.DDBK", "num": 10.0, "prePctChangeRate": -0.39, "numChange": -10.01, "rank": 220}}}, "000018.BKDL": {"categoryCode": "000018.BKDL", "categoryName": "**一带一路", "num": 90.07, "prePctChangeRate": 0.51, "numChange": 10.0, "blocks": ["885494.DDBK", "885733.DDBK", "885770.DDBK"], "industry": 0, "rank": 15, "block_dict": {"885494.DDBK": {"blockCode": "885494.DDBK", "num": 90.07, "prePctChangeRate": 0.58, "numChange": 40.03, "rank": 5}, "885733.DDBK": {"blockCode": "885733.DDBK", "num": 30.03, "prePctChangeRate": 5.33, "numChange": 20.02, "rank": 35}, "885770.DDBK": {"blockCode": "885770.DDBK", "num": 20.02, "prePctChangeRate": 1.06, "numChange": 20.02, "rank": 72}}}, "000003.BKDL": {"categoryCode": "000003.BKDL", "categoryName": "数字科技", "num": 80.07, "prePctChangeRate": 0.52, "numChange": 30.05, "blocks": ["886041.DDBK", "885976.DDBK", "886013.DDBK", "885452.DDBK", "885757.DDBK", "885842.DDBK", "885933.DDBK", "886023.DDBK", "885887.DDBK", "885333.DDBK", "885942.DDBK", "885975.DDBK"], "industry": 0, "rank": 16, "block_dict": {"886041.DDBK": {"blockCode": "886041.DDBK", "num": 30.03, "prePctChangeRate": 0.94, "numChange": 20.03, "rank": 36}, "885976.DDBK": {"blockCode": "885976.DDBK", "num": 30.03, "prePctChangeRate": 0.75, "numChange": 0.02, "rank": 37}, "886013.DDBK": {"blockCode": "886013.DDBK", "num": 20.02, "prePctChangeRate": 0.9, "numChange": 20.02, "rank": 73}, "885452.DDBK": {"blockCode": "885452.DDBK", "num": 20.02, "prePctChangeRate": 0.79, "numChange": 0.01, "rank": 74}, "885757.DDBK": {"blockCode": "885757.DDBK", "num": 20.02, "prePctChangeRate": 0.7, "numChange": 10.02, "rank": 75}, "885842.DDBK": {"blockCode": "885842.DDBK", "num": 10.01, "prePctChangeRate": 1.39, "numChange": 10.01, "rank": 146}, "885933.DDBK": {"blockCode": "885933.DDBK", "num": 10.01, "prePctChangeRate": 0.38, "numChange": 10.01, "rank": 147}, "886023.DDBK": {"blockCode": "886023.DDBK", "num": 10.01, "prePctChangeRate": 0.71, "numChange": 10.01, "rank": 148}, "885887.DDBK": {"blockCode": "885887.DDBK", "num": 10.01, "prePctChangeRate": 0.62, "numChange": -19.99, "rank": 149}, "885333.DDBK": {"blockCode": "885333.DDBK", "num": 10.01, "prePctChangeRate": 1.56, "numChange": 10.01, "rank": 150}, "885942.DDBK": {"blockCode": "885942.DDBK", "num": 10.01, "prePctChangeRate": 0.42, "numChange": 10.01, "rank": 151}, "885975.DDBK": {"blockCode": "885975.DDBK", "num": 10.0, "prePctChangeRate": 0.32, "numChange": 10.0, "rank": 221}}}, "000009.BKDL": {"categoryCode": "000009.BKDL", "categoryName": "**医药医疗", "num": 80.06, "prePctChangeRate": -0.04, "numChange": -20.02, "blocks": ["885539.DDBK", "885989.DDBK", "885952.DDBK", "885955.DDBK", "885490.DDBK", "885403.DDBK", "885927.DDBK", "885845.DDBK", "886056.DDBK", "885856.DDBK", "885879.DDBK", "885703.DDBK", "886077.DDBK", "885852.DDBK", "886015.DDBK"], "industry": 0, "rank": 17, "block_dict": {"885539.DDBK": {"blockCode": "885539.DDBK", "num": 40.03, "prePctChangeRate": -0.21, "numChange": 0.0, "rank": 28}, "885989.DDBK": {"blockCode": "885989.DDBK", "num": 30.02, "prePctChangeRate": -0.31, "numChange": 20.01, "rank": 47}, "885952.DDBK": {"blockCode": "885952.DDBK", "num": 20.01, "prePctChangeRate": -0.4, "numChange": 20.01, "rank": 96}, "885955.DDBK": {"blockCode": "885955.DDBK", "num": 10.01, "prePctChangeRate": -0.53, "numChange": 0.0, "rank": 152}, "885490.DDBK": {"blockCode": "885490.DDBK", "num": 10.01, "prePctChangeRate": -0.13, "numChange": 0.0, "rank": 153}, "885403.DDBK": {"blockCode": "885403.DDBK", "num": 10.01, "prePctChangeRate": -0.19, "numChange": 0.0, "rank": 154}, "885927.DDBK": {"blockCode": "885927.DDBK", "num": 10.01, "prePctChangeRate": -0.27, "numChange": 10.01, "rank": 155}, "885845.DDBK": {"blockCode": "885845.DDBK", "num": 10.01, "prePctChangeRate": 0.13, "numChange": 0.0, "rank": 156}, "886056.DDBK": {"blockCode": "886056.DDBK", "num": 10.01, "prePctChangeRate": -0.18, "numChange": 10.01, "rank": 157}, "885856.DDBK": {"blockCode": "885856.DDBK", "num": 10.0, "prePctChangeRate": -0.28, "numChange": 10.0, "rank": 222}, "885879.DDBK": {"blockCode": "885879.DDBK", "num": 10.0, "prePctChangeRate": -0.31, "numChange": 10.0, "rank": 223}, "885703.DDBK": {"blockCode": "885703.DDBK", "num": 10.0, "prePctChangeRate": 0.12, "numChange": -0.01, "rank": 224}, "886077.DDBK": {"blockCode": "886077.DDBK", "num": 10.0, "prePctChangeRate": 0.07, "numChange": 0.0, "rank": 225}, "885852.DDBK": {"blockCode": "885852.DDBK", "num": 10.0, "prePctChangeRate": 0.34, "numChange": 10.0, "rank": 226}, "886015.DDBK": {"blockCode": "886015.DDBK", "num": 10.0, "prePctChangeRate": -0.38, "numChange": 10.0, "rank": 227}}}, "000035.BKDL": {"categoryCode": "000035.BKDL", "categoryName": "**风光储电", "num": 80.06, "prePctChangeRate": -0.1, "numChange": -60.05, "blocks": ["885531.DDBK", "885571.DDBK", "885819.DDBK", "885922.DDBK", "885789.DDBK", "885958.DDBK", "885936.DDBK", "885710.DDBK", "885311.DDBK"], "industry": 0, "rank": 18, "block_dict": {"885531.DDBK": {"blockCode": "885531.DDBK", "num": 30.03, "prePctChangeRate": -0.24, "numChange": -30.01, "rank": 38}, "885571.DDBK": {"blockCode": "885571.DDBK", "num": 20.01, "prePctChangeRate": -0.02, "numChange": -10.02, "rank": 97}, "885819.DDBK": {"blockCode": "885819.DDBK", "num": 10.01, "prePctChangeRate": -0.18, "numChange": 10.01, "rank": 158}, "885922.DDBK": {"blockCode": "885922.DDBK", "num": 10.01, "prePctChangeRate": -0.16, "numChange": 0.0, "rank": 159}, "885789.DDBK": {"blockCode": "885789.DDBK", "num": 10.01, "prePctChangeRate": -0.27, "numChange": 0.0, "rank": 160}, "885958.DDBK": {"blockCode": "885958.DDBK", "num": 10.01, "prePctChangeRate": -0.85, "numChange": 0.0, "rank": 161}, "885936.DDBK": {"blockCode": "885936.DDBK", "num": 10.01, "prePctChangeRate": 0.07, "numChange": 0.0, "rank": 162}, "885710.DDBK": {"blockCode": "885710.DDBK", "num": 10.01, "prePctChangeRate": -0.08, "numChange": -20.01, "rank": 163}, "885311.DDBK": {"blockCode": "885311.DDBK", "num": 10.0, "prePctChangeRate": -0.4, "numChange": 0.0, "rank": 228}}}, "000011.BKDL": {"categoryCode": "000011.BKDL", "categoryName": "**环保碳中和", "num": 70.07, "prePctChangeRate": 0.08, "numChange": 10.04, "blocks": ["885563.DDBK", "885919.DDBK", "885690.DDBK", "885410.DDBK", "885823.DDBK", "885412.DDBK", "885555.DDBK"], "industry": 0, "rank": 19, "block_dict": {"885563.DDBK": {"blockCode": "885563.DDBK", "num": 20.02, "prePctChangeRate": -0.04, "numChange": -19.99, "rank": 76}, "885919.DDBK": {"blockCode": "885919.DDBK", "num": 20.02, "prePctChangeRate": 0.39, "numChange": 0.0, "rank": 77}, "885690.DDBK": {"blockCode": "885690.DDBK", "num": 10.01, "prePctChangeRate": -0.3, "numChange": 10.01, "rank": 164}, "885410.DDBK": {"blockCode": "885410.DDBK", "num": 10.01, "prePctChangeRate": -0.12, "numChange": 10.01, "rank": 165}, "885823.DDBK": {"blockCode": "885823.DDBK", "num": 10.01, "prePctChangeRate": 0.05, "numChange": -20.0, "rank": 166}, "885412.DDBK": {"blockCode": "885412.DDBK", "num": 10.01, "prePctChangeRate": -0.01, "numChange": 0.01, "rank": 167}, "885555.DDBK": {"blockCode": "885555.DDBK", "num": 10.01, "prePctChangeRate": -0.15, "numChange": 10.01, "rank": 168}}}, "000007.BKDL": {"categoryCode": "000007.BKDL", "categoryName": "**房产建筑", "num": 70.07, "prePctChangeRate": -0.05, "numChange": 30.04, "blocks": ["885991.DDBK", "885478.DDBK", "885760.DDBK", "885915.DDBK", "885750.DDBK", "885386.DDBK", "885984.DDBK"], "industry": 0, "rank": 20, "block_dict": {"885991.DDBK": {"blockCode": "885991.DDBK", "num": 40.04, "prePctChangeRate": 0.04, "numChange": 20.02, "rank": 19}, "885478.DDBK": {"blockCode": "885478.DDBK", "num": 20.02, "prePctChangeRate": -0.31, "numChange": 20.02, "rank": 78}, "885760.DDBK": {"blockCode": "885760.DDBK", "num": 20.02, "prePctChangeRate": 0.16, "numChange": -20.01, "rank": 79}, "885915.DDBK": {"blockCode": "885915.DDBK", "num": 10.01, "prePctChangeRate": 0.23, "numChange": 10.01, "rank": 169}, "885750.DDBK": {"blockCode": "885750.DDBK", "num": 10.01, "prePctChangeRate": -0.22, "numChange": 10.01, "rank": 170}, "885386.DDBK": {"blockCode": "885386.DDBK", "num": 10.01, "prePctChangeRate": 0.06, "numChange": -10.0, "rank": 171}, "885984.DDBK": {"blockCode": "885984.DDBK", "num": 10.01, "prePctChangeRate": -0.54, "numChange": 0.0, "rank": 172}}}, "000013.BKDL": {"categoryCode": "000013.BKDL", "categoryName": "**国防军工", "num": 70.05, "prePctChangeRate": -0.25, "numChange": -50.05, "blocks": ["885700.DDBK", "885743.DDBK", "886030.DDBK", "885566.DDBK"], "industry": 0, "rank": 21, "block_dict": {"885700.DDBK": {"blockCode": "885700.DDBK", "num": 70.05, "prePctChangeRate": -0.33, "numChange": -40.04, "rank": 7}, "885743.DDBK": {"blockCode": "885743.DDBK", "num": 10.0, "prePctChangeRate": -0.53, "numChange": -20.01, "rank": 229}, "886030.DDBK": {"blockCode": "886030.DDBK", "num": 10.0, "prePctChangeRate": -0.18, "numChange": -40.03, "rank": 230}, "885566.DDBK": {"blockCode": "885566.DDBK", "num": 10.0, "prePctChangeRate": -0.2, "numChange": -40.03, "rank": 231}}}, "000008.BKDL": {"categoryCode": "000008.BKDL", "categoryName": "**传统行业", "num": 60.06, "prePctChangeRate": 0.19, "numChange": 50.05, "blocks": ["885433.DDBK", "885572.DDBK", "885428.DDBK", "885423.DDBK", "885692.DDBK"], "industry": 0, "rank": 22, "block_dict": {"885433.DDBK": {"blockCode": "885433.DDBK", "num": 20.02, "prePctChangeRate": 0.04, "numChange": 10.01, "rank": 80}, "885572.DDBK": {"blockCode": "885572.DDBK", "num": 20.02, "prePctChangeRate": 0.06, "numChange": 20.02, "rank": 81}, "885428.DDBK": {"blockCode": "885428.DDBK", "num": 10.01, "prePctChangeRate": -0.1, "numChange": 10.01, "rank": 173}, "885423.DDBK": {"blockCode": "885423.DDBK", "num": 10.01, "prePctChangeRate": -0.07, "numChange": 10.01, "rank": 174}, "885692.DDBK": {"blockCode": "885692.DDBK", "num": 10.01, "prePctChangeRate": 0.01, "numChange": 10.01, "rank": 175}}}, "000016.BKDL": {"categoryCode": "000016.BKDL", "categoryName": "软件互联网", "num": 60.05, "prePctChangeRate": 0.47, "numChange": 40.05, "blocks": ["885378.DDBK", "885568.DDBK", "885459.DDBK", "885956.DDBK", "885730.DDBK", "885844.DDBK"], "industry": 0, "rank": 23, "block_dict": {"885378.DDBK": {"blockCode": "885378.DDBK", "num": 30.03, "prePctChangeRate": 0.49, "numChange": 30.03, "rank": 39}, "885568.DDBK": {"blockCode": "885568.DDBK", "num": 30.02, "prePctChangeRate": 0.6, "numChange": 30.02, "rank": 48}, "885459.DDBK": {"blockCode": "885459.DDBK", "num": 20.02, "prePctChangeRate": 0.79, "numChange": 10.02, "rank": 82}, "885956.DDBK": {"blockCode": "885956.DDBK", "num": 10.01, "prePctChangeRate": 1.09, "numChange": 0.01, "rank": 176}, "885730.DDBK": {"blockCode": "885730.DDBK", "num": 10.01, "prePctChangeRate": 0.25, "numChange": 10.01, "rank": 177}, "885844.DDBK": {"blockCode": "885844.DDBK", "num": 10.0, "prePctChangeRate": 0.25, "numChange": 10.0, "rank": 232}}}, "000014.BKDL": {"categoryCode": "000014.BKDL", "categoryName": "泛科技", "num": 60.05, "prePctChangeRate": 0.05, "numChange": 50.04, "blocks": ["885785.DDBK", "885759.DDBK", "886036.DDBK", "885454.DDBK"], "industry": 0, "rank": 24, "block_dict": {"885785.DDBK": {"blockCode": "885785.DDBK", "num": 30.03, "prePctChangeRate": -0.11, "numChange": 20.02, "rank": 40}, "885759.DDBK": {"blockCode": "885759.DDBK", "num": 20.01, "prePctChangeRate": 0.36, "numChange": 20.01, "rank": 98}, "886036.DDBK": {"blockCode": "886036.DDBK", "num": 10.01, "prePctChangeRate": 1.28, "numChange": 10.01, "rank": 178}, "885454.DDBK": {"blockCode": "885454.DDBK", "num": 10.01, "prePctChangeRate": -0.05, "numChange": 10.01, "rank": 179}}}, "T17.ZHBK": {"categoryCode": "T17.ZHBK", "categoryName": "交通运输", "num": 60.05, "prePctChangeRate": 3.23, "numChange": 50.04, "blocks": ["T17.ZHBK"], "industry": 1, "rank": 25, "block_dict": {"T17.ZHBK": {"blockCode": "T17.ZHBK", "num": 60.05, "prePctChangeRate": 3.23, "numChange": 50.04, "rank": 11}}}, "000031.BKDL": {"categoryCode": "000031.BKDL", "categoryName": "算力芯片", "num": 60.05, "prePctChangeRate": 0.21, "numChange": -60.03, "blocks": ["885756.DDBK", "885362.DDBK", "886009.DDBK", "885957.DDBK", "886073.DDBK", "886033.DDBK"], "industry": 0, "rank": 26, "block_dict": {"885756.DDBK": {"blockCode": "885756.DDBK", "num": 40.04, "prePctChangeRate": 0.01, "numChange": -30.01, "rank": 20}, "885362.DDBK": {"blockCode": "885362.DDBK", "num": 30.02, "prePctChangeRate": 0.9, "numChange": 0.01, "rank": 49}, "886009.DDBK": {"blockCode": "886009.DDBK", "num": 10.01, "prePctChangeRate": 0.14, "numChange": 10.01, "rank": 180}, "885957.DDBK": {"blockCode": "885957.DDBK", "num": 10.01, "prePctChangeRate": 0.79, "numChange": -9.99, "rank": 181}, "886073.DDBK": {"blockCode": "886073.DDBK", "num": 10.01, "prePctChangeRate": 0.13, "numChange": -10.01, "rank": 182}, "886033.DDBK": {"blockCode": "886033.DDBK", "num": 10.01, "prePctChangeRate": 0.58, "numChange": -20.01, "rank": 183}}}, "T07.ZHBK": {"categoryCode": "T07.ZHBK", "categoryName": "机械设备", "num": 60.04, "prePctChangeRate": -0.24, "numChange": -20.02, "blocks": ["T07.ZHBK"], "industry": 1, "rank": 27, "block_dict": {"T07.ZHBK": {"blockCode": "T07.ZHBK", "num": 60.04, "prePctChangeRate": -0.24, "numChange": -20.02, "rank": 13}}}, "000036.BKDL": {"categoryCode": "000036.BKDL", "categoryName": "新材料", "num": 60.04, "prePctChangeRate": 0.09, "numChange": 20.02, "blocks": ["885544.DDBK"], "industry": 0, "rank": 28, "block_dict": {"885544.DDBK": {"blockCode": "885544.DDBK", "num": 60.04, "prePctChangeRate": 0.1, "numChange": 20.02, "rank": 14}}}, "000029.BKDL": {"categoryCode": "000029.BKDL", "categoryName": "电子类", "num": 50.04, "prePctChangeRate": -0.15, "numChange": 10.0, "blocks": ["885800.DDBK", "885738.DDBK", "885809.DDBK", "885774.DDBK", "885959.DDBK"], "industry": 0, "rank": 29, "block_dict": {"885800.DDBK": {"blockCode": "885800.DDBK", "num": 30.03, "prePctChangeRate": -0.19, "numChange": 0.0, "rank": 41}, "885738.DDBK": {"blockCode": "885738.DDBK", "num": 20.01, "prePctChangeRate": 0.07, "numChange": -0.01, "rank": 99}, "885809.DDBK": {"blockCode": "885809.DDBK", "num": 10.01, "prePctChangeRate": 0.27, "numChange": 0.0, "rank": 184}, "885774.DDBK": {"blockCode": "885774.DDBK", "num": 10.01, "prePctChangeRate": -0.2, "numChange": 10.01, "rank": 185}, "885959.DDBK": {"blockCode": "885959.DDBK", "num": 10.01, "prePctChangeRate": -0.15, "numChange": 10.01, "rank": 186}}}, "000028.BKDL": {"categoryCode": "000028.BKDL", "categoryName": "**ST板块", "num": 50.04, "prePctChangeRate": -1.12, "numChange": 30.02, "blocks": ["885699.DDBK"], "industry": 0, "rank": 30, "block_dict": {"885699.DDBK": {"blockCode": "885699.DDBK", "num": 50.04, "prePctChangeRate": -1.12, "numChange": 30.02, "rank": 17}}}, "T08.ZHBK": {"categoryCode": "T08.ZHBK", "categoryName": "电子", "num": 40.04, "prePctChangeRate": -0.1, "numChange": 20.02, "blocks": ["T08.ZHBK"], "industry": 1, "rank": 31, "block_dict": {"T08.ZHBK": {"blockCode": "T08.ZHBK", "num": 40.04, "prePctChangeRate": -0.1, "numChange": 20.02, "rank": 21}}}, "000032.BKDL": {"categoryCode": "000032.BKDL", "categoryName": "新股次新股", "num": 40.04, "prePctChangeRate": -0.2, "numChange": 0.02, "blocks": ["885598.DDBK", "885905.DDBK", "885906.DDBK"], "industry": 0, "rank": 32, "block_dict": {"885598.DDBK": {"blockCode": "885598.DDBK", "num": 40.04, "prePctChangeRate": -0.2, "numChange": 0.02, "rank": 22}, "885905.DDBK": {"blockCode": "885905.DDBK", "num": 20.02, "prePctChangeRate": -0.25, "numChange": -10.0, "rank": 83}, "885906.DDBK": {"blockCode": "885906.DDBK", "num": 10.01, "prePctChangeRate": 0.44, "numChange": 0.01, "rank": 187}}}, "T10.ZHBK": {"categoryCode": "T10.ZHBK", "categoryName": "计算机", "num": 30.03, "prePctChangeRate": 0.93, "numChange": 30.03, "blocks": ["T10.ZHBK"], "industry": 1, "rank": 33, "block_dict": {"T10.ZHBK": {"blockCode": "T10.ZHBK", "num": 30.03, "prePctChangeRate": 0.93, "numChange": 30.03, "rank": 42}}}, "000037.BKDL": {"categoryCode": "000037.BKDL", "categoryName": "**大市值股票", "num": 30.03, "prePctChangeRate": 0.29, "numChange": 0.0, "blocks": ["885520.DDBK", "883303.DDBK"], "industry": 0, "rank": 34, "block_dict": {"885520.DDBK": {"blockCode": "885520.DDBK", "num": 30.03, "prePctChangeRate": 0.24, "numChange": 10.01, "rank": 43}, "883303.DDBK": {"blockCode": "883303.DDBK", "num": 10.01, "prePctChangeRate": 0.36, "numChange": 10.01, "rank": 188}}}, "T22.ZHBK": {"categoryCode": "T22.ZHBK", "categoryName": "传媒", "num": 30.02, "prePctChangeRate": 0.36, "numChange": 20.01, "blocks": ["T22.ZHBK"], "industry": 1, "rank": 35, "block_dict": {"T22.ZHBK": {"blockCode": "T22.ZHBK", "num": 30.02, "prePctChangeRate": 0.36, "numChange": 20.01, "rank": 50}}}, "T28.ZHBK": {"categoryCode": "T28.ZHBK", "categoryName": "通信", "num": 30.02, "prePctChangeRate": 0.33, "numChange": 10.01, "blocks": ["T28.ZHBK"], "industry": 1, "rank": 36, "block_dict": {"T28.ZHBK": {"blockCode": "T28.ZHBK", "num": 30.02, "prePctChangeRate": 0.33, "numChange": 10.01, "rank": 51}}}, "T25.ZHBK": {"categoryCode": "T25.ZHBK", "categoryName": "建筑装饰", "num": 20.02, "prePctChangeRate": 0.02, "numChange": 0.0, "blocks": ["T25.ZHBK"], "industry": 1, "rank": 37, "block_dict": {"T25.ZHBK": {"blockCode": "T25.ZHBK", "num": 20.02, "prePctChangeRate": 0.02, "numChange": 0.0, "rank": 84}}}, "000024.BKDL": {"categoryCode": "000024.BKDL", "categoryName": "**农业", "num": 20.02, "prePctChangeRate": 0.38, "numChange": 10.01, "blocks": ["885706.DDBK", "885811.DDBK", "885995.DDBK"], "industry": 0, "rank": 38, "block_dict": {"885706.DDBK": {"blockCode": "885706.DDBK", "num": 10.01, "prePctChangeRate": 0.58, "numChange": 10.01, "rank": 189}, "885811.DDBK": {"blockCode": "885811.DDBK", "num": 10.01, "prePctChangeRate": 1.13, "numChange": 0.0, "rank": 190}, "885995.DDBK": {"blockCode": "885995.DDBK", "num": 10.01, "prePctChangeRate": 0.18, "numChange": 0.0, "rank": 191}}}, "000022.BKDL": {"categoryCode": "000022.BKDL", "categoryName": "**化工", "num": 20.02, "prePctChangeRate": 0.55, "numChange": -20.02, "blocks": ["885912.DDBK", "885937.DDBK"], "industry": 0, "rank": 39, "block_dict": {"885912.DDBK": {"blockCode": "885912.DDBK", "num": 10.01, "prePctChangeRate": 0.19, "numChange": 0.0, "rank": 192}, "885937.DDBK": {"blockCode": "885937.DDBK", "num": 10.01, "prePctChangeRate": 0.36, "numChange": 10.01, "rank": 193}}}, "T03.ZHBK": {"categoryCode": "T03.ZHBK", "categoryName": "基础化工", "num": 20.02, "prePctChangeRate": 0.65, "numChange": -30.02, "blocks": ["T03.ZHBK"], "industry": 1, "rank": 40, "block_dict": {"T03.ZHBK": {"blockCode": "T03.ZHBK", "num": 20.02, "prePctChangeRate": 0.65, "numChange": -30.02, "rank": 85}}}, "000001.BKDL": {"categoryCode": "000001.BKDL", "categoryName": "**有色贵金属", "num": 20.02, "prePctChangeRate": 0.55, "numChange": -20.01, "blocks": ["885973.DDBK", "885355.DDBK", "885552.DDBK", "885970.DDBK"], "industry": 0, "rank": 41, "block_dict": {"885973.DDBK": {"blockCode": "885973.DDBK", "num": 10.01, "prePctChangeRate": 1.0, "numChange": 10.01, "rank": 194}, "885355.DDBK": {"blockCode": "885355.DDBK", "num": 10.01, "prePctChangeRate": 0.72, "numChange": 0.0, "rank": 195}, "885552.DDBK": {"blockCode": "885552.DDBK", "num": 10.01, "prePctChangeRate": 0.11, "numChange": 0.01, "rank": 196}, "885970.DDBK": {"blockCode": "885970.DDBK", "num": 10.01, "prePctChangeRate": 0.74, "numChange": 10.01, "rank": 197}}}, "T24.ZHBK": {"categoryCode": "T24.ZHBK", "categoryName": "国防军工", "num": 20.01, "prePctChangeRate": -0.97, "numChange": -20.02, "blocks": ["T24.ZHBK"], "industry": 1, "rank": 42, "block_dict": {"T24.ZHBK": {"blockCode": "T24.ZHBK", "num": 20.01, "prePctChangeRate": -0.97, "numChange": -20.02, "rank": 100}}}, "T15.ZHBK": {"categoryCode": "T15.ZHBK", "categoryName": "医药生物", "num": 20.01, "prePctChangeRate": -0.34, "numChange": -30.03, "blocks": ["T15.ZHBK"], "industry": 1, "rank": 43, "block_dict": {"T15.ZHBK": {"blockCode": "T15.ZHBK", "num": 20.01, "prePctChangeRate": -0.34, "numChange": -30.03, "rank": 101}}}, "000021.BKDL": {"categoryCode": "000021.BKDL", "categoryName": "**三胎", "num": 10.01, "prePctChangeRate": -0.35, "numChange": 10.01, "blocks": ["885909.DDBK"], "industry": 0, "rank": 44, "block_dict": {"885909.DDBK": {"blockCode": "885909.DDBK", "num": 10.01, "prePctChangeRate": -0.62, "numChange": 10.01, "rank": 198}}}, "T23.ZHBK": {"categoryCode": "T23.ZHBK", "categoryName": "综合", "num": 10.01, "prePctChangeRate": -0.24, "numChange": 0.0, "blocks": ["T23.ZHBK"], "industry": 1, "rank": 45, "block_dict": {"T23.ZHBK": {"blockCode": "T23.ZHBK", "num": 10.01, "prePctChangeRate": -0.24, "numChange": 0.0, "rank": 199}}}, "T12.ZHBK": {"categoryCode": "T12.ZHBK", "categoryName": "食品饮料", "num": 10.01, "prePctChangeRate": 0.28, "numChange": 10.01, "blocks": ["T12.ZHBK"], "industry": 1, "rank": 46, "block_dict": {"T12.ZHBK": {"blockCode": "T12.ZHBK", "num": 10.01, "prePctChangeRate": 0.28, "numChange": 10.01, "rank": 200}}}, "000005.BKDL": {"categoryCode": "000005.BKDL", "categoryName": "**油气", "num": 10.01, "prePctChangeRate": 0.28, "numChange": 10.01, "blocks": ["885430.DDBK"], "industry": 0, "rank": 47, "block_dict": {"885430.DDBK": {"blockCode": "885430.DDBK", "num": 10.01, "prePctChangeRate": 0.35, "numChange": 10.01, "rank": 201}}}, "T01.ZHBK": {"categoryCode": "T01.ZHBK", "categoryName": "农林牧渔", "num": 10.01, "prePctChangeRate": 0.62, "numChange": -10.01, "blocks": ["T01.ZHBK"], "industry": 1, "rank": 48, "block_dict": {"T01.ZHBK": {"blockCode": "T01.ZHBK", "num": 10.01, "prePctChangeRate": 0.62, "numChange": -10.01, "rank": 202}}}, "T29.ZHBK": {"categoryCode": "T29.ZHBK", "categoryName": "石油石化", "num": 10.01, "prePctChangeRate": 0.88, "numChange": 0.0, "blocks": ["T29.ZHBK"], "industry": 1, "rank": 49, "block_dict": {"T29.ZHBK": {"blockCode": "T29.ZHBK", "num": 10.01, "prePctChangeRate": 0.88, "numChange": 0.0, "rank": 203}}}, "T18.ZHBK": {"categoryCode": "T18.ZHBK", "categoryName": "房地产", "num": 10.01, "prePctChangeRate": 0.8, "numChange": 10.01, "blocks": ["T18.ZHBK"], "industry": 1, "rank": 50, "block_dict": {"T18.ZHBK": {"blockCode": "T18.ZHBK", "num": 10.01, "prePctChangeRate": 0.8, "numChange": 10.01, "rank": 204}}}, "T26.ZHBK": {"categoryCode": "T26.ZHBK", "categoryName": "电力设备", "num": 10.01, "prePctChangeRate": -0.63, "numChange": -20.01, "blocks": ["T26.ZHBK"], "industry": 1, "rank": 51, "block_dict": {"T26.ZHBK": {"blockCode": "T26.ZHBK", "num": 10.01, "prePctChangeRate": -0.63, "numChange": -20.01, "rank": 205}}}, "T09.ZHBK": {"categoryCode": "T09.ZHBK", "categoryName": "汽车", "num": 10.01, "prePctChangeRate": -0.51, "numChange": -10.01, "blocks": ["T09.ZHBK"], "industry": 1, "rank": 52, "block_dict": {"T09.ZHBK": {"blockCode": "T09.ZHBK", "num": 10.01, "prePctChangeRate": -0.51, "numChange": -10.01, "rank": 206}}}, "T14.ZHBK": {"categoryCode": "T14.ZHBK", "categoryName": "轻工制造", "num": 10.01, "prePctChangeRate": 0.02, "numChange": -20.0, "blocks": ["T14.ZHBK"], "industry": 1, "rank": 53, "block_dict": {"T14.ZHBK": {"blockCode": "T14.ZHBK", "num": 10.01, "prePctChangeRate": 0.02, "numChange": -20.0, "rank": 207}}}, "T11.ZHBK": {"categoryCode": "T11.ZHBK", "categoryName": "家用电器", "num": 10.01, "prePctChangeRate": -0.71, "numChange": 10.01, "blocks": ["T11.ZHBK"], "industry": 1, "rank": 54, "block_dict": {"T11.ZHBK": {"blockCode": "T11.ZHBK", "num": 10.01, "prePctChangeRate": -0.71, "numChange": 10.01, "rank": 208}}}, "T13.ZHBK": {"categoryCode": "T13.ZHBK", "categoryName": "纺织服饰", "num": 10.0, "prePctChangeRate": 0.36, "numChange": 0.0, "blocks": ["T13.ZHBK"], "industry": 1, "rank": 55, "block_dict": {"T13.ZHBK": {"blockCode": "T13.ZHBK", "num": 10.0, "prePctChangeRate": 0.36, "numChange": 0.0, "rank": 233}}}}

In [28]:
abs = []
for c,d in z.items():
    blocks = d['blocks']
    abs.extend(blocks)

In [29]:
t = {"000038.BKDL": {"categoryCode": "000038.BKDL", "categoryName": "**中市值股票", "num": 520.21, "prePctChangeRate": -1.12, "numChange": 230.02, "blocks": ["885694.DDBK", "883304.DDBK"], "industry": 0, "rank": 1, "block_dict": {"885694.DDBK": {"blockCode": "885694.DDBK", "num": 200.07, "prePctChangeRate": -1.11, "numChange": 110.0, "rank": 2}, "883304.DDBK": {"blockCode": "883304.DDBK", "num": 40.02, "prePctChangeRate": -0.85, "numChange": 40.02, "rank": 66}}}, "000033.BKDL": {"categoryCode": "000033.BKDL", "categoryName": "智造机器人", "num": 410.14, "prePctChangeRate": -1.37, "numChange": 189.97, "blocks": ["885929.DDBK", "885517.DDBK", "885941.DDBK", "885619.DDBK", "885427.DDBK", "886008.DDBK", "886057.DDBK", "886002.DDBK", "885930.DDBK", "885786.DDBK", "885946.DDBK", "885537.DDBK", "886069.DDBK"], "industry": 0, "rank": 2, "block_dict": {"885929.DDBK": {"blockCode": "885929.DDBK", "num": 190.06, "prePctChangeRate": -1.35, "numChange": 49.96, "rank": 3}, "885517.DDBK": {"blockCode": "885517.DDBK", "num": 180.05, "prePctChangeRate": -1.65, "numChange": 140.02, "rank": 4}, "885941.DDBK": {"blockCode": "885941.DDBK", "num": 140.03, "prePctChangeRate": -1.26, "numChange": 89.98, "rank": 10}, "885619.DDBK": {"blockCode": "885619.DDBK", "num": 80.03, "prePctChangeRate": -1.24, "numChange": 50.01, "rank": 25}, "885427.DDBK": {"blockCode": "885427.DDBK", "num": 40.03, "prePctChangeRate": -1.15, "numChange": -20.01, "rank": 63}, "886008.DDBK": {"blockCode": "886008.DDBK", "num": 40.03, "prePctChangeRate": -1.98, "numChange": 40.03, "rank": 64}, "886057.DDBK": {"blockCode": "886057.DDBK", "num": 40.01, "prePctChangeRate": -1.72, "numChange": 40.01, "rank": 71}, "886002.DDBK": {"blockCode": "886002.DDBK", "num": 30.01, "prePctChangeRate": -1.36, "numChange": 20.01, "rank": 90}, "885930.DDBK": {"blockCode": "885930.DDBK", "num": 20.02, "prePctChangeRate": -1.24, "numChange": 20.02, "rank": 111}, "885786.DDBK": {"blockCode": "885786.DDBK", "num": 10.01, "prePctChangeRate": -1.58, "numChange": 10.01, "rank": 160}, "885946.DDBK": {"blockCode": "885946.DDBK", "num": 10.01, "prePctChangeRate": -1.42, "numChange": 0.01, "rank": 161}, "885537.DDBK": {"blockCode": "885537.DDBK", "num": 10.0, "prePctChangeRate": -1.5, "numChange": 10.0, "rank": 209}, "886069.DDBK": {"blockCode": "886069.DDBK", "num": 10.0, "prePctChangeRate": -2.7, "numChange": 10.0, "rank": 210}}}, "000023.BKDL": {"categoryCode": "000023.BKDL", "categoryName": "业绩股权类", "num": 390.13, "prePctChangeRate": -0.98, "numChange": 229.99, "blocks": ["885869.DDBK", "885802.DDBK", "885729.DDBK", "886072.DDBK", "885742.DDBK", "885739.DDBK", "885796.DDBK"], "industry": 0, "rank": 3, "block_dict": {"885869.DDBK": {"blockCode": "885869.DDBK", "num": 90.01, "prePctChangeRate": -1.04, "numChange": 80.0, "rank": 19}, "885802.DDBK": {"blockCode": "885802.DDBK", "num": 50.03, "prePctChangeRate": -0.21, "numChange": 20.0, "rank": 46}, "885729.DDBK": {"blockCode": "885729.DDBK", "num": 50.0, "prePctChangeRate": -1.28, "numChange": 19.97, "rank": 60}, "886072.DDBK": {"blockCode": "886072.DDBK", "num": 30.01, "prePctChangeRate": -0.22, "numChange": 30.01, "rank": 91}, "885742.DDBK": {"blockCode": "885742.DDBK", "num": 20.01, "prePctChangeRate": -0.95, "numChange": 20.01, "rank": 114}, "885739.DDBK": {"blockCode": "885739.DDBK", "num": 20.0, "prePctChangeRate": -1.07, "numChange": 20.0, "rank": 133}, "885796.DDBK": {"blockCode": "885796.DDBK", "num": 10.0, "prePctChangeRate": -1.59, "numChange": -10.02, "rank": 211}}}, "000027.BKDL": {"categoryCode": "000027.BKDL", "categoryName": "汽车低空", "num": 360.1, "prePctChangeRate": -1.39, "numChange": 149.95, "blocks": ["885431.DDBK", "885997.DDBK", "885467.DDBK", "885564.DDBK", "886067.DDBK", "885461.DDBK", "885834.DDBK", "886061.DDBK", "885662.DDBK", "886000.DDBK", "886064.DDBK", "885545.DDBK", "886066.DDBK", "885999.DDBK", "885766.DDBK", "885775.DDBK", "885871.DDBK", "885736.DDBK", "886001.DDBK"], "industry": 0, "rank": 4, "block_dict": {"885431.DDBK": {"blockCode": "885431.DDBK", "num": 180.05, "prePctChangeRate": -1.52, "numChange": 99.99, "rank": 5}, "885997.DDBK": {"blockCode": "885997.DDBK", "num": 150.07, "prePctChangeRate": -1.4, "numChange": 110.06, "rank": 6}, "885467.DDBK": {"blockCode": "885467.DDBK", "num": 60.03, "prePctChangeRate": -1.54, "numChange": 60.03, "rank": 37}, "885564.DDBK": {"blockCode": "885564.DDBK", "num": 60.02, "prePctChangeRate": -1.41, "numChange": 9.98, "rank": 40}, "886067.DDBK": {"blockCode": "886067.DDBK", "num": 60.02, "prePctChangeRate": -1.25, "numChange": 40.0, "rank": 41}, "885461.DDBK": {"blockCode": "885461.DDBK", "num": 60.01, "prePctChangeRate": -1.38, "numChange": 19.97, "rank": 43}, "885834.DDBK": {"blockCode": "885834.DDBK", "num": 50.02, "prePctChangeRate": -1.28, "numChange": 30.0, "rank": 52}, "886061.DDBK": {"blockCode": "886061.DDBK", "num": 50.01, "prePctChangeRate": -1.54, "numChange": 30.0, "rank": 55}, "885662.DDBK": {"blockCode": "885662.DDBK", "num": 40.01, "prePctChangeRate": -1.4, "numChange": 30.01, "rank": 72}, "886000.DDBK": {"blockCode": "886000.DDBK", "num": 30.02, "prePctChangeRate": -1.78, "numChange": 30.02, "rank": 84}, "886064.DDBK": {"blockCode": "886064.DDBK", "num": 30.02, "prePctChangeRate": -1.03, "numChange": 30.02, "rank": 85}, "885545.DDBK": {"blockCode": "885545.DDBK", "num": 30.01, "prePctChangeRate": -1.54, "numChange": 20.01, "rank": 92}, "886066.DDBK": {"blockCode": "886066.DDBK", "num": 30.01, "prePctChangeRate": -1.43, "numChange": 30.01, "rank": 93}, "885999.DDBK": {"blockCode": "885999.DDBK", "num": 20.01, "prePctChangeRate": -1.4, "numChange": 20.01, "rank": 115}, "885766.DDBK": {"blockCode": "885766.DDBK", "num": 20.0, "prePctChangeRate": -1.48, "numChange": 9.99, "rank": 134}, "885775.DDBK": {"blockCode": "885775.DDBK", "num": 10.01, "prePctChangeRate": -1.13, "numChange": -10.01, "rank": 162}, "885871.DDBK": {"blockCode": "885871.DDBK", "num": 10.01, "prePctChangeRate": -1.01, "numChange": 10.01, "rank": 163}, "885736.DDBK": {"blockCode": "885736.DDBK", "num": 10.0, "prePctChangeRate": -1.34, "numChange": 0.0, "rank": 212}, "886001.DDBK": {"blockCode": "886001.DDBK", "num": 10.0, "prePctChangeRate": -1.6, "numChange": 10.0, "rank": 213}}}, "000004.BKDL": {"categoryCode": "000004.BKDL", "categoryName": "**区域经济", "num": 300.12, "prePctChangeRate": -0.95, "numChange": 119.97, "blocks": ["885924.DDBK", "885832.DDBK", "885521.DDBK", "885514.DDBK", "885744.DDBK", "885617.DDBK", "885939.DDBK", "885734.DDBK", "885894.DDBK"], "industry": 0, "rank": 5, "block_dict": {"885924.DDBK": {"blockCode": "885924.DDBK", "num": 70.04, "prePctChangeRate": -0.54, "numChange": 40.01, "rank": 28}, "885832.DDBK": {"blockCode": "885832.DDBK", "num": 50.03, "prePctChangeRate": -1.09, "numChange": 40.02, "rank": 47}, "885521.DDBK": {"blockCode": "885521.DDBK", "num": 40.01, "prePctChangeRate": -1.07, "numChange": 40.01, "rank": 73}, "885514.DDBK": {"blockCode": "885514.DDBK", "num": 30.01, "prePctChangeRate": -0.98, "numChange": 10.0, "rank": 94}, "885744.DDBK": {"blockCode": "885744.DDBK", "num": 20.0, "prePctChangeRate": -1.24, "numChange": -0.02, "rank": 135}, "885617.DDBK": {"blockCode": "885617.DDBK", "num": 10.01, "prePctChangeRate": -1.1, "numChange": 0.01, "rank": 164}, "885939.DDBK": {"blockCode": "885939.DDBK", "num": 10.01, "prePctChangeRate": -1.2, "numChange": 10.01, "rank": 165}, "885734.DDBK": {"blockCode": "885734.DDBK", "num": 10.0, "prePctChangeRate": -0.89, "numChange": 10.0, "rank": 214}, "885894.DDBK": {"blockCode": "885894.DDBK", "num": 10.0, "prePctChangeRate": -1.19, "numChange": 10.0, "rank": 215}}}, "000035.BKDL": {"categoryCode": "000035.BKDL", "categoryName": "**风光储电", "num": 280.12, "prePctChangeRate": -1.07, "numChange": 69.94, "blocks": ["885531.DDBK", "885789.DDBK", "886032.DDBK", "885710.DDBK", "885921.DDBK", "885936.DDBK", "885641.DDBK", "885944.DDBK", "885928.DDBK", "885819.DDBK", "885920.DDBK", "885922.DDBK", "886065.DDBK", "885311.DDBK", "885571.DDBK", "885878.DDBK", "886011.DDBK"], "industry": 0, "rank": 6, "block_dict": {"885531.DDBK": {"blockCode": "885531.DDBK", "num": 90.04, "prePctChangeRate": -1.21, "numChange": -10.04, "rank": 18}, "885789.DDBK": {"blockCode": "885789.DDBK", "num": 80.04, "prePctChangeRate": -1.23, "numChange": 80.04, "rank": 23}, "886032.DDBK": {"blockCode": "886032.DDBK", "num": 70.03, "prePctChangeRate": -0.15, "numChange": 70.03, "rank": 29}, "885710.DDBK": {"blockCode": "885710.DDBK", "num": 60.01, "prePctChangeRate": -0.85, "numChange": 40.0, "rank": 44}, "885921.DDBK": {"blockCode": "885921.DDBK", "num": 50.02, "prePctChangeRate": -1.08, "numChange": -0.03, "rank": 53}, "885936.DDBK": {"blockCode": "885936.DDBK", "num": 40.02, "prePctChangeRate": -0.18, "numChange": -10.02, "rank": 67}, "885641.DDBK": {"blockCode": "885641.DDBK", "num": 30.01, "prePctChangeRate": -0.95, "numChange": -30.04, "rank": 95}, "885944.DDBK": {"blockCode": "885944.DDBK", "num": 30.01, "prePctChangeRate": -0.82, "numChange": 30.01, "rank": 96}, "885928.DDBK": {"blockCode": "885928.DDBK", "num": 20.01, "prePctChangeRate": -0.18, "numChange": 20.01, "rank": 116}, "885819.DDBK": {"blockCode": "885819.DDBK", "num": 20.0, "prePctChangeRate": -1.41, "numChange": 20.0, "rank": 136}, "885920.DDBK": {"blockCode": "885920.DDBK", "num": 10.01, "prePctChangeRate": -1.08, "numChange": -10.01, "rank": 166}, "885922.DDBK": {"blockCode": "885922.DDBK", "num": 10.01, "prePctChangeRate": -0.78, "numChange": 0.0, "rank": 167}, "886065.DDBK": {"blockCode": "886065.DDBK", "num": 10.01, "prePctChangeRate": -1.87, "numChange": -20.02, "rank": 168}, "885311.DDBK": {"blockCode": "885311.DDBK", "num": 10.0, "prePctChangeRate": -1.42, "numChange": -10.02, "rank": 216}, "885571.DDBK": {"blockCode": "885571.DDBK", "num": 10.0, "prePctChangeRate": -1.3, "numChange": -10.02, "rank": 217}, "885878.DDBK": {"blockCode": "885878.DDBK", "num": 10.0, "prePctChangeRate": -1.85, "numChange": 10.0, "rank": 218}, "886011.DDBK": {"blockCode": "886011.DDBK", "num": 10.0, "prePctChangeRate": -2.22, "numChange": 10.0, "rank": 219}}}, "000010.BKDL": {"categoryCode": "000010.BKDL", "categoryName": "**大消费类", "num": 270.12, "prePctChangeRate": -0.81, "numChange": 190.05, "blocks": ["885420.DDBK", "885784.DDBK", "885705.DDBK", "885543.DDBK", "885910.DDBK", "885990.DDBK", "885406.DDBK", "885888.DDBK", "885629.DDBK", "885851.DDBK", "885988.DDBK"], "industry": 0, "rank": 7, "block_dict": {"885420.DDBK": {"blockCode": "885420.DDBK", "num": 140.09, "prePctChangeRate": -0.53, "numChange": 90.05, "rank": 8}, "885784.DDBK": {"blockCode": "885784.DDBK", "num": 80.05, "prePctChangeRate": -0.84, "numChange": 40.01, "rank": 20}, "885705.DDBK": {"blockCode": "885705.DDBK", "num": 70.03, "prePctChangeRate": -1.06, "numChange": 40.0, "rank": 30}, "885543.DDBK": {"blockCode": "885543.DDBK", "num": 30.0, "prePctChangeRate": -0.65, "numChange": 19.99, "rank": 100}, "885910.DDBK": {"blockCode": "885910.DDBK", "num": 20.01, "prePctChangeRate": -1.51, "numChange": 20.01, "rank": 117}, "885990.DDBK": {"blockCode": "885990.DDBK", "num": 20.0, "prePctChangeRate": -0.85, "numChange": 20.0, "rank": 137}, "885406.DDBK": {"blockCode": "885406.DDBK", "num": 10.01, "prePctChangeRate": -1.01, "numChange": 10.01, "rank": 169}, "885888.DDBK": {"blockCode": "885888.DDBK", "num": 10.01, "prePctChangeRate": -1.07, "numChange": 0.0, "rank": 170}, "885629.DDBK": {"blockCode": "885629.DDBK", "num": 10.0, "prePctChangeRate": -1.53, "numChange": 10.0, "rank": 220}, "885851.DDBK": {"blockCode": "885851.DDBK", "num": 10.0, "prePctChangeRate": -0.78, "numChange": 10.0, "rank": 221}, "885988.DDBK": {"blockCode": "885988.DDBK", "num": 10.0, "prePctChangeRate": -1.11, "numChange": 10.0, "rank": 222}}}, "000026.BKDL": {"categoryCode": "000026.BKDL", "categoryName": "**外贸出口", "num": 270.1, "prePctChangeRate": -1.22, "numChange": 160.03, "blocks": ["885840.DDBK", "885642.DDBK", "885965.DDBK", "885611.DDBK"], "industry": 0, "rank": 8, "block_dict": {"885840.DDBK": {"blockCode": "885840.DDBK", "num": 200.08, "prePctChangeRate": -1.01, "numChange": 140.06, "rank": 1}, "885642.DDBK": {"blockCode": "885642.DDBK", "num": 120.06, "prePctChangeRate": -0.99, "numChange": 70.02, "rank": 14}, "885965.DDBK": {"blockCode": "885965.DDBK", "num": 30.01, "prePctChangeRate": -1.05, "numChange": 9.99, "rank": 97}, "885611.DDBK": {"blockCode": "885611.DDBK", "num": 10.0, "prePctChangeRate": -2.22, "numChange": -20.03, "rank": 223}}}, "000019.BKDL": {"categoryCode": "000019.BKDL", "categoryName": "其他小概念", "num": 250.12, "prePctChangeRate": -0.76, "numChange": 100.0, "blocks": ["885522.DDBK", "886012.DDBK", "885825.DDBK"], "industry": 0, "rank": 9, "block_dict": {"885522.DDBK": {"blockCode": "885522.DDBK", "num": 50.01, "prePctChangeRate": -1.3, "numChange": 40.0, "rank": 56}, "886012.DDBK": {"blockCode": "886012.DDBK", "num": 10.01, "prePctChangeRate": -0.66, "numChange": 0.01, "rank": 171}, "885825.DDBK": {"blockCode": "885825.DDBK", "num": 10.0, "prePctChangeRate": -1.74, "numChange": -0.01, "rank": 224}}}, "000009.BKDL": {"categoryCode": "000009.BKDL", "categoryName": "**医药医疗", "num": 230.11, "prePctChangeRate": -0.75, "numChange": 140.04, "blocks": ["885880.DDBK", "885539.DDBK", "885818.DDBK", "885952.DDBK", "886022.DDBK", "885765.DDBK", "885913.DDBK", "885989.DDBK", "886077.DDBK", "885883.DDBK", "885993.DDBK", "885402.DDBK", "885403.DDBK", "885490.DDBK", "885578.DDBK", "885703.DDBK", "885838.DDBK", "885856.DDBK", "885879.DDBK", "885900.DDBK", "885927.DDBK", "885954.DDBK", "885955.DDBK", "885983.DDBK", "885994.DDBK", "886015.DDBK", "886018.DDBK", "886024.DDBK", "886056.DDBK"], "industry": 0, "rank": 10, "block_dict": {"885880.DDBK": {"blockCode": "885880.DDBK", "num": 80.05, "prePctChangeRate": -0.26, "numChange": 60.04, "rank": 21}, "885539.DDBK": {"blockCode": "885539.DDBK", "num": 50.03, "prePctChangeRate": -0.77, "numChange": 10.0, "rank": 48}, "885818.DDBK": {"blockCode": "885818.DDBK", "num": 30.02, "prePctChangeRate": -0.74, "numChange": 20.01, "rank": 86}, "885952.DDBK": {"blockCode": "885952.DDBK", "num": 20.01, "prePctChangeRate": -0.64, "numChange": 20.01, "rank": 118}, "886022.DDBK": {"blockCode": "886022.DDBK", "num": 20.01, "prePctChangeRate": -0.43, "numChange": 20.01, "rank": 119}, "885765.DDBK": {"blockCode": "885765.DDBK", "num": 20.0, "prePctChangeRate": -1.6, "numChange": -0.02, "rank": 138}, "885913.DDBK": {"blockCode": "885913.DDBK", "num": 20.0, "prePctChangeRate": -0.78, "numChange": 9.99, "rank": 139}, "885989.DDBK": {"blockCode": "885989.DDBK", "num": 20.0, "prePctChangeRate": -0.8, "numChange": 9.99, "rank": 140}, "886077.DDBK": {"blockCode": "886077.DDBK", "num": 20.0, "prePctChangeRate": -0.59, "numChange": -0.01, "rank": 141}, "885883.DDBK": {"blockCode": "885883.DDBK", "num": 10.01, "prePctChangeRate": -0.66, "numChange": 0.0, "rank": 172}, "885993.DDBK": {"blockCode": "885993.DDBK", "num": 10.01, "prePctChangeRate": -1.12, "numChange": 10.01, "rank": 173}, "885402.DDBK": {"blockCode": "885402.DDBK", "num": 10.0, "prePctChangeRate": -1.67, "numChange": 10.0, "rank": 225}, "885403.DDBK": {"blockCode": "885403.DDBK", "num": 10.0, "prePctChangeRate": -0.62, "numChange": -0.01, "rank": 226}, "885490.DDBK": {"blockCode": "885490.DDBK", "num": 10.0, "prePctChangeRate": -1.3, "numChange": -0.01, "rank": 227}, "885578.DDBK": {"blockCode": "885578.DDBK", "num": 10.0, "prePctChangeRate": -1.0, "numChange": 10.0, "rank": 228}, "885703.DDBK": {"blockCode": "885703.DDBK", "num": 10.0, "prePctChangeRate": -0.55, "numChange": 10.0, "rank": 229}, "885838.DDBK": {"blockCode": "885838.DDBK", "num": 10.0, "prePctChangeRate": -0.62, "numChange": -0.01, "rank": 230}, "885856.DDBK": {"blockCode": "885856.DDBK", "num": 10.0, "prePctChangeRate": -0.61, "numChange": 10.0, "rank": 231}, "885879.DDBK": {"blockCode": "885879.DDBK", "num": 10.0, "prePctChangeRate": -0.44, "numChange": 10.0, "rank": 232}, "885900.DDBK": {"blockCode": "885900.DDBK", "num": 10.0, "prePctChangeRate": -0.97, "numChange": 10.0, "rank": 233}, "885927.DDBK": {"blockCode": "885927.DDBK", "num": 10.0, "prePctChangeRate": -1.08, "numChange": -0.01, "rank": 234}, "885954.DDBK": {"blockCode": "885954.DDBK", "num": 10.0, "prePctChangeRate": -0.51, "numChange": 10.0, "rank": 235}, "885955.DDBK": {"blockCode": "885955.DDBK", "num": 10.0, "prePctChangeRate": -0.32, "numChange": 10.0, "rank": 236}, "885983.DDBK": {"blockCode": "885983.DDBK", "num": 10.0, "prePctChangeRate": -1.38, "numChange": -0.01, "rank": 237}, "885994.DDBK": {"blockCode": "885994.DDBK", "num": 10.0, "prePctChangeRate": -0.62, "numChange": 10.0, "rank": 238}, "886015.DDBK": {"blockCode": "886015.DDBK", "num": 10.0, "prePctChangeRate": -0.51, "numChange": -0.01, "rank": 239}, "886018.DDBK": {"blockCode": "886018.DDBK", "num": 10.0, "prePctChangeRate": -1.49, "numChange": -0.01, "rank": 240}, "886024.DDBK": {"blockCode": "886024.DDBK", "num": 10.0, "prePctChangeRate": -0.33, "numChange": 10.0, "rank": 241}, "886056.DDBK": {"blockCode": "886056.DDBK", "num": 10.0, "prePctChangeRate": -0.65, "numChange": 10.0, "rank": 242}}}, "000025.BKDL": {"categoryCode": "000025.BKDL", "categoryName": "通信类", "num": 220.08, "prePctChangeRate": -1.46, "numChange": 59.96, "blocks": ["885556.DDBK", "885312.DDBK", "885783.DDBK", "885998.DDBK", "885940.DDBK", "885574.DDBK", "885868.DDBK", "886037.DDBK"], "industry": 0, "rank": 11, "block_dict": {"885556.DDBK": {"blockCode": "885556.DDBK", "num": 130.04, "prePctChangeRate": -1.44, "numChange": 49.98, "rank": 12}, "885312.DDBK": {"blockCode": "885312.DDBK", "num": 80.03, "prePctChangeRate": -1.68, "numChange": 29.99, "rank": 26}, "885783.DDBK": {"blockCode": "885783.DDBK", "num": 70.03, "prePctChangeRate": -1.47, "numChange": 29.99, "rank": 31}, "885998.DDBK": {"blockCode": "885998.DDBK", "num": 30.01, "prePctChangeRate": -1.21, "numChange": 30.01, "rank": 98}, "885940.DDBK": {"blockCode": "885940.DDBK", "num": 20.01, "prePctChangeRate": -1.91, "numChange": 20.01, "rank": 120}, "885574.DDBK": {"blockCode": "885574.DDBK", "num": 20.0, "prePctChangeRate": -1.07, "numChange": -30.03, "rank": 142}, "885868.DDBK": {"blockCode": "885868.DDBK", "num": 10.01, "prePctChangeRate": -1.87, "numChange": 0.0, "rank": 174}, "886037.DDBK": {"blockCode": "886037.DDBK", "num": 10.0, "prePctChangeRate": -0.74, "numChange": -10.02, "rank": 243}}}, "000012.BKDL": {"categoryCode": "000012.BKDL", "categoryName": "传媒游戏娱乐", "num": 170.07, "prePctChangeRate": -1.32, "numChange": 69.98, "blocks": ["885788.DDBK", "885615.DDBK", "885457.DDBK", "885890.DDBK", "885876.DDBK", "885418.DDBK", "885497.DDBK", "885480.DDBK", "885709.DDBK", "885772.DDBK", "885779.DDBK", "885797.DDBK", "885701.DDBK", "885792.DDBK", "886014.DDBK", "886046.DDBK", "885580.DDBK", "885586.DDBK", "885603.DDBK", "885820.DDBK", "885934.DDBK"], "industry": 0, "rank": 12, "block_dict": {"885788.DDBK": {"blockCode": "885788.DDBK", "num": 60.03, "prePctChangeRate": -0.91, "numChange": 50.03, "rank": 38}, "885615.DDBK": {"blockCode": "885615.DDBK", "num": 40.01, "prePctChangeRate": -1.35, "numChange": 30.0, "rank": 74}, "885457.DDBK": {"blockCode": "885457.DDBK", "num": 30.0, "prePctChangeRate": -1.54, "numChange": 30.0, "rank": 101}, "885890.DDBK": {"blockCode": "885890.DDBK", "num": 30.0, "prePctChangeRate": -1.57, "numChange": 30.0, "rank": 102}, "885876.DDBK": {"blockCode": "885876.DDBK", "num": 20.02, "prePctChangeRate": -0.89, "numChange": 20.02, "rank": 112}, "885418.DDBK": {"blockCode": "885418.DDBK", "num": 20.01, "prePctChangeRate": -1.63, "numChange": 20.01, "rank": 121}, "885497.DDBK": {"blockCode": "885497.DDBK", "num": 20.01, "prePctChangeRate": -1.03, "numChange": -0.01, "rank": 122}, "885480.DDBK": {"blockCode": "885480.DDBK", "num": 20.0, "prePctChangeRate": -1.65, "numChange": -10.03, "rank": 143}, "885709.DDBK": {"blockCode": "885709.DDBK", "num": 20.0, "prePctChangeRate": -1.79, "numChange": 20.0, "rank": 144}, "885772.DDBK": {"blockCode": "885772.DDBK", "num": 20.0, "prePctChangeRate": -1.9, "numChange": 20.0, "rank": 145}, "885779.DDBK": {"blockCode": "885779.DDBK", "num": 20.0, "prePctChangeRate": -2.03, "numChange": -0.02, "rank": 146}, "885797.DDBK": {"blockCode": "885797.DDBK", "num": 20.0, "prePctChangeRate": -1.71, "numChange": -0.02, "rank": 147}, "885701.DDBK": {"blockCode": "885701.DDBK", "num": 10.01, "prePctChangeRate": -1.2, "numChange": 0.0, "rank": 175}, "885792.DDBK": {"blockCode": "885792.DDBK", "num": 10.01, "prePctChangeRate": 0.4, "numChange": 0.0, "rank": 176}, "886014.DDBK": {"blockCode": "886014.DDBK", "num": 10.01, "prePctChangeRate": -1.22, "numChange": 10.01, "rank": 177}, "886046.DDBK": {"blockCode": "886046.DDBK", "num": 10.01, "prePctChangeRate": -1.89, "numChange": 10.01, "rank": 178}, "885580.DDBK": {"blockCode": "885580.DDBK", "num": 10.0, "prePctChangeRate": -1.08, "numChange": 10.0, "rank": 244}, "885586.DDBK": {"blockCode": "885586.DDBK", "num": 10.0, "prePctChangeRate": -1.54, "numChange": -10.02, "rank": 245}, "885603.DDBK": {"blockCode": "885603.DDBK", "num": 10.0, "prePctChangeRate": -1.6, "numChange": 10.0, "rank": 246}, "885820.DDBK": {"blockCode": "885820.DDBK", "num": 10.0, "prePctChangeRate": -1.79, "numChange": -0.01, "rank": 247}, "885934.DDBK": {"blockCode": "885934.DDBK", "num": 10.0, "prePctChangeRate": -1.86, "numChange": -10.02, "rank": 248}}}, "000003.BKDL": {"categoryCode": "000003.BKDL", "categoryName": "数字科技", "num": 170.05, "prePctChangeRate": -1.52, "numChange": 49.94, "blocks": ["885887.DDBK", "885976.DDBK", "885452.DDBK", "886041.DDBK", "885757.DDBK", "886013.DDBK", "885842.DDBK", "885933.DDBK", "885333.DDBK", "885866.DDBK", "885942.DDBK", "885966.DDBK"], "industry": 0, "rank": 13, "block_dict": {"885887.DDBK": {"blockCode": "885887.DDBK", "num": 70.0, "prePctChangeRate": -1.62, "numChange": 19.95, "rank": 35}, "885976.DDBK": {"blockCode": "885976.DDBK", "num": 50.03, "prePctChangeRate": -1.56, "numChange": 20.0, "rank": 49}, "885452.DDBK": {"blockCode": "885452.DDBK", "num": 50.01, "prePctChangeRate": -1.84, "numChange": 19.98, "rank": 57}, "886041.DDBK": {"blockCode": "886041.DDBK", "num": 30.0, "prePctChangeRate": -1.87, "numChange": 9.98, "rank": 103}, "885757.DDBK": {"blockCode": "885757.DDBK", "num": 20.0, "prePctChangeRate": -1.95, "numChange": -10.03, "rank": 148}, "886013.DDBK": {"blockCode": "886013.DDBK", "num": 20.0, "prePctChangeRate": -2.09, "numChange": -30.04, "rank": 149}, "885842.DDBK": {"blockCode": "885842.DDBK", "num": 10.01, "prePctChangeRate": -1.72, "numChange": 0.0, "rank": 179}, "885933.DDBK": {"blockCode": "885933.DDBK", "num": 10.01, "prePctChangeRate": -1.18, "numChange": 10.01, "rank": 180}, "885333.DDBK": {"blockCode": "885333.DDBK", "num": 10.0, "prePctChangeRate": -1.92, "numChange": -10.02, "rank": 249}, "885866.DDBK": {"blockCode": "885866.DDBK", "num": 10.0, "prePctChangeRate": -1.73, "numChange": -0.01, "rank": 250}, "885942.DDBK": {"blockCode": "885942.DDBK", "num": 10.0, "prePctChangeRate": -1.93, "numChange": 10.0, "rank": 251}, "885966.DDBK": {"blockCode": "885966.DDBK", "num": 10.0, "prePctChangeRate": -1.07, "numChange": -0.01, "rank": 252}}}, "000006.BKDL": {"categoryCode": "000006.BKDL", "categoryName": "华为大科技", "num": 170.05, "prePctChangeRate": -1.63, "numChange": 69.98, "blocks": ["885806.DDBK", "885923.DDBK", "885376.DDBK", "886059.DDBK", "885980.DDBK", "885982.DDBK", "886055.DDBK"], "industry": 0, "rank": 14, "block_dict": {"885806.DDBK": {"blockCode": "885806.DDBK", "num": 140.04, "prePctChangeRate": -1.68, "numChange": 49.98, "rank": 9}, "885923.DDBK": {"blockCode": "885923.DDBK", "num": 30.0, "prePctChangeRate": -1.49, "numChange": 9.98, "rank": 104}, "885376.DDBK": {"blockCode": "885376.DDBK", "num": 10.01, "prePctChangeRate": -1.78, "numChange": 10.01, "rank": 181}, "886059.DDBK": {"blockCode": "886059.DDBK", "num": 10.01, "prePctChangeRate": -1.76, "numChange": 10.01, "rank": 182}, "885980.DDBK": {"blockCode": "885980.DDBK", "num": 10.0, "prePctChangeRate": -1.69, "numChange": -10.01, "rank": 253}, "885982.DDBK": {"blockCode": "885982.DDBK", "num": 10.0, "prePctChangeRate": -2.12, "numChange": -10.01, "rank": 254}, "886055.DDBK": {"blockCode": "886055.DDBK", "num": 10.0, "prePctChangeRate": -1.97, "numChange": 10.0, "rank": 255}}}, "000034.BKDL": {"categoryCode": "000034.BKDL", "categoryName": "人工智能", "num": 170.03, "prePctChangeRate": -1.62, "numChange": 79.95, "blocks": ["885728.DDBK", "886019.DDBK", "886031.DDBK", "886062.DDBK"], "industry": 0, "rank": 15, "block_dict": {"885728.DDBK": {"blockCode": "885728.DDBK", "num": 130.02, "prePctChangeRate": -1.74, "numChange": 69.96, "rank": 13}, "886019.DDBK": {"blockCode": "886019.DDBK", "num": 10.01, "prePctChangeRate": -1.91, "numChange": 0.0, "rank": 183}, "886031.DDBK": {"blockCode": "886031.DDBK", "num": 10.0, "prePctChangeRate": -2.08, "numChange": -0.01, "rank": 256}, "886062.DDBK": {"blockCode": "886062.DDBK", "num": 10.0, "prePctChangeRate": -2.17, "numChange": 10.0, "rank": 257}}}, "000013.BKDL": {"categoryCode": "000013.BKDL", "categoryName": "**国防军工", "num": 150.05, "prePctChangeRate": -1.24, "numChange": 29.93, "blocks": ["885700.DDBK", "885743.DDBK", "886076.DDBK", "885566.DDBK", "886030.DDBK", "885860.DDBK"], "industry": 0, "rank": 16, "block_dict": {"885700.DDBK": {"blockCode": "885700.DDBK", "num": 150.05, "prePctChangeRate": -1.23, "numChange": 49.95, "rank": 7}, "885743.DDBK": {"blockCode": "885743.DDBK", "num": 50.01, "prePctChangeRate": -1.1, "numChange": -10.05, "rank": 58}, "886076.DDBK": {"blockCode": "886076.DDBK", "num": 30.0, "prePctChangeRate": -0.61, "numChange": 9.98, "rank": 105}, "885566.DDBK": {"blockCode": "885566.DDBK", "num": 20.01, "prePctChangeRate": -1.7, "numChange": -10.02, "rank": 123}, "886030.DDBK": {"blockCode": "886030.DDBK", "num": 20.01, "prePctChangeRate": -1.03, "numChange": -0.01, "rank": 124}, "885860.DDBK": {"blockCode": "885860.DDBK", "num": 10.0, "prePctChangeRate": -0.53, "numChange": 10.0, "rank": 258}}}, "000031.BKDL": {"categoryCode": "000031.BKDL", "categoryName": "算力芯片", "num": 150.04, "prePctChangeRate": -1.87, "numChange": 29.94, "blocks": ["885756.DDBK", "886033.DDBK", "885790.DDBK", "885362.DDBK", "885957.DDBK", "885925.DDBK", "886044.DDBK", "885908.DDBK", "886042.DDBK", "886050.DDBK", "886073.DDBK"], "industry": 0, "rank": 17, "block_dict": {"885756.DDBK": {"blockCode": "885756.DDBK", "num": 50.02, "prePctChangeRate": -1.92, "numChange": 9.98, "rank": 54}, "886033.DDBK": {"blockCode": "886033.DDBK", "num": 50.01, "prePctChangeRate": -1.71, "numChange": 40.0, "rank": 59}, "885790.DDBK": {"blockCode": "885790.DDBK", "num": 40.01, "prePctChangeRate": -1.96, "numChange": 40.01, "rank": 75}, "885362.DDBK": {"blockCode": "885362.DDBK", "num": 40.0, "prePctChangeRate": -1.94, "numChange": -10.04, "rank": 80}, "885957.DDBK": {"blockCode": "885957.DDBK", "num": 20.0, "prePctChangeRate": -1.8, "numChange": -0.02, "rank": 150}, "885925.DDBK": {"blockCode": "885925.DDBK", "num": 10.01, "prePctChangeRate": -1.85, "numChange": 10.01, "rank": 184}, "886044.DDBK": {"blockCode": "886044.DDBK", "num": 10.01, "prePctChangeRate": -1.73, "numChange": -10.01, "rank": 185}, "885908.DDBK": {"blockCode": "885908.DDBK", "num": 10.0, "prePctChangeRate": -1.58, "numChange": -10.02, "rank": 259}, "886042.DDBK": {"blockCode": "886042.DDBK", "num": 10.0, "prePctChangeRate": -2.68, "numChange": -0.01, "rank": 260}, "886050.DDBK": {"blockCode": "886050.DDBK", "num": 10.0, "prePctChangeRate": -1.7, "numChange": -10.02, "rank": 261}, "886073.DDBK": {"blockCode": "886073.DDBK", "num": 10.0, "prePctChangeRate": -2.11, "numChange": 0.0, "rank": 262}}}, "T09.ZHBK": {"categoryCode": "T09.ZHBK", "categoryName": "汽车", "num": 140.03, "prePctChangeRate": -1.53, "numChange": 120.02, "blocks": ["T09.ZHBK"], "industry": 1, "rank": 18, "block_dict": {"T09.ZHBK": {"blockCode": "T09.ZHBK", "num": 140.03, "prePctChangeRate": -1.53, "numChange": 120.02, "rank": 11}}}, "000015.BKDL": {"categoryCode": "000015.BKDL", "categoryName": "**非银金融", "num": 130.08, "prePctChangeRate": -1.0, "numChange": 30.0, "blocks": ["885413.DDBK", "885456.DDBK", "885657.DDBK", "885620.DDBK", "885835.DDBK", "885749.DDBK"], "industry": 0, "rank": 19, "block_dict": {"885413.DDBK": {"blockCode": "885413.DDBK", "num": 60.03, "prePctChangeRate": -1.01, "numChange": 30.0, "rank": 39}, "885456.DDBK": {"blockCode": "885456.DDBK", "num": 40.02, "prePctChangeRate": -1.7, "numChange": 20.0, "rank": 68}, "885657.DDBK": {"blockCode": "885657.DDBK", "num": 30.01, "prePctChangeRate": -0.89, "numChange": 30.01, "rank": 99}, "885620.DDBK": {"blockCode": "885620.DDBK", "num": 10.01, "prePctChangeRate": -0.75, "numChange": -10.0, "rank": 186}, "885835.DDBK": {"blockCode": "885835.DDBK", "num": 10.01, "prePctChangeRate": -0.83, "numChange": -10.01, "rank": 187}, "885749.DDBK": {"blockCode": "885749.DDBK", "num": 10.0, "prePctChangeRate": -1.17, "numChange": -0.01, "rank": 263}}}, "000014.BKDL": {"categoryCode": "000014.BKDL", "categoryName": "泛科技", "num": 130.03, "prePctChangeRate": -1.69, "numChange": 80.0, "blocks": ["885785.DDBK", "885759.DDBK", "886036.DDBK", "886047.DDBK", "885454.DDBK"], "industry": 0, "rank": 20, "block_dict": {"885785.DDBK": {"blockCode": "885785.DDBK", "num": 50.03, "prePctChangeRate": -1.58, "numChange": 20.02, "rank": 50}, "885759.DDBK": {"blockCode": "885759.DDBK", "num": 40.0, "prePctChangeRate": -1.91, "numChange": 40.0, "rank": 81}, "886036.DDBK": {"blockCode": "886036.DDBK", "num": 20.0, "prePctChangeRate": -1.69, "numChange": 9.99, "rank": 151}, "886047.DDBK": {"blockCode": "886047.DDBK", "num": 20.0, "prePctChangeRate": -1.91, "numChange": 9.99, "rank": 152}, "885454.DDBK": {"blockCode": "885454.DDBK", "num": 10.0, "prePctChangeRate": -1.91, "numChange": 10.0, "rank": 264}}}, "T07.ZHBK": {"categoryCode": "T07.ZHBK", "categoryName": "机械设备", "num": 110.07, "prePctChangeRate": -1.3, "numChange": 20.0, "blocks": ["T07.ZHBK"], "industry": 1, "rank": 21, "block_dict": {"T07.ZHBK": {"blockCode": "T07.ZHBK", "num": 110.07, "prePctChangeRate": -1.3, "numChange": 20.0, "rank": 15}}}, "000017.BKDL": {"categoryCode": "000017.BKDL", "categoryName": "**中字头国企", "num": 110.06, "prePctChangeRate": -0.79, "numChange": 40.01, "blocks": ["886021.DDBK", "885595.DDBK", "885741.DDBK"], "industry": 0, "rank": 22, "block_dict": {"886021.DDBK": {"blockCode": "886021.DDBK", "num": 110.06, "prePctChangeRate": -0.75, "numChange": 40.01, "rank": 16}, "885595.DDBK": {"blockCode": "885595.DDBK", "num": 40.01, "prePctChangeRate": -0.68, "numChange": 9.99, "rank": 76}, "885741.DDBK": {"blockCode": "885741.DDBK", "num": 10.0, "prePctChangeRate": -0.6, "numChange": 10.0, "rank": 265}}}, "T13.ZHBK": {"categoryCode": "T13.ZHBK", "categoryName": "纺织服饰", "num": 110.05, "prePctChangeRate": 1.26, "numChange": 90.04, "blocks": ["T13.ZHBK"], "industry": 1, "rank": 23, "block_dict": {"T13.ZHBK": {"blockCode": "T13.ZHBK", "num": 110.05, "prePctChangeRate": 1.26, "numChange": 90.04, "rank": 17}}}, "000011.BKDL": {"categoryCode": "000011.BKDL", "categoryName": "**环保碳中和", "num": 110.04, "prePctChangeRate": -0.85, "numChange": 29.97, "blocks": ["885690.DDBK", "885823.DDBK", "885412.DDBK", "885410.DDBK", "885563.DDBK", "885854.DDBK", "885961.DDBK"], "industry": 0, "rank": 24, "block_dict": {"885690.DDBK": {"blockCode": "885690.DDBK", "num": 40.02, "prePctChangeRate": -0.89, "numChange": 20.0, "rank": 69}, "885823.DDBK": {"blockCode": "885823.DDBK", "num": 40.01, "prePctChangeRate": -1.02, "numChange": 30.0, "rank": 77}, "885412.DDBK": {"blockCode": "885412.DDBK", "num": 30.02, "prePctChangeRate": -0.79, "numChange": 10.0, "rank": 87}, "885410.DDBK": {"blockCode": "885410.DDBK", "num": 10.01, "prePctChangeRate": -0.72, "numChange": 0.0, "rank": 188}, "885563.DDBK": {"blockCode": "885563.DDBK", "num": 10.01, "prePctChangeRate": -0.68, "numChange": -30.02, "rank": 189}, "885854.DDBK": {"blockCode": "885854.DDBK", "num": 10.01, "prePctChangeRate": -0.77, "numChange": -10.01, "rank": 190}, "885961.DDBK": {"blockCode": "885961.DDBK", "num": 10.0, "prePctChangeRate": -0.8, "numChange": 10.0, "rank": 266}}}, "000018.BKDL": {"categoryCode": "000018.BKDL", "categoryName": "**一带一路", "num": 110.02, "prePctChangeRate": -1.16, "numChange": 69.98, "blocks": ["885494.DDBK", "885770.DDBK", "885964.DDBK", "885733.DDBK"], "industry": 0, "rank": 25, "block_dict": {"885494.DDBK": {"blockCode": "885494.DDBK", "num": 80.02, "prePctChangeRate": -1.12, "numChange": 60.0, "rank": 27}, "885770.DDBK": {"blockCode": "885770.DDBK", "num": 30.0, "prePctChangeRate": -1.4, "numChange": 19.99, "rank": 106}, "885964.DDBK": {"blockCode": "885964.DDBK", "num": 20.0, "prePctChangeRate": -0.98, "numChange": 20.0, "rank": 153}, "885733.DDBK": {"blockCode": "885733.DDBK", "num": 10.0, "prePctChangeRate": -0.49, "numChange": 10.0, "rank": 267}}}, "000021.BKDL": {"categoryCode": "000021.BKDL", "categoryName": "**三胎", "num": 100.04, "prePctChangeRate": -0.91, "numChange": 90.03, "blocks": ["885540.DDBK", "885968.DDBK", "885909.DDBK"], "industry": 0, "rank": 26, "block_dict": {"885540.DDBK": {"blockCode": "885540.DDBK", "num": 80.04, "prePctChangeRate": -0.91, "numChange": 70.03, "rank": 24}, "885968.DDBK": {"blockCode": "885968.DDBK", "num": 20.02, "prePctChangeRate": -1.19, "numChange": 20.02, "rank": 113}, "885909.DDBK": {"blockCode": "885909.DDBK", "num": 20.0, "prePctChangeRate": -1.27, "numChange": 20.0, "rank": 154}}}, "000037.BKDL": {"categoryCode": "000037.BKDL", "categoryName": "**大市值股票", "num": 100.03, "prePctChangeRate": -0.94, "numChange": 39.99, "blocks": ["885520.DDBK", "885745.DDBK", "883303.DDBK", "883300.DDBK"], "industry": 0, "rank": 27, "block_dict": {"885520.DDBK": {"blockCode": "885520.DDBK", "num": 70.02, "prePctChangeRate": -0.92, "numChange": 9.98, "rank": 33}, "885745.DDBK": {"blockCode": "885745.DDBK", "num": 40.02, "prePctChangeRate": -0.85, "numChange": 30.02, "rank": 70}, "883303.DDBK": {"blockCode": "883303.DDBK", "num": 10.01, "prePctChangeRate": -0.74, "numChange": 0.01, "rank": 191}, "883300.DDBK": {"blockCode": "883300.DDBK", "num": 10.0, "prePctChangeRate": -0.63, "numChange": 10.0, "rank": 268}}}, "000028.BKDL": {"categoryCode": "000028.BKDL", "categoryName": "**ST板块", "num": 80.05, "prePctChangeRate": -1.6, "numChange": 50.02, "blocks": ["885699.DDBK"], "industry": 0, "rank": 28, "block_dict": {"885699.DDBK": {"blockCode": "885699.DDBK", "num": 80.05, "prePctChangeRate": -1.6, "numChange": 50.02, "rank": 22}}}, "000007.BKDL": {"categoryCode": "000007.BKDL", "categoryName": "**房产建筑", "num": 80.05, "prePctChangeRate": -0.96, "numChange": -0.02, "blocks": ["885478.DDBK", "885991.DDBK", "885760.DDBK", "885984.DDBK", "885386.DDBK", "885750.DDBK", "885915.DDBK"], "industry": 0, "rank": 29, "block_dict": {"885478.DDBK": {"blockCode": "885478.DDBK", "num": 30.02, "prePctChangeRate": -0.88, "numChange": 10.0, "rank": 88}, "885991.DDBK": {"blockCode": "885991.DDBK", "num": 30.02, "prePctChangeRate": -0.82, "numChange": -20.02, "rank": 89}, "885760.DDBK": {"blockCode": "885760.DDBK", "num": 20.01, "prePctChangeRate": -0.54, "numChange": -0.01, "rank": 125}, "885984.DDBK": {"blockCode": "885984.DDBK", "num": 20.01, "prePctChangeRate": -1.08, "numChange": 10.0, "rank": 126}, "885386.DDBK": {"blockCode": "885386.DDBK", "num": 10.01, "prePctChangeRate": -1.02, "numChange": 0.0, "rank": 192}, "885750.DDBK": {"blockCode": "885750.DDBK", "num": 10.01, "prePctChangeRate": -1.39, "numChange": 0.0, "rank": 193}, "885915.DDBK": {"blockCode": "885915.DDBK", "num": 10.01, "prePctChangeRate": -1.61, "numChange": -10.01, "rank": 194}}}, "000029.BKDL": {"categoryCode": "000029.BKDL", "categoryName": "电子类", "num": 80.03, "prePctChangeRate": -1.71, "numChange": 9.97, "blocks": ["885800.DDBK", "885875.DDBK", "885959.DDBK", "885809.DDBK", "885675.DDBK"], "industry": 0, "rank": 30, "block_dict": {"885800.DDBK": {"blockCode": "885800.DDBK", "num": 30.03, "prePctChangeRate": -1.71, "numChange": 0.01, "rank": 83}, "885875.DDBK": {"blockCode": "885875.DDBK", "num": 30.0, "prePctChangeRate": -1.72, "numChange": 30.0, "rank": 107}, "885959.DDBK": {"blockCode": "885959.DDBK", "num": 20.0, "prePctChangeRate": -1.56, "numChange": 20.0, "rank": 155}, "885809.DDBK": {"blockCode": "885809.DDBK", "num": 10.01, "prePctChangeRate": -1.49, "numChange": -10.0, "rank": 195}, "885675.DDBK": {"blockCode": "885675.DDBK", "num": 10.0, "prePctChangeRate": -2.03, "numChange": -30.04, "rank": 269}}}, "000016.BKDL": {"categoryCode": "000016.BKDL", "categoryName": "软件互联网", "num": 80.0, "prePctChangeRate": -1.72, "numChange": -40.1, "blocks": ["885378.DDBK", "885568.DDBK", "885459.DDBK", "885881.DDBK", "885730.DDBK", "885956.DDBK"], "industry": 0, "rank": 31, "block_dict": {"885378.DDBK": {"blockCode": "885378.DDBK", "num": 40.0, "prePctChangeRate": -1.69, "numChange": -10.05, "rank": 82}, "885568.DDBK": {"blockCode": "885568.DDBK", "num": 30.0, "prePctChangeRate": -1.75, "numChange": -0.02, "rank": 108}, "885459.DDBK": {"blockCode": "885459.DDBK", "num": 20.0, "prePctChangeRate": -1.76, "numChange": -10.03, "rank": 156}, "885881.DDBK": {"blockCode": "885881.DDBK", "num": 20.0, "prePctChangeRate": -2.01, "numChange": 9.99, "rank": 157}, "885730.DDBK": {"blockCode": "885730.DDBK", "num": 10.0, "prePctChangeRate": -1.34, "numChange": -30.03, "rank": 270}, "885956.DDBK": {"blockCode": "885956.DDBK", "num": 10.0, "prePctChangeRate": -2.07, "numChange": -20.03, "rank": 271}}}, "000036.BKDL": {"categoryCode": "000036.BKDL", "categoryName": "新材料", "num": 70.03, "prePctChangeRate": -1.25, "numChange": 30.02, "blocks": ["885544.DDBK", "886063.DDBK"], "industry": 0, "rank": 32, "block_dict": {"885544.DDBK": {"blockCode": "885544.DDBK", "num": 70.03, "prePctChangeRate": -1.22, "numChange": 30.02, "rank": 32}, "886063.DDBK": {"blockCode": "886063.DDBK", "num": 10.01, "prePctChangeRate": -2.12, "numChange": 10.01, "rank": 196}}}, "000032.BKDL": {"categoryCode": "000032.BKDL", "categoryName": "新股次新股", "num": 70.01, "prePctChangeRate": -1.22, "numChange": 9.98, "blocks": ["885598.DDBK", "885905.DDBK", "885906.DDBK"], "industry": 0, "rank": 33, "block_dict": {"885598.DDBK": {"blockCode": "885598.DDBK", "num": 70.01, "prePctChangeRate": -1.22, "numChange": 9.98, "rank": 34}, "885905.DDBK": {"blockCode": "885905.DDBK", "num": 50.0, "prePctChangeRate": -1.27, "numChange": -0.03, "rank": 61}, "885906.DDBK": {"blockCode": "885906.DDBK", "num": 10.0, "prePctChangeRate": -0.75, "numChange": 10.0, "rank": 272}}}, "T03.ZHBK": {"categoryCode": "T03.ZHBK", "categoryName": "基础化工", "num": 60.05, "prePctChangeRate": -1.09, "numChange": 30.03, "blocks": ["T03.ZHBK"], "industry": 1, "rank": 34, "block_dict": {"T03.ZHBK": {"blockCode": "T03.ZHBK", "num": 60.05, "prePctChangeRate": -1.09, "numChange": 30.03, "rank": 36}}}, "000022.BKDL": {"categoryCode": "000022.BKDL", "categoryName": "**化工", "num": 60.03, "prePctChangeRate": -1.21, "numChange": 50.02, "blocks": ["885901.DDBK", "885551.DDBK", "885912.DDBK", "885937.DDBK", "885882.DDBK"], "industry": 0, "rank": 35, "block_dict": {"885901.DDBK": {"blockCode": "885901.DDBK", "num": 20.0, "prePctChangeRate": -1.26, "numChange": 20.0, "rank": 158}, "885551.DDBK": {"blockCode": "885551.DDBK", "num": 10.01, "prePctChangeRate": -0.47, "numChange": 10.01, "rank": 197}, "885912.DDBK": {"blockCode": "885912.DDBK", "num": 10.01, "prePctChangeRate": -1.46, "numChange": 10.01, "rank": 198}, "885937.DDBK": {"blockCode": "885937.DDBK", "num": 10.01, "prePctChangeRate": -0.7, "numChange": 0.0, "rank": 199}, "885882.DDBK": {"blockCode": "885882.DDBK", "num": 10.0, "prePctChangeRate": -0.95, "numChange": 10.0, "rank": 273}}}, "T26.ZHBK": {"categoryCode": "T26.ZHBK", "categoryName": "电力设备", "num": 60.02, "prePctChangeRate": -1.12, "numChange": 30.01, "blocks": ["T26.ZHBK"], "industry": 1, "rank": 36, "block_dict": {"T26.ZHBK": {"blockCode": "T26.ZHBK", "num": 60.02, "prePctChangeRate": -1.12, "numChange": 30.01, "rank": 42}}}, "000008.BKDL": {"categoryCode": "000008.BKDL", "categoryName": "**传统行业", "num": 60.01, "prePctChangeRate": -1.21, "numChange": -10.05, "blocks": ["885423.DDBK", "885572.DDBK", "885692.DDBK", "885398.DDBK"], "industry": 0, "rank": 37, "block_dict": {"885423.DDBK": {"blockCode": "885423.DDBK", "num": 30.0, "prePctChangeRate": -1.55, "numChange": -10.03, "rank": 109}, "885572.DDBK": {"blockCode": "885572.DDBK", "num": 20.01, "prePctChangeRate": -1.16, "numChange": 10.0, "rank": 127}, "885692.DDBK": {"blockCode": "885692.DDBK", "num": 10.01, "prePctChangeRate": -0.9, "numChange": 0.0, "rank": 200}, "885398.DDBK": {"blockCode": "885398.DDBK", "num": 10.0, "prePctChangeRate": -0.94, "numChange": -0.01, "rank": 274}}}, "T28.ZHBK": {"categoryCode": "T28.ZHBK", "categoryName": "通信", "num": 60.01, "prePctChangeRate": -1.55, "numChange": 40.0, "blocks": ["T28.ZHBK"], "industry": 1, "rank": 38, "block_dict": {"T28.ZHBK": {"blockCode": "T28.ZHBK", "num": 60.01, "prePctChangeRate": -1.55, "numChange": 40.0, "rank": 45}}}, "T14.ZHBK": {"categoryCode": "T14.ZHBK", "categoryName": "轻工制造", "num": 50.03, "prePctChangeRate": -0.39, "numChange": 40.02, "blocks": ["T14.ZHBK"], "industry": 1, "rank": 39, "block_dict": {"T14.ZHBK": {"blockCode": "T14.ZHBK", "num": 50.03, "prePctChangeRate": -0.39, "numChange": 40.02, "rank": 51}}}, "T10.ZHBK": {"categoryCode": "T10.ZHBK", "categoryName": "计算机", "num": 50.0, "prePctChangeRate": -1.96, "numChange": 19.98, "blocks": ["T10.ZHBK"], "industry": 1, "rank": 40, "block_dict": {"T10.ZHBK": {"blockCode": "T10.ZHBK", "num": 50.0, "prePctChangeRate": -1.96, "numChange": 19.98, "rank": 62}}}, "T25.ZHBK": {"categoryCode": "T25.ZHBK", "categoryName": "建筑装饰", "num": 40.03, "prePctChangeRate": -0.58, "numChange": -20.03, "blocks": ["T25.ZHBK"], "industry": 1, "rank": 41, "block_dict": {"T25.ZHBK": {"blockCode": "T25.ZHBK", "num": 40.03, "prePctChangeRate": -0.58, "numChange": -20.03, "rank": 65}}}, "000024.BKDL": {"categoryCode": "000024.BKDL", "categoryName": "**农业", "num": 40.02, "prePctChangeRate": -0.94, "numChange": 30.02, "blocks": ["885706.DDBK", "885505.DDBK", "885810.DDBK", "885812.DDBK", "885863.DDBK", "885967.DDBK"], "industry": 0, "rank": 42, "block_dict": {"885706.DDBK": {"blockCode": "885706.DDBK", "num": 20.01, "prePctChangeRate": -1.02, "numChange": 10.01, "rank": 128}, "885505.DDBK": {"blockCode": "885505.DDBK", "num": 10.01, "prePctChangeRate": -0.64, "numChange": 0.01, "rank": 201}, "885810.DDBK": {"blockCode": "885810.DDBK", "num": 10.01, "prePctChangeRate": -1.39, "numChange": 10.01, "rank": 202}, "885812.DDBK": {"blockCode": "885812.DDBK", "num": 10.0, "prePctChangeRate": -0.96, "numChange": 10.0, "rank": 275}, "885863.DDBK": {"blockCode": "885863.DDBK", "num": 10.0, "prePctChangeRate": -1.17, "numChange": 10.0, "rank": 276}, "885967.DDBK": {"blockCode": "885967.DDBK", "num": 10.0, "prePctChangeRate": -1.06, "numChange": 10.0, "rank": 277}}}, "000001.BKDL": {"categoryCode": "000001.BKDL", "categoryName": "**有色贵金属", "num": 40.02, "prePctChangeRate": -0.82, "numChange": -10.01, "blocks": ["885343.DDBK", "885552.DDBK", "885970.DDBK", "885973.DDBK", "885355.DDBK", "885972.DDBK"], "industry": 0, "rank": 43, "block_dict": {"885343.DDBK": {"blockCode": "885343.DDBK", "num": 20.01, "prePctChangeRate": -0.97, "numChange": 0.0, "rank": 129}, "885552.DDBK": {"blockCode": "885552.DDBK", "num": 10.01, "prePctChangeRate": -0.58, "numChange": 0.0, "rank": 203}, "885970.DDBK": {"blockCode": "885970.DDBK", "num": 10.01, "prePctChangeRate": -0.47, "numChange": 0.0, "rank": 204}, "885973.DDBK": {"blockCode": "885973.DDBK", "num": 10.01, "prePctChangeRate": 0.02, "numChange": 0.0, "rank": 205}, "885355.DDBK": {"blockCode": "885355.DDBK", "num": 10.0, "prePctChangeRate": -0.9, "numChange": -10.01, "rank": 278}, "885972.DDBK": {"blockCode": "885972.DDBK", "num": 10.0, "prePctChangeRate": -1.0, "numChange": 10.0, "rank": 279}}}, "T08.ZHBK": {"categoryCode": "T08.ZHBK", "categoryName": "电子", "num": 40.01, "prePctChangeRate": -1.92, "numChange": 9.99, "blocks": ["T08.ZHBK"], "industry": 1, "rank": 44, "block_dict": {"T08.ZHBK": {"blockCode": "T08.ZHBK", "num": 40.01, "prePctChangeRate": -1.92, "numChange": 9.99, "rank": 78}}}, "T24.ZHBK": {"categoryCode": "T24.ZHBK", "categoryName": "国防军工", "num": 40.01, "prePctChangeRate": -1.12, "numChange": -10.04, "blocks": ["T24.ZHBK"], "industry": 1, "rank": 45, "block_dict": {"T24.ZHBK": {"blockCode": "T24.ZHBK", "num": 40.01, "prePctChangeRate": -1.12, "numChange": -10.04, "rank": 79}}}, "T15.ZHBK": {"categoryCode": "T15.ZHBK", "categoryName": "医药生物", "num": 30.0, "prePctChangeRate": -0.67, "numChange": -0.03, "blocks": ["T15.ZHBK"], "industry": 1, "rank": 46, "block_dict": {"T15.ZHBK": {"blockCode": "T15.ZHBK", "num": 30.0, "prePctChangeRate": -0.67, "numChange": -0.03, "rank": 110}}}, "T16.ZHBK": {"categoryCode": "T16.ZHBK", "categoryName": "公用事业", "num": 20.01, "prePctChangeRate": 0.46, "numChange": 0.0, "blocks": ["T16.ZHBK"], "industry": 1, "rank": 47, "block_dict": {"T16.ZHBK": {"blockCode": "T16.ZHBK", "num": 20.01, "prePctChangeRate": 0.46, "numChange": 0.0, "rank": 130}}}, "T12.ZHBK": {"categoryCode": "T12.ZHBK", "categoryName": "食品饮料", "num": 20.01, "prePctChangeRate": 0.16, "numChange": 20.01, "blocks": ["T12.ZHBK"], "industry": 1, "rank": 48, "block_dict": {"T12.ZHBK": {"blockCode": "T12.ZHBK", "num": 20.01, "prePctChangeRate": 0.16, "numChange": 20.01, "rank": 131}}}, "T05.ZHBK": {"categoryCode": "T05.ZHBK", "categoryName": "有色金属", "num": 20.01, "prePctChangeRate": -0.8, "numChange": 20.01, "blocks": ["T05.ZHBK"], "industry": 1, "rank": 49, "block_dict": {"T05.ZHBK": {"blockCode": "T05.ZHBK", "num": 20.01, "prePctChangeRate": -0.8, "numChange": 20.01, "rank": 132}}}, "T11.ZHBK": {"categoryCode": "T11.ZHBK", "categoryName": "家用电器", "num": 20.0, "prePctChangeRate": -0.46, "numChange": 9.99, "blocks": ["T11.ZHBK"], "industry": 1, "rank": 50, "block_dict": {"T11.ZHBK": {"blockCode": "T11.ZHBK", "num": 20.0, "prePctChangeRate": -0.46, "numChange": 9.99, "rank": 159}}}, "T20.ZHBK": {"categoryCode": "T20.ZHBK", "categoryName": "商贸零售", "num": 10.01, "prePctChangeRate": -1.79, "numChange": 10.01, "blocks": ["T20.ZHBK"], "industry": 1, "rank": 51, "block_dict": {"T20.ZHBK": {"blockCode": "T20.ZHBK", "num": 10.01, "prePctChangeRate": -1.79, "numChange": 10.01, "rank": 206}}}, "T18.ZHBK": {"categoryCode": "T18.ZHBK", "categoryName": "房地产", "num": 10.01, "prePctChangeRate": -1.27, "numChange": 0.0, "blocks": ["T18.ZHBK"], "industry": 1, "rank": 52, "block_dict": {"T18.ZHBK": {"blockCode": "T18.ZHBK", "num": 10.01, "prePctChangeRate": -1.27, "numChange": 0.0, "rank": 207}}}, "T30.ZHBK": {"categoryCode": "T30.ZHBK", "categoryName": "环保", "num": 10.01, "prePctChangeRate": -0.03, "numChange": 0.0, "blocks": ["T30.ZHBK"], "industry": 1, "rank": 53, "block_dict": {"T30.ZHBK": {"blockCode": "T30.ZHBK", "num": 10.01, "prePctChangeRate": -0.03, "numChange": 0.0, "rank": 208}}}, "T23.ZHBK": {"categoryCode": "T23.ZHBK", "categoryName": "综合", "num": 10.0, "prePctChangeRate": -1.95, "numChange": 10.0, "blocks": ["T23.ZHBK"], "industry": 1, "rank": 54, "block_dict": {"T23.ZHBK": {"blockCode": "T23.ZHBK", "num": 10.0, "prePctChangeRate": -1.95, "numChange": 10.0, "rank": 280}}}, "000005.BKDL": {"categoryCode": "000005.BKDL", "categoryName": "**油气", "num": 10.0, "prePctChangeRate": -0.81, "numChange": 10.0, "blocks": ["885372.DDBK", "885430.DDBK", "885748.DDBK"], "industry": 0, "rank": 55, "block_dict": {"885372.DDBK": {"blockCode": "885372.DDBK", "num": 10.0, "prePctChangeRate": -1.45, "numChange": 10.0, "rank": 281}, "885430.DDBK": {"blockCode": "885430.DDBK", "num": 10.0, "prePctChangeRate": -0.79, "numChange": 10.0, "rank": 282}, "885748.DDBK": {"blockCode": "885748.DDBK", "num": 10.0, "prePctChangeRate": -1.28, "numChange": 10.0, "rank": 283}}}, "T22.ZHBK": {"categoryCode": "T22.ZHBK", "categoryName": "传媒", "num": 10.0, "prePctChangeRate": -1.82, "numChange": 10.0, "blocks": ["T22.ZHBK"], "industry": 1, "rank": 56, "block_dict": {"T22.ZHBK": {"blockCode": "T22.ZHBK", "num": 10.0, "prePctChangeRate": -1.82, "numChange": 10.0, "rank": 284}}}, "T17.ZHBK": {"categoryCode": "T17.ZHBK", "categoryName": "交通运输", "num": 10.0, "prePctChangeRate": -0.42, "numChange": -0.01, "blocks": ["T17.ZHBK"], "industry": 1, "rank": 57, "block_dict": {"T17.ZHBK": {"blockCode": "T17.ZHBK", "num": 10.0, "prePctChangeRate": -0.42, "numChange": -0.01, "rank": 285}}}}

In [30]:
abs1 = []
for c,d in t.items():
    blocks = d['blocks']
    abs1.extend(blocks)

In [31]:
len(abs)

233

In [32]:
len(abs1)

285

In [33]:
for x in abs:
    if x not in abs1:
        print(x)

886079.DDBK
886097.DDBK
885867.DDBK
886089.DDBK
885787.DDBK
885932.DDBK
885740.DDBK
883437.DDBK
885338.DDBK
885816.DDBK
886098.DDBK
885791.DDBK
886060.DDBK
885918.DDBK
885647.DDBK
886078.DDBK
885945.DDBK
886035.DDBK
885899.DDBK
883436.DDBK
885963.DDBK
886086.DDBK
885827.DDBK
885345.DDBK
886099.DDBK
886100.DDBK
886074.DDBK
886070.DDBK
886071.DDBK
886068.DDBK
885767.DDBK
885502.DDBK
886084.DDBK
886023.DDBK
885975.DDBK
885845.DDBK
885852.DDBK
885958.DDBK
885919.DDBK
885555.DDBK
885433.DDBK
885428.DDBK
885844.DDBK
886009.DDBK
885738.DDBK
885774.DDBK
885811.DDBK
885995.DDBK
T01.ZHBK
T29.ZHBK


In [51]:
tt = {"000038.BKDL": {"categoryCode": "000038.BKDL", "categoryName": "**中市值股票", "num": 600.23, "prePctChangeRate": 1.46, "numChange": 80.02, "blocks": ["885694.DDBK", "883304.DDBK"], "industry": 0, "rank": 1, "block_dict": {"885694.DDBK": {"blockCode": "885694.DDBK", "num": 170.06, "prePctChangeRate": 1.45, "numChange": -30.01, "rank": 5}, "883304.DDBK": {"blockCode": "883304.DDBK", "num": 50.02, "prePctChangeRate": 1.16, "numChange": 10.0, "rank": 53}}}, "000033.BKDL": {"categoryCode": "000033.BKDL", "categoryName": "智造机器人", "num": 490.17, "prePctChangeRate": 2.07, "numChange": 80.03, "blocks": ["885517.DDBK", "885929.DDBK", "885941.DDBK", "885619.DDBK", "885427.DDBK", "886057.DDBK", "885786.DDBK", "885946.DDBK", "886002.DDBK", "886008.DDBK", "886069.DDBK", "885930.DDBK", "885537.DDBK"], "industry": 0, "rank": 2, "block_dict": {"885517.DDBK": {"blockCode": "885517.DDBK", "num": 240.1, "prePctChangeRate": 2.39, "numChange": 60.05, "rank": 1}, "885929.DDBK": {"blockCode": "885929.DDBK", "num": 240.08, "prePctChangeRate": 2.11, "numChange": 50.02, "rank": 3}, "885941.DDBK": {"blockCode": "885941.DDBK", "num": 120.03, "prePctChangeRate": 2.14, "numChange": -20.0, "rank": 14}, "885619.DDBK": {"blockCode": "885619.DDBK", "num": 60.01, "prePctChangeRate": 2.24, "numChange": -20.02, "rank": 46}, "885427.DDBK": {"blockCode": "885427.DDBK", "num": 40.02, "prePctChangeRate": 2.5, "numChange": -0.01, "rank": 61}, "886057.DDBK": {"blockCode": "886057.DDBK", "num": 40.0, "prePctChangeRate": 2.75, "numChange": -0.01, "rank": 80}, "885786.DDBK": {"blockCode": "885786.DDBK", "num": 30.02, "prePctChangeRate": 2.45, "numChange": 20.01, "rank": 82}, "885946.DDBK": {"blockCode": "885946.DDBK", "num": 30.01, "prePctChangeRate": 2.33, "numChange": 20.0, "rank": 96}, "886002.DDBK": {"blockCode": "886002.DDBK", "num": 30.01, "prePctChangeRate": 2.6, "numChange": 0.0, "rank": 97}, "886008.DDBK": {"blockCode": "886008.DDBK", "num": 30.01, "prePctChangeRate": 3.24, "numChange": -10.02, "rank": 98}, "886069.DDBK": {"blockCode": "886069.DDBK", "num": 20.01, "prePctChangeRate": 3.32, "numChange": 10.01, "rank": 126}, "885930.DDBK": {"blockCode": "885930.DDBK", "num": 10.01, "prePctChangeRate": 2.77, "numChange": -10.01, "rank": 180}, "885537.DDBK": {"blockCode": "885537.DDBK", "num": 10.0, "prePctChangeRate": 2.34, "numChange": 0.0, "rank": 215}}}, "000027.BKDL": {"categoryCode": "000027.BKDL", "categoryName": "汽车低空", "num": 430.18, "prePctChangeRate": 2.15, "numChange": 70.08, "blocks": ["885431.DDBK", "885997.DDBK", "885564.DDBK", "885545.DDBK", "886061.DDBK", "885461.DDBK", "885467.DDBK", "885662.DDBK", "885736.DDBK", "885834.DDBK", "886035.DDBK", "885766.DDBK", "886067.DDBK", "885775.DDBK", "886064.DDBK", "885871.DDBK", "885974.DDBK", "886066.DDBK", "885861.DDBK", "885886.DDBK", "885902.DDBK", "885999.DDBK", "886001.DDBK"], "industry": 0, "rank": 3, "block_dict": {"885431.DDBK": {"blockCode": "885431.DDBK", "num": 160.07, "prePctChangeRate": 2.08, "numChange": -19.98, "rank": 6}, "885997.DDBK": {"blockCode": "885997.DDBK", "num": 150.06, "prePctChangeRate": 2.33, "numChange": -0.01, "rank": 7}, "885564.DDBK": {"blockCode": "885564.DDBK", "num": 100.04, "prePctChangeRate": 2.97, "numChange": 40.02, "rank": 18}, "885545.DDBK": {"blockCode": "885545.DDBK", "num": 90.03, "prePctChangeRate": 2.08, "numChange": 60.02, "rank": 23}, "886061.DDBK": {"blockCode": "886061.DDBK", "num": 70.03, "prePctChangeRate": 2.27, "numChange": 20.02, "rank": 30}, "885461.DDBK": {"blockCode": "885461.DDBK", "num": 40.02, "prePctChangeRate": 1.61, "numChange": -19.99, "rank": 62}, "885467.DDBK": {"blockCode": "885467.DDBK", "num": 40.02, "prePctChangeRate": 2.29, "numChange": -20.01, "rank": 63}, "885662.DDBK": {"blockCode": "885662.DDBK", "num": 40.02, "prePctChangeRate": 2.13, "numChange": 0.01, "rank": 64}, "885736.DDBK": {"blockCode": "885736.DDBK", "num": 40.01, "prePctChangeRate": 1.99, "numChange": 30.01, "rank": 75}, "885834.DDBK": {"blockCode": "885834.DDBK", "num": 30.02, "prePctChangeRate": 2.24, "numChange": -20.0, "rank": 83}, "886035.DDBK": {"blockCode": "886035.DDBK", "num": 30.01, "prePctChangeRate": 2.3, "numChange": 30.01, "rank": 99}, "885766.DDBK": {"blockCode": "885766.DDBK", "num": 30.0, "prePctChangeRate": 1.91, "numChange": 10.0, "rank": 112}, "886067.DDBK": {"blockCode": "886067.DDBK", "num": 20.02, "prePctChangeRate": 2.61, "numChange": -40.0, "rank": 118}, "885775.DDBK": {"blockCode": "885775.DDBK", "num": 20.01, "prePctChangeRate": 1.54, "numChange": 10.0, "rank": 127}, "886064.DDBK": {"blockCode": "886064.DDBK", "num": 20.01, "prePctChangeRate": 2.14, "numChange": -10.01, "rank": 128}, "885871.DDBK": {"blockCode": "885871.DDBK", "num": 10.01, "prePctChangeRate": 1.89, "numChange": 0.0, "rank": 181}, "885974.DDBK": {"blockCode": "885974.DDBK", "num": 10.01, "prePctChangeRate": 2.59, "numChange": 10.01, "rank": 182}, "886066.DDBK": {"blockCode": "886066.DDBK", "num": 10.01, "prePctChangeRate": 3.05, "numChange": -20.0, "rank": 183}, "885861.DDBK": {"blockCode": "885861.DDBK", "num": 10.0, "prePctChangeRate": 2.2, "numChange": 10.0, "rank": 216}, "885886.DDBK": {"blockCode": "885886.DDBK", "num": 10.0, "prePctChangeRate": 2.36, "numChange": 10.0, "rank": 217}, "885902.DDBK": {"blockCode": "885902.DDBK", "num": 10.0, "prePctChangeRate": 1.55, "numChange": 10.0, "rank": 218}, "885999.DDBK": {"blockCode": "885999.DDBK", "num": 10.0, "prePctChangeRate": 2.08, "numChange": -10.01, "rank": 219}, "886001.DDBK": {"blockCode": "886001.DDBK", "num": 10.0, "prePctChangeRate": 2.33, "numChange": 0.0, "rank": 220}}}, "000023.BKDL": {"categoryCode": "000023.BKDL", "categoryName": "业绩股权类", "num": 430.17, "prePctChangeRate": 1.13, "numChange": 40.04, "blocks": ["885869.DDBK", "885802.DDBK", "886072.DDBK", "885796.DDBK", "885587.DDBK", "885729.DDBK", "885742.DDBK", "885560.DDBK"], "industry": 0, "rank": 4, "block_dict": {"885869.DDBK": {"blockCode": "885869.DDBK", "num": 80.02, "prePctChangeRate": 1.34, "numChange": -9.99, "rank": 26}, "885802.DDBK": {"blockCode": "885802.DDBK", "num": 60.03, "prePctChangeRate": 1.08, "numChange": 10.0, "rank": 39}, "886072.DDBK": {"blockCode": "886072.DDBK", "num": 40.02, "prePctChangeRate": 0.63, "numChange": 10.01, "rank": 65}, "885796.DDBK": {"blockCode": "885796.DDBK", "num": 30.01, "prePctChangeRate": 2.0, "numChange": 20.01, "rank": 100}, "885587.DDBK": {"blockCode": "885587.DDBK", "num": 20.01, "prePctChangeRate": 1.82, "numChange": 20.01, "rank": 129}, "885729.DDBK": {"blockCode": "885729.DDBK", "num": 20.01, "prePctChangeRate": 1.09, "numChange": -29.99, "rank": 130}, "885742.DDBK": {"blockCode": "885742.DDBK", "num": 10.01, "prePctChangeRate": 1.02, "numChange": -10.0, "rank": 184}, "885560.DDBK": {"blockCode": "885560.DDBK", "num": 10.0, "prePctChangeRate": 0.57, "numChange": 10.0, "rank": 221}}}, "000004.BKDL": {"categoryCode": "000004.BKDL", "categoryName": "**区域经济", "num": 300.11, "prePctChangeRate": 1.1, "numChange": -0.01, "blocks": ["885924.DDBK", "885832.DDBK", "885521.DDBK", "885617.DDBK", "885939.DDBK", "885744.DDBK", "885894.DDBK"], "industry": 0, "rank": 5, "block_dict": {"885924.DDBK": {"blockCode": "885924.DDBK", "num": 40.01, "prePctChangeRate": 1.37, "numChange": -30.03, "rank": 76}, "885832.DDBK": {"blockCode": "885832.DDBK", "num": 30.02, "prePctChangeRate": 1.03, "numChange": -20.01, "rank": 84}, "885521.DDBK": {"blockCode": "885521.DDBK", "num": 30.0, "prePctChangeRate": 0.98, "numChange": -10.01, "rank": 113}, "885617.DDBK": {"blockCode": "885617.DDBK", "num": 30.0, "prePctChangeRate": 1.21, "numChange": 19.99, "rank": 114}, "885939.DDBK": {"blockCode": "885939.DDBK", "num": 20.01, "prePctChangeRate": 1.37, "numChange": 10.0, "rank": 131}, "885744.DDBK": {"blockCode": "885744.DDBK", "num": 20.0, "prePctChangeRate": 0.81, "numChange": 0.0, "rank": 158}, "885894.DDBK": {"blockCode": "885894.DDBK", "num": 10.0, "prePctChangeRate": 0.37, "numChange": 0.0, "rank": 222}}}, "000026.BKDL": {"categoryCode": "000026.BKDL", "categoryName": "**外贸出口", "num": 290.11, "prePctChangeRate": 1.69, "numChange": 20.01, "blocks": ["885840.DDBK", "885642.DDBK", "885965.DDBK", "885611.DDBK"], "industry": 0, "rank": 6, "block_dict": {"885840.DDBK": {"blockCode": "885840.DDBK", "num": 240.09, "prePctChangeRate": 1.99, "numChange": 40.01, "rank": 2}, "885642.DDBK": {"blockCode": "885642.DDBK", "num": 100.02, "prePctChangeRate": 1.83, "numChange": -20.04, "rank": 20}, "885965.DDBK": {"blockCode": "885965.DDBK", "num": 30.02, "prePctChangeRate": 1.21, "numChange": 0.01, "rank": 85}, "885611.DDBK": {"blockCode": "885611.DDBK", "num": 20.0, "prePctChangeRate": 1.61, "numChange": 10.0, "rank": 159}}}, "000018.BKDL": {"categoryCode": "000018.BKDL", "categoryName": "**一带一路", "num": 230.08, "prePctChangeRate": 1.57, "numChange": 120.06, "blocks": ["885494.DDBK", "885770.DDBK", "885500.DDBK", "885733.DDBK", "885562.DDBK", "885964.DDBK"], "industry": 0, "rank": 7, "block_dict": {"885494.DDBK": {"blockCode": "885494.DDBK", "num": 120.03, "prePctChangeRate": 1.29, "numChange": 40.01, "rank": 15}, "885770.DDBK": {"blockCode": "885770.DDBK", "num": 50.03, "prePctChangeRate": 1.98, "numChange": 20.03, "rank": 50}, "885500.DDBK": {"blockCode": "885500.DDBK", "num": 30.01, "prePctChangeRate": 3.12, "numChange": 30.01, "rank": 101}, "885733.DDBK": {"blockCode": "885733.DDBK", "num": 20.02, "prePctChangeRate": 1.74, "numChange": 10.02, "rank": 119}, "885562.DDBK": {"blockCode": "885562.DDBK", "num": 20.0, "prePctChangeRate": 2.56, "numChange": 20.0, "rank": 160}, "885964.DDBK": {"blockCode": "885964.DDBK", "num": 10.0, "prePctChangeRate": 0.51, "numChange": -10.0, "rank": 223}}}, "000006.BKDL": {"categoryCode": "000006.BKDL", "categoryName": "华为大科技", "num": 220.1, "prePctChangeRate": 1.86, "numChange": 50.05, "blocks": ["885806.DDBK", "885376.DDBK", "885923.DDBK", "885980.DDBK", "885982.DDBK", "886059.DDBK", "885843.DDBK", "886058.DDBK"], "industry": 0, "rank": 8, "block_dict": {"885806.DDBK": {"blockCode": "885806.DDBK", "num": 140.06, "prePctChangeRate": 1.97, "numChange": 0.02, "rank": 8}, "885376.DDBK": {"blockCode": "885376.DDBK", "num": 70.04, "prePctChangeRate": 2.92, "numChange": 60.03, "rank": 29}, "885923.DDBK": {"blockCode": "885923.DDBK", "num": 40.01, "prePctChangeRate": 1.68, "numChange": 10.01, "rank": 77}, "885980.DDBK": {"blockCode": "885980.DDBK", "num": 30.02, "prePctChangeRate": 2.55, "numChange": 20.02, "rank": 86}, "885982.DDBK": {"blockCode": "885982.DDBK", "num": 20.01, "prePctChangeRate": 1.84, "numChange": 10.01, "rank": 132}, "886059.DDBK": {"blockCode": "886059.DDBK", "num": 20.0, "prePctChangeRate": 2.36, "numChange": 9.99, "rank": 161}, "885843.DDBK": {"blockCode": "885843.DDBK", "num": 10.01, "prePctChangeRate": 2.34, "numChange": 10.01, "rank": 185}, "886058.DDBK": {"blockCode": "886058.DDBK", "num": 10.01, "prePctChangeRate": 2.14, "numChange": 10.01, "rank": 186}}}, "000013.BKDL": {"categoryCode": "000013.BKDL", "categoryName": "**国防军工", "num": 220.08, "prePctChangeRate": 2.83, "numChange": 70.03, "blocks": ["885700.DDBK", "885566.DDBK", "885743.DDBK", "886030.DDBK", "885884.DDBK", "886076.DDBK", "885860.DDBK", "885426.DDBK"], "industry": 0, "rank": 9, "block_dict": {"885700.DDBK": {"blockCode": "885700.DDBK", "num": 170.07, "prePctChangeRate": 3.02, "numChange": 20.02, "rank": 4}, "885566.DDBK": {"blockCode": "885566.DDBK", "num": 70.03, "prePctChangeRate": 3.02, "numChange": 50.02, "rank": 31}, "885743.DDBK": {"blockCode": "885743.DDBK", "num": 70.01, "prePctChangeRate": 3.5, "numChange": 20.0, "rank": 38}, "886030.DDBK": {"blockCode": "886030.DDBK", "num": 40.02, "prePctChangeRate": 5.06, "numChange": 20.01, "rank": 66}, "885884.DDBK": {"blockCode": "885884.DDBK", "num": 30.01, "prePctChangeRate": 3.52, "numChange": 30.01, "rank": 102}, "886076.DDBK": {"blockCode": "886076.DDBK", "num": 30.01, "prePctChangeRate": 4.26, "numChange": 0.01, "rank": 103}, "885860.DDBK": {"blockCode": "885860.DDBK", "num": 10.01, "prePctChangeRate": 6.55, "numChange": 0.01, "rank": 187}, "885426.DDBK": {"blockCode": "885426.DDBK", "num": 10.0, "prePctChangeRate": 2.68, "numChange": 10.0, "rank": 224}}}, "000012.BKDL": {"categoryCode": "000012.BKDL", "categoryName": "传媒游戏娱乐", "num": 210.1, "prePctChangeRate": 1.27, "numChange": 40.03, "blocks": ["885709.DDBK", "885615.DDBK", "885797.DDBK", "885497.DDBK", "886046.DDBK", "885586.DDBK", "885788.DDBK", "885934.DDBK", "885950.DDBK", "885480.DDBK", "885772.DDBK", "886014.DDBK", "885792.DDBK", "885876.DDBK", "885418.DDBK", "885647.DDBK", "885737.DDBK", "885779.DDBK", "885791.DDBK", "885816.DDBK", "885890.DDBK"], "industry": 0, "rank": 10, "block_dict": {"885709.DDBK": {"blockCode": "885709.DDBK", "num": 40.03, "prePctChangeRate": 1.95, "numChange": 20.03, "rank": 58}, "885615.DDBK": {"blockCode": "885615.DDBK", "num": 40.01, "prePctChangeRate": 0.67, "numChange": 0.0, "rank": 78}, "885797.DDBK": {"blockCode": "885797.DDBK", "num": 30.01, "prePctChangeRate": 1.64, "numChange": 10.01, "rank": 104}, "885497.DDBK": {"blockCode": "885497.DDBK", "num": 20.02, "prePctChangeRate": 0.96, "numChange": 0.01, "rank": 120}, "886046.DDBK": {"blockCode": "886046.DDBK", "num": 20.02, "prePctChangeRate": 2.38, "numChange": 10.01, "rank": 121}, "885586.DDBK": {"blockCode": "885586.DDBK", "num": 20.01, "prePctChangeRate": 0.92, "numChange": 10.01, "rank": 133}, "885788.DDBK": {"blockCode": "885788.DDBK", "num": 20.01, "prePctChangeRate": 0.6, "numChange": -40.02, "rank": 134}, "885934.DDBK": {"blockCode": "885934.DDBK", "num": 20.01, "prePctChangeRate": 1.86, "numChange": 10.01, "rank": 135}, "885950.DDBK": {"blockCode": "885950.DDBK", "num": 20.01, "prePctChangeRate": 1.38, "numChange": 20.01, "rank": 136}, "885480.DDBK": {"blockCode": "885480.DDBK", "num": 20.0, "prePctChangeRate": 0.76, "numChange": 0.0, "rank": 162}, "885772.DDBK": {"blockCode": "885772.DDBK", "num": 20.0, "prePctChangeRate": 2.41, "numChange": 0.0, "rank": 163}, "886014.DDBK": {"blockCode": "886014.DDBK", "num": 20.0, "prePctChangeRate": 0.84, "numChange": 9.99, "rank": 164}, "885792.DDBK": {"blockCode": "885792.DDBK", "num": 10.01, "prePctChangeRate": 1.08, "numChange": 0.0, "rank": 188}, "885876.DDBK": {"blockCode": "885876.DDBK", "num": 10.01, "prePctChangeRate": 0.59, "numChange": -10.01, "rank": 189}, "885418.DDBK": {"blockCode": "885418.DDBK", "num": 10.0, "prePctChangeRate": 0.72, "numChange": -10.01, "rank": 225}, "885647.DDBK": {"blockCode": "885647.DDBK", "num": 10.0, "prePctChangeRate": 0.7, "numChange": 10.0, "rank": 226}, "885737.DDBK": {"blockCode": "885737.DDBK", "num": 10.0, "prePctChangeRate": 0.86, "numChange": 10.0, "rank": 227}, "885779.DDBK": {"blockCode": "885779.DDBK", "num": 10.0, "prePctChangeRate": 1.39, "numChange": -10.0, "rank": 228}, "885791.DDBK": {"blockCode": "885791.DDBK", "num": 10.0, "prePctChangeRate": 0.97, "numChange": 10.0, "rank": 229}, "885816.DDBK": {"blockCode": "885816.DDBK", "num": 10.0, "prePctChangeRate": 1.4, "numChange": 10.0, "rank": 230}, "885890.DDBK": {"blockCode": "885890.DDBK", "num": 10.0, "prePctChangeRate": 1.06, "numChange": -20.0, "rank": 231}}}, "000025.BKDL": {"categoryCode": "000025.BKDL", "categoryName": "通信类", "num": 210.08, "prePctChangeRate": 2.16, "numChange": -10.0, "blocks": ["885312.DDBK", "885868.DDBK", "885783.DDBK", "885556.DDBK", "885574.DDBK", "885940.DDBK", "885998.DDBK"], "industry": 0, "rank": 11, "block_dict": {"885312.DDBK": {"blockCode": "885312.DDBK", "num": 80.03, "prePctChangeRate": 1.79, "numChange": 0.0, "rank": 24}, "885868.DDBK": {"blockCode": "885868.DDBK", "num": 60.02, "prePctChangeRate": 2.72, "numChange": 50.01, "rank": 43}, "885783.DDBK": {"blockCode": "885783.DDBK", "num": 50.02, "prePctChangeRate": 2.09, "numChange": -20.01, "rank": 54}, "885556.DDBK": {"blockCode": "885556.DDBK", "num": 40.02, "prePctChangeRate": 2.16, "numChange": -90.02, "rank": 67}, "885574.DDBK": {"blockCode": "885574.DDBK", "num": 30.01, "prePctChangeRate": 2.95, "numChange": 10.01, "rank": 105}, "885940.DDBK": {"blockCode": "885940.DDBK", "num": 10.01, "prePctChangeRate": 2.67, "numChange": -10.0, "rank": 190}, "885998.DDBK": {"blockCode": "885998.DDBK", "num": 10.01, "prePctChangeRate": 1.75, "numChange": -20.0, "rank": 191}}}, "000009.BKDL": {"categoryCode": "000009.BKDL", "categoryName": "**医药医疗", "num": 210.04, "prePctChangeRate": 0.76, "numChange": -20.07, "blocks": ["885539.DDBK", "885402.DDBK", "885765.DDBK", "885983.DDBK", "885403.DDBK", "885703.DDBK", "885856.DDBK", "885880.DDBK", "885954.DDBK", "886015.DDBK", "886028.DDBK", "886077.DDBK", "885818.DDBK", "885879.DDBK", "885900.DDBK", "885913.DDBK", "885917.DDBK", "885927.DDBK", "885955.DDBK", "885993.DDBK", "885994.DDBK", "886024.DDBK", "886025.DDBK", "886027.DDBK", "886029.DDBK", "886051.DDBK"], "industry": 0, "rank": 12, "block_dict": {"885539.DDBK": {"blockCode": "885539.DDBK", "num": 80.02, "prePctChangeRate": 1.26, "numChange": 29.99, "rank": 27}, "885402.DDBK": {"blockCode": "885402.DDBK", "num": 30.01, "prePctChangeRate": 1.57, "numChange": 20.01, "rank": 106}, "885765.DDBK": {"blockCode": "885765.DDBK", "num": 30.01, "prePctChangeRate": 0.99, "numChange": 10.01, "rank": 107}, "885983.DDBK": {"blockCode": "885983.DDBK", "num": 20.01, "prePctChangeRate": 1.94, "numChange": 10.01, "rank": 137}, "885403.DDBK": {"blockCode": "885403.DDBK", "num": 20.0, "prePctChangeRate": -0.11, "numChange": 10.0, "rank": 165}, "885703.DDBK": {"blockCode": "885703.DDBK", "num": 20.0, "prePctChangeRate": 0.72, "numChange": 10.0, "rank": 166}, "885856.DDBK": {"blockCode": "885856.DDBK", "num": 20.0, "prePctChangeRate": -0.12, "numChange": 10.0, "rank": 167}, "885880.DDBK": {"blockCode": "885880.DDBK", "num": 20.0, "prePctChangeRate": 1.14, "numChange": -60.05, "rank": 168}, "885954.DDBK": {"blockCode": "885954.DDBK", "num": 20.0, "prePctChangeRate": -0.52, "numChange": 10.0, "rank": 169}, "886015.DDBK": {"blockCode": "886015.DDBK", "num": 20.0, "prePctChangeRate": -0.79, "numChange": 10.0, "rank": 170}, "886028.DDBK": {"blockCode": "886028.DDBK", "num": 20.0, "prePctChangeRate": 1.64, "numChange": 20.0, "rank": 171}, "886077.DDBK": {"blockCode": "886077.DDBK", "num": 20.0, "prePctChangeRate": 0.11, "numChange": 0.0, "rank": 172}, "885818.DDBK": {"blockCode": "885818.DDBK", "num": 10.0, "prePctChangeRate": 0.48, "numChange": -20.02, "rank": 232}, "885879.DDBK": {"blockCode": "885879.DDBK", "num": 10.0, "prePctChangeRate": 0.2, "numChange": 0.0, "rank": 233}, "885900.DDBK": {"blockCode": "885900.DDBK", "num": 10.0, "prePctChangeRate": 0.08, "numChange": 0.0, "rank": 234}, "885913.DDBK": {"blockCode": "885913.DDBK", "num": 10.0, "prePctChangeRate": 0.41, "numChange": -10.0, "rank": 235}, "885917.DDBK": {"blockCode": "885917.DDBK", "num": 10.0, "prePctChangeRate": 0.89, "numChange": 10.0, "rank": 236}, "885927.DDBK": {"blockCode": "885927.DDBK", "num": 10.0, "prePctChangeRate": 0.5, "numChange": 0.0, "rank": 237}, "885955.DDBK": {"blockCode": "885955.DDBK", "num": 10.0, "prePctChangeRate": -0.75, "numChange": 0.0, "rank": 238}, "885993.DDBK": {"blockCode": "885993.DDBK", "num": 10.0, "prePctChangeRate": 1.24, "numChange": -0.01, "rank": 239}, "885994.DDBK": {"blockCode": "885994.DDBK", "num": 10.0, "prePctChangeRate": 0.58, "numChange": 0.0, "rank": 240}, "886024.DDBK": {"blockCode": "886024.DDBK", "num": 10.0, "prePctChangeRate": 1.06, "numChange": 0.0, "rank": 241}, "886025.DDBK": {"blockCode": "886025.DDBK", "num": 10.0, "prePctChangeRate": -0.71, "numChange": 10.0, "rank": 242}, "886027.DDBK": {"blockCode": "886027.DDBK", "num": 10.0, "prePctChangeRate": -0.38, "numChange": 10.0, "rank": 243}, "886029.DDBK": {"blockCode": "886029.DDBK", "num": 10.0, "prePctChangeRate": 0.21, "numChange": 10.0, "rank": 244}, "886051.DDBK": {"blockCode": "886051.DDBK", "num": 10.0, "prePctChangeRate": -0.21, "numChange": 10.0, "rank": 245}}}, "000034.BKDL": {"categoryCode": "000034.BKDL", "categoryName": "人工智能", "num": 200.07, "prePctChangeRate": 1.86, "numChange": 30.04, "blocks": ["885728.DDBK", "886031.DDBK", "886062.DDBK", "886019.DDBK"], "industry": 0, "rank": 13, "block_dict": {"885728.DDBK": {"blockCode": "885728.DDBK", "num": 130.05, "prePctChangeRate": 1.91, "numChange": 0.03, "rank": 10}, "886031.DDBK": {"blockCode": "886031.DDBK", "num": 40.02, "prePctChangeRate": 1.66, "numChange": 30.02, "rank": 68}, "886062.DDBK": {"blockCode": "886062.DDBK", "num": 20.02, "prePctChangeRate": 2.0, "numChange": 10.02, "rank": 122}, "886019.DDBK": {"blockCode": "886019.DDBK", "num": 10.0, "prePctChangeRate": 1.58, "numChange": -0.01, "rank": 246}}}, "000010.BKDL": {"categoryCode": "000010.BKDL", "categoryName": "**大消费类", "num": 190.08, "prePctChangeRate": 0.88, "numChange": -80.04, "blocks": ["885420.DDBK", "885784.DDBK", "885705.DDBK", "885543.DDBK", "885768.DDBK", "885851.DDBK", "885990.DDBK"], "industry": 0, "rank": 14, "block_dict": {"885420.DDBK": {"blockCode": "885420.DDBK", "num": 100.02, "prePctChangeRate": 0.77, "numChange": -40.07, "rank": 21}, "885784.DDBK": {"blockCode": "885784.DDBK", "num": 60.02, "prePctChangeRate": 0.44, "numChange": -20.03, "rank": 44}, "885705.DDBK": {"blockCode": "885705.DDBK", "num": 40.01, "prePctChangeRate": 0.59, "numChange": -30.02, "rank": 79}, "885543.DDBK": {"blockCode": "885543.DDBK", "num": 30.02, "prePctChangeRate": 1.7, "numChange": 0.02, "rank": 87}, "885768.DDBK": {"blockCode": "885768.DDBK", "num": 30.02, "prePctChangeRate": 2.34, "numChange": 30.02, "rank": 88}, "885851.DDBK": {"blockCode": "885851.DDBK", "num": 10.01, "prePctChangeRate": 1.48, "numChange": 0.01, "rank": 192}, "885990.DDBK": {"blockCode": "885990.DDBK", "num": 10.0, "prePctChangeRate": 1.64, "numChange": -10.0, "rank": 247}}}, "000019.BKDL": {"categoryCode": "000019.BKDL", "categoryName": "其他小概念", "num": 190.07, "prePctChangeRate": 0.88, "numChange": -60.05, "blocks": ["885522.DDBK", "885753.DDBK", "886012.DDBK"], "industry": 0, "rank": 15, "block_dict": {"885522.DDBK": {"blockCode": "885522.DDBK", "num": 30.0, "prePctChangeRate": 0.92, "numChange": -20.01, "rank": 115}, "885753.DDBK": {"blockCode": "885753.DDBK", "num": 10.0, "prePctChangeRate": 0.21, "numChange": 10.0, "rank": 248}, "886012.DDBK": {"blockCode": "886012.DDBK", "num": 10.0, "prePctChangeRate": 0.98, "numChange": -0.01, "rank": 249}}}, "000014.BKDL": {"categoryCode": "000014.BKDL", "categoryName": "泛科技", "num": 190.05, "prePctChangeRate": 1.99, "numChange": 60.02, "blocks": ["885785.DDBK", "885454.DDBK", "886047.DDBK", "885771.DDBK", "885759.DDBK", "886036.DDBK"], "industry": 0, "rank": 16, "block_dict": {"885785.DDBK": {"blockCode": "885785.DDBK", "num": 90.05, "prePctChangeRate": 1.98, "numChange": 40.02, "rank": 22}, "885454.DDBK": {"blockCode": "885454.DDBK", "num": 70.02, "prePctChangeRate": 2.48, "numChange": 60.02, "rank": 34}, "886047.DDBK": {"blockCode": "886047.DDBK", "num": 30.0, "prePctChangeRate": 1.45, "numChange": 10.0, "rank": 116}, "885771.DDBK": {"blockCode": "885771.DDBK", "num": 20.0, "prePctChangeRate": 2.7, "numChange": 20.0, "rank": 173}, "885759.DDBK": {"blockCode": "885759.DDBK", "num": 10.0, "prePctChangeRate": 2.01, "numChange": -30.0, "rank": 250}, "886036.DDBK": {"blockCode": "886036.DDBK", "num": 10.0, "prePctChangeRate": 1.88, "numChange": -10.0, "rank": 251}}}, "000035.BKDL": {"categoryCode": "000035.BKDL", "categoryName": "**风光储电", "num": 170.1, "prePctChangeRate": 1.52, "numChange": -110.02, "blocks": ["885710.DDBK", "885531.DDBK", "885921.DDBK", "885789.DDBK", "886032.DDBK", "885944.DDBK", "886065.DDBK", "885425.DDBK", "885922.DDBK", "885936.DDBK", "885311.DDBK", "885781.DDBK", "886004.DDBK", "886038.DDBK"], "industry": 0, "rank": 17, "block_dict": {"885710.DDBK": {"blockCode": "885710.DDBK", "num": 60.03, "prePctChangeRate": 1.78, "numChange": 0.02, "rank": 40}, "885531.DDBK": {"blockCode": "885531.DDBK", "num": 50.04, "prePctChangeRate": 1.33, "numChange": -40.0, "rank": 48}, "885921.DDBK": {"blockCode": "885921.DDBK", "num": 50.04, "prePctChangeRate": 1.76, "numChange": 0.02, "rank": 49}, "885789.DDBK": {"blockCode": "885789.DDBK", "num": 40.02, "prePctChangeRate": 2.34, "numChange": -40.02, "rank": 69}, "886032.DDBK": {"blockCode": "886032.DDBK", "num": 20.02, "prePctChangeRate": 2.23, "numChange": -50.01, "rank": 123}, "885944.DDBK": {"blockCode": "885944.DDBK", "num": 20.01, "prePctChangeRate": 2.13, "numChange": -10.0, "rank": 138}, "886065.DDBK": {"blockCode": "886065.DDBK", "num": 20.01, "prePctChangeRate": 2.03, "numChange": 10.0, "rank": 139}, "885425.DDBK": {"blockCode": "885425.DDBK", "num": 10.01, "prePctChangeRate": 1.35, "numChange": 10.01, "rank": 193}, "885922.DDBK": {"blockCode": "885922.DDBK", "num": 10.01, "prePctChangeRate": 0.68, "numChange": 0.0, "rank": 194}, "885936.DDBK": {"blockCode": "885936.DDBK", "num": 10.01, "prePctChangeRate": -0.06, "numChange": -30.01, "rank": 195}, "885311.DDBK": {"blockCode": "885311.DDBK", "num": 10.0, "prePctChangeRate": 1.3, "numChange": 0.0, "rank": 252}, "885781.DDBK": {"blockCode": "885781.DDBK", "num": 10.0, "prePctChangeRate": 1.5, "numChange": 10.0, "rank": 253}, "886004.DDBK": {"blockCode": "886004.DDBK", "num": 10.0, "prePctChangeRate": 1.36, "numChange": 10.0, "rank": 254}, "886038.DDBK": {"blockCode": "886038.DDBK", "num": 10.0, "prePctChangeRate": 2.41, "numChange": 10.0, "rank": 255}}}, "000029.BKDL": {"categoryCode": "000029.BKDL", "categoryName": "电子类", "num": 160.08, "prePctChangeRate": 1.88, "numChange": 80.05, "blocks": ["885800.DDBK", "885959.DDBK", "885675.DDBK", "885774.DDBK", "885809.DDBK", "885738.DDBK", "885875.DDBK"], "industry": 0, "rank": 18, "block_dict": {"885800.DDBK": {"blockCode": "885800.DDBK", "num": 120.06, "prePctChangeRate": 2.26, "numChange": 90.03, "rank": 12}, "885959.DDBK": {"blockCode": "885959.DDBK", "num": 30.01, "prePctChangeRate": 1.96, "numChange": 10.01, "rank": 108}, "885675.DDBK": {"blockCode": "885675.DDBK", "num": 10.01, "prePctChangeRate": 1.56, "numChange": 0.01, "rank": 196}, "885774.DDBK": {"blockCode": "885774.DDBK", "num": 10.01, "prePctChangeRate": 2.56, "numChange": 10.01, "rank": 197}, "885809.DDBK": {"blockCode": "885809.DDBK", "num": 10.01, "prePctChangeRate": 1.69, "numChange": 0.0, "rank": 198}, "885738.DDBK": {"blockCode": "885738.DDBK", "num": 10.0, "prePctChangeRate": 1.42, "numChange": 10.0, "rank": 256}, "885875.DDBK": {"blockCode": "885875.DDBK", "num": 10.0, "prePctChangeRate": 1.72, "numChange": -20.0, "rank": 257}}}, "T08.ZHBK": {"categoryCode": "T08.ZHBK", "categoryName": "电子", "num": 140.06, "prePctChangeRate": 1.88, "numChange": 100.05, "blocks": ["T08.ZHBK"], "industry": 1, "rank": 19, "block_dict": {"T08.ZHBK": {"blockCode": "T08.ZHBK", "num": 140.06, "prePctChangeRate": 1.88, "numChange": 100.05, "rank": 9}}}, "000031.BKDL": {"categoryCode": "000031.BKDL", "categoryName": "算力芯片", "num": 130.07, "prePctChangeRate": 1.83, "numChange": -19.97, "blocks": ["885957.DDBK", "885756.DDBK", "885362.DDBK", "885790.DDBK", "886050.DDBK", "885908.DDBK", "885925.DDBK", "886033.DDBK", "886049.DDBK", "886042.DDBK", "886044.DDBK", "886048.DDBK"], "industry": 0, "rank": 20, "block_dict": {"885957.DDBK": {"blockCode": "885957.DDBK", "num": 40.03, "prePctChangeRate": 1.65, "numChange": 20.03, "rank": 59}, "885756.DDBK": {"blockCode": "885756.DDBK", "num": 40.02, "prePctChangeRate": 1.92, "numChange": -10.0, "rank": 70}, "885362.DDBK": {"blockCode": "885362.DDBK", "num": 30.0, "prePctChangeRate": 1.65, "numChange": -10.0, "rank": 117}, "885790.DDBK": {"blockCode": "885790.DDBK", "num": 20.02, "prePctChangeRate": 1.73, "numChange": -19.99, "rank": 124}, "886050.DDBK": {"blockCode": "886050.DDBK", "num": 20.01, "prePctChangeRate": 1.77, "numChange": 10.01, "rank": 140}, "885908.DDBK": {"blockCode": "885908.DDBK", "num": 10.01, "prePctChangeRate": 1.95, "numChange": 0.01, "rank": 199}, "885925.DDBK": {"blockCode": "885925.DDBK", "num": 10.01, "prePctChangeRate": 2.01, "numChange": 0.0, "rank": 200}, "886033.DDBK": {"blockCode": "886033.DDBK", "num": 10.01, "prePctChangeRate": 2.14, "numChange": -40.0, "rank": 201}, "886049.DDBK": {"blockCode": "886049.DDBK", "num": 10.01, "prePctChangeRate": 2.23, "numChange": 10.01, "rank": 202}, "886042.DDBK": {"blockCode": "886042.DDBK", "num": 10.0, "prePctChangeRate": 2.07, "numChange": 0.0, "rank": 258}, "886044.DDBK": {"blockCode": "886044.DDBK", "num": 10.0, "prePctChangeRate": 1.9, "numChange": -0.01, "rank": 259}, "886048.DDBK": {"blockCode": "886048.DDBK", "num": 10.0, "prePctChangeRate": 2.18, "numChange": 10.0, "rank": 260}}}, "000016.BKDL": {"categoryCode": "000016.BKDL", "categoryName": "软件互联网", "num": 130.05, "prePctChangeRate": 1.96, "numChange": 50.05, "blocks": ["885378.DDBK", "885568.DDBK", "885956.DDBK", "885459.DDBK", "885730.DDBK", "886017.DDBK", "885844.DDBK", "885881.DDBK", "886039.DDBK"], "industry": 0, "rank": 21, "block_dict": {"885378.DDBK": {"blockCode": "885378.DDBK", "num": 60.02, "prePctChangeRate": 1.65, "numChange": 20.02, "rank": 45}, "885568.DDBK": {"blockCode": "885568.DDBK", "num": 40.02, "prePctChangeRate": 2.17, "numChange": 10.02, "rank": 71}, "885956.DDBK": {"blockCode": "885956.DDBK", "num": 30.02, "prePctChangeRate": 1.84, "numChange": 20.02, "rank": 89}, "885459.DDBK": {"blockCode": "885459.DDBK", "num": 20.01, "prePctChangeRate": 2.03, "numChange": 0.01, "rank": 141}, "885730.DDBK": {"blockCode": "885730.DDBK", "num": 20.01, "prePctChangeRate": 2.18, "numChange": 10.01, "rank": 142}, "886017.DDBK": {"blockCode": "886017.DDBK", "num": 20.01, "prePctChangeRate": 2.01, "numChange": 20.01, "rank": 143}, "885844.DDBK": {"blockCode": "885844.DDBK", "num": 10.0, "prePctChangeRate": 1.2, "numChange": 10.0, "rank": 261}, "885881.DDBK": {"blockCode": "885881.DDBK", "num": 10.0, "prePctChangeRate": 1.65, "numChange": -10.0, "rank": 262}, "886039.DDBK": {"blockCode": "886039.DDBK", "num": 10.0, "prePctChangeRate": 1.92, "numChange": 10.0, "rank": 263}}}, "T07.ZHBK": {"categoryCode": "T07.ZHBK", "categoryName": "机械设备", "num": 130.05, "prePctChangeRate": 2.28, "numChange": 19.98, "blocks": ["T07.ZHBK"], "industry": 1, "rank": 22, "block_dict": {"T07.ZHBK": {"blockCode": "T07.ZHBK", "num": 130.05, "prePctChangeRate": 2.28, "numChange": 19.98, "rank": 11}}}, "000007.BKDL": {"categoryCode": "000007.BKDL", "categoryName": "**房产建筑", "num": 120.06, "prePctChangeRate": 0.87, "numChange": 40.01, "blocks": ["885478.DDBK", "885991.DDBK", "885750.DDBK", "885915.DDBK", "885386.DDBK"], "industry": 0, "rank": 23, "block_dict": {"885478.DDBK": {"blockCode": "885478.DDBK", "num": 70.03, "prePctChangeRate": 1.45, "numChange": 40.01, "rank": 32}, "885991.DDBK": {"blockCode": "885991.DDBK", "num": 30.02, "prePctChangeRate": 0.93, "numChange": 0.0, "rank": 90}, "885750.DDBK": {"blockCode": "885750.DDBK", "num": 10.01, "prePctChangeRate": 0.59, "numChange": 0.0, "rank": 203}, "885915.DDBK": {"blockCode": "885915.DDBK", "num": 10.01, "prePctChangeRate": -0.04, "numChange": 0.0, "rank": 204}, "885386.DDBK": {"blockCode": "885386.DDBK", "num": 10.0, "prePctChangeRate": 0.65, "numChange": -0.01, "rank": 264}}}, "000032.BKDL": {"categoryCode": "000032.BKDL", "categoryName": "新股次新股", "num": 120.04, "prePctChangeRate": 1.92, "numChange": 50.03, "blocks": ["885598.DDBK", "885905.DDBK", "885907.DDBK", "885906.DDBK"], "industry": 0, "rank": 24, "block_dict": {"885598.DDBK": {"blockCode": "885598.DDBK", "num": 120.04, "prePctChangeRate": 1.92, "numChange": 50.03, "rank": 13}, "885905.DDBK": {"blockCode": "885905.DDBK", "num": 100.04, "prePctChangeRate": 2.07, "numChange": 50.04, "rank": 19}, "885907.DDBK": {"blockCode": "885907.DDBK", "num": 20.0, "prePctChangeRate": 2.15, "numChange": 20.0, "rank": 174}, "885906.DDBK": {"blockCode": "885906.DDBK", "num": 10.0, "prePctChangeRate": 1.02, "numChange": 0.0, "rank": 265}}}, "000003.BKDL": {"categoryCode": "000003.BKDL", "categoryName": "数字科技", "num": 110.04, "prePctChangeRate": 1.5, "numChange": -60.01, "blocks": ["885976.DDBK", "885452.DDBK", "886041.DDBK", "885757.DDBK", "886013.DDBK", "885887.DDBK", "885942.DDBK", "885333.DDBK", "885866.DDBK", "885966.DDBK", "885975.DDBK", "886023.DDBK", "886034.DDBK"], "industry": 0, "rank": 25, "block_dict": {"885976.DDBK": {"blockCode": "885976.DDBK", "num": 70.03, "prePctChangeRate": 1.65, "numChange": 20.0, "rank": 33}, "885452.DDBK": {"blockCode": "885452.DDBK", "num": 50.03, "prePctChangeRate": 1.6, "numChange": 0.02, "rank": 51}, "886041.DDBK": {"blockCode": "886041.DDBK", "num": 50.02, "prePctChangeRate": 1.89, "numChange": 20.02, "rank": 55}, "885757.DDBK": {"blockCode": "885757.DDBK", "num": 50.01, "prePctChangeRate": 1.25, "numChange": 30.01, "rank": 56}, "886013.DDBK": {"blockCode": "886013.DDBK", "num": 40.03, "prePctChangeRate": 1.96, "numChange": 20.03, "rank": 60}, "885887.DDBK": {"blockCode": "885887.DDBK", "num": 30.02, "prePctChangeRate": 1.65, "numChange": -39.98, "rank": 91}, "885942.DDBK": {"blockCode": "885942.DDBK", "num": 20.02, "prePctChangeRate": 1.74, "numChange": 10.02, "rank": 125}, "885333.DDBK": {"blockCode": "885333.DDBK", "num": 20.01, "prePctChangeRate": 1.83, "numChange": 10.01, "rank": 144}, "885866.DDBK": {"blockCode": "885866.DDBK", "num": 20.01, "prePctChangeRate": 1.93, "numChange": 10.01, "rank": 145}, "885966.DDBK": {"blockCode": "885966.DDBK", "num": 20.01, "prePctChangeRate": 1.05, "numChange": 10.01, "rank": 146}, "885975.DDBK": {"blockCode": "885975.DDBK", "num": 20.01, "prePctChangeRate": 1.59, "numChange": 20.01, "rank": 147}, "886023.DDBK": {"blockCode": "886023.DDBK", "num": 10.01, "prePctChangeRate": 1.26, "numChange": 10.01, "rank": 205}, "886034.DDBK": {"blockCode": "886034.DDBK", "num": 10.0, "prePctChangeRate": 1.01, "numChange": 10.0, "rank": 266}}}, "000017.BKDL": {"categoryCode": "000017.BKDL", "categoryName": "**中字头国企", "num": 110.03, "prePctChangeRate": 1.06, "numChange": -0.03, "blocks": ["886021.DDBK", "885595.DDBK"], "industry": 0, "rank": 26, "block_dict": {"886021.DDBK": {"blockCode": "886021.DDBK", "num": 110.03, "prePctChangeRate": 1.06, "numChange": -0.03, "rank": 16}, "885595.DDBK": {"blockCode": "885595.DDBK", "num": 30.02, "prePctChangeRate": 1.62, "numChange": -9.99, "rank": 92}}}, "000011.BKDL": {"categoryCode": "000011.BKDL", "categoryName": "**环保碳中和", "num": 100.05, "prePctChangeRate": 1.05, "numChange": -9.99, "blocks": ["885563.DDBK", "885410.DDBK", "885412.DDBK", "885690.DDBK", "885823.DDBK", "885555.DDBK", "885961.DDBK", "885992.DDBK", "886052.DDBK"], "industry": 0, "rank": 27, "block_dict": {"885563.DDBK": {"blockCode": "885563.DDBK", "num": 30.01, "prePctChangeRate": 0.77, "numChange": 20.0, "rank": 109}, "885410.DDBK": {"blockCode": "885410.DDBK", "num": 20.01, "prePctChangeRate": 0.52, "numChange": 10.0, "rank": 148}, "885412.DDBK": {"blockCode": "885412.DDBK", "num": 20.01, "prePctChangeRate": 0.82, "numChange": -10.01, "rank": 149}, "885690.DDBK": {"blockCode": "885690.DDBK", "num": 20.01, "prePctChangeRate": 0.59, "numChange": -20.01, "rank": 150}, "885823.DDBK": {"blockCode": "885823.DDBK", "num": 20.01, "prePctChangeRate": 1.19, "numChange": -20.0, "rank": 151}, "885555.DDBK": {"blockCode": "885555.DDBK", "num": 10.01, "prePctChangeRate": 0.7, "numChange": 10.01, "rank": 206}, "885961.DDBK": {"blockCode": "885961.DDBK", "num": 10.01, "prePctChangeRate": 0.22, "numChange": 0.01, "rank": 207}, "885992.DDBK": {"blockCode": "885992.DDBK", "num": 10.0, "prePctChangeRate": 1.98, "numChange": 10.0, "rank": 267}, "886052.DDBK": {"blockCode": "886052.DDBK", "num": 10.0, "prePctChangeRate": 1.22, "numChange": 10.0, "rank": 268}}}, "T09.ZHBK": {"categoryCode": "T09.ZHBK", "categoryName": "汽车", "num": 100.05, "prePctChangeRate": 2.33, "numChange": -39.98, "blocks": ["T09.ZHBK"], "industry": 1, "rank": 28, "block_dict": {"T09.ZHBK": {"blockCode": "T09.ZHBK", "num": 100.05, "prePctChangeRate": 2.33, "numChange": -39.98, "rank": 17}}}, "000015.BKDL": {"categoryCode": "000015.BKDL", "categoryName": "**非银金融", "num": 90.04, "prePctChangeRate": 1.01, "numChange": -40.04, "blocks": ["885413.DDBK", "885456.DDBK", "885749.DDBK", "885657.DDBK", "885570.DDBK", "885620.DDBK", "885835.DDBK"], "industry": 0, "rank": 29, "block_dict": {"885413.DDBK": {"blockCode": "885413.DDBK", "num": 60.03, "prePctChangeRate": 1.1, "numChange": 0.0, "rank": 41}, "885456.DDBK": {"blockCode": "885456.DDBK", "num": 20.01, "prePctChangeRate": 1.98, "numChange": -20.01, "rank": 152}, "885749.DDBK": {"blockCode": "885749.DDBK", "num": 20.01, "prePctChangeRate": 1.04, "numChange": 10.01, "rank": 153}, "885657.DDBK": {"blockCode": "885657.DDBK", "num": 10.01, "prePctChangeRate": 0.79, "numChange": -20.0, "rank": 208}, "885570.DDBK": {"blockCode": "885570.DDBK", "num": 10.0, "prePctChangeRate": 1.62, "numChange": 10.0, "rank": 269}, "885620.DDBK": {"blockCode": "885620.DDBK", "num": 10.0, "prePctChangeRate": 0.68, "numChange": -0.01, "rank": 270}, "885835.DDBK": {"blockCode": "885835.DDBK", "num": 10.0, "prePctChangeRate": 0.52, "numChange": -0.01, "rank": 271}}}, "000037.BKDL": {"categoryCode": "000037.BKDL", "categoryName": "**大市值股票", "num": 90.03, "prePctChangeRate": 1.1, "numChange": -10.0, "blocks": ["885520.DDBK", "885745.DDBK", "883303.DDBK", "885663.DDBK"], "industry": 0, "rank": 30, "block_dict": {"885520.DDBK": {"blockCode": "885520.DDBK", "num": 70.02, "prePctChangeRate": 1.06, "numChange": 0.0, "rank": 35}, "885745.DDBK": {"blockCode": "885745.DDBK", "num": 40.02, "prePctChangeRate": 1.31, "numChange": 0.0, "rank": 72}, "883303.DDBK": {"blockCode": "883303.DDBK", "num": 30.01, "prePctChangeRate": 1.23, "numChange": 20.0, "rank": 110}, "885663.DDBK": {"blockCode": "885663.DDBK", "num": 10.0, "prePctChangeRate": 0.81, "numChange": 10.0, "rank": 272}}}, "000008.BKDL": {"categoryCode": "000008.BKDL", "categoryName": "**传统行业", "num": 90.02, "prePctChangeRate": 1.23, "numChange": 30.01, "blocks": ["885572.DDBK", "885433.DDBK", "885692.DDBK", "885423.DDBK", "885914.DDBK"], "industry": 0, "rank": 31, "block_dict": {"885572.DDBK": {"blockCode": "885572.DDBK", "num": 30.01, "prePctChangeRate": 1.03, "numChange": 10.0, "rank": 111}, "885433.DDBK": {"blockCode": "885433.DDBK", "num": 20.01, "prePctChangeRate": 1.21, "numChange": 20.01, "rank": 154}, "885692.DDBK": {"blockCode": "885692.DDBK", "num": 20.01, "prePctChangeRate": 1.0, "numChange": 10.0, "rank": 155}, "885423.DDBK": {"blockCode": "885423.DDBK", "num": 20.0, "prePctChangeRate": 2.26, "numChange": -10.0, "rank": 175}, "885914.DDBK": {"blockCode": "885914.DDBK", "num": 10.0, "prePctChangeRate": 0.15, "numChange": 10.0, "rank": 273}}}, "000021.BKDL": {"categoryCode": "000021.BKDL", "categoryName": "**三胎", "num": 80.03, "prePctChangeRate": 0.7, "numChange": -20.01, "blocks": ["885540.DDBK", "885968.DDBK"], "industry": 0, "rank": 32, "block_dict": {"885540.DDBK": {"blockCode": "885540.DDBK", "num": 80.03, "prePctChangeRate": 0.88, "numChange": -0.01, "rank": 25}, "885968.DDBK": {"blockCode": "885968.DDBK", "num": 20.01, "prePctChangeRate": 0.06, "numChange": -0.01, "rank": 156}}}, "T15.ZHBK": {"categoryCode": "T15.ZHBK", "categoryName": "医药生物", "num": 80.0, "prePctChangeRate": 0.21, "numChange": 50.0, "blocks": ["T15.ZHBK"], "industry": 1, "rank": 33, "block_dict": {"T15.ZHBK": {"blockCode": "T15.ZHBK", "num": 80.0, "prePctChangeRate": 0.21, "numChange": 50.0, "rank": 28}}}, "T24.ZHBK": {"categoryCode": "T24.ZHBK", "categoryName": "国防军工", "num": 70.02, "prePctChangeRate": 4.53, "numChange": 30.01, "blocks": ["T24.ZHBK"], "industry": 1, "rank": 34, "block_dict": {"T24.ZHBK": {"blockCode": "T24.ZHBK", "num": 70.02, "prePctChangeRate": 4.53, "numChange": 30.01, "rank": 36}}}, "T03.ZHBK": {"categoryCode": "T03.ZHBK", "categoryName": "基础化工", "num": 70.02, "prePctChangeRate": 1.14, "numChange": 9.97, "blocks": ["T03.ZHBK"], "industry": 1, "rank": 35, "block_dict": {"T03.ZHBK": {"blockCode": "T03.ZHBK", "num": 70.02, "prePctChangeRate": 1.14, "numChange": 9.97, "rank": 37}}}, "T10.ZHBK": {"categoryCode": "T10.ZHBK", "categoryName": "计算机", "num": 60.03, "prePctChangeRate": 2.03, "numChange": 10.03, "blocks": ["T10.ZHBK"], "industry": 1, "rank": 36, "block_dict": {"T10.ZHBK": {"blockCode": "T10.ZHBK", "num": 60.03, "prePctChangeRate": 2.03, "numChange": 10.03, "rank": 42}}}, "T14.ZHBK": {"categoryCode": "T14.ZHBK", "categoryName": "轻工制造", "num": 60.01, "prePctChangeRate": 0.88, "numChange": 9.98, "blocks": ["T14.ZHBK"], "industry": 1, "rank": 37, "block_dict": {"T14.ZHBK": {"blockCode": "T14.ZHBK", "num": 60.01, "prePctChangeRate": 0.88, "numChange": 9.98, "rank": 47}}}, "000024.BKDL": {"categoryCode": "000024.BKDL", "categoryName": "**农业", "num": 60.0, "prePctChangeRate": 0.43, "numChange": 19.98, "blocks": ["885706.DDBK", "885967.DDBK", "885863.DDBK", "885962.DDBK"], "industry": 0, "rank": 38, "block_dict": {"885706.DDBK": {"blockCode": "885706.DDBK", "num": 20.0, "prePctChangeRate": 2.06, "numChange": -0.01, "rank": 176}, "885967.DDBK": {"blockCode": "885967.DDBK", "num": 20.0, "prePctChangeRate": 0.27, "numChange": 10.0, "rank": 177}, "885863.DDBK": {"blockCode": "885863.DDBK", "num": 10.0, "prePctChangeRate": 0.95, "numChange": 0.0, "rank": 274}, "885962.DDBK": {"blockCode": "885962.DDBK", "num": 10.0, "prePctChangeRate": 0.31, "numChange": 10.0, "rank": 275}}}, "T26.ZHBK": {"categoryCode": "T26.ZHBK", "categoryName": "电力设备", "num": 50.03, "prePctChangeRate": 2.05, "numChange": -9.99, "blocks": ["T26.ZHBK"], "industry": 1, "rank": 39, "block_dict": {"T26.ZHBK": {"blockCode": "T26.ZHBK", "num": 50.03, "prePctChangeRate": 2.05, "numChange": -9.99, "rank": 52}}}, "000001.BKDL": {"categoryCode": "000001.BKDL", "categoryName": "**有色贵金属", "num": 50.02, "prePctChangeRate": 1.0, "numChange": 10.0, "blocks": ["885355.DDBK", "885343.DDBK", "885552.DDBK", "885970.DDBK", "885973.DDBK", "885530.DDBK"], "industry": 0, "rank": 40, "block_dict": {"885355.DDBK": {"blockCode": "885355.DDBK", "num": 20.0, "prePctChangeRate": 1.61, "numChange": 10.0, "rank": 178}, "885343.DDBK": {"blockCode": "885343.DDBK", "num": 10.01, "prePctChangeRate": 1.92, "numChange": -10.0, "rank": 209}, "885552.DDBK": {"blockCode": "885552.DDBK", "num": 10.01, "prePctChangeRate": 0.84, "numChange": 0.0, "rank": 210}, "885970.DDBK": {"blockCode": "885970.DDBK", "num": 10.01, "prePctChangeRate": 1.01, "numChange": 0.0, "rank": 211}, "885973.DDBK": {"blockCode": "885973.DDBK", "num": 10.01, "prePctChangeRate": 0.97, "numChange": 0.0, "rank": 212}, "885530.DDBK": {"blockCode": "885530.DDBK", "num": 10.0, "prePctChangeRate": -0.1, "numChange": 10.0, "rank": 276}}}, "000028.BKDL": {"categoryCode": "000028.BKDL", "categoryName": "**ST板块", "num": 40.04, "prePctChangeRate": -0.72, "numChange": -40.01, "blocks": ["885699.DDBK"], "industry": 0, "rank": 41, "block_dict": {"885699.DDBK": {"blockCode": "885699.DDBK", "num": 40.04, "prePctChangeRate": -0.72, "numChange": -40.01, "rank": 57}}}, "T13.ZHBK": {"categoryCode": "T13.ZHBK", "categoryName": "纺织服饰", "num": 40.02, "prePctChangeRate": 0.48, "numChange": -70.03, "blocks": ["T13.ZHBK"], "industry": 1, "rank": 42, "block_dict": {"T13.ZHBK": {"blockCode": "T13.ZHBK", "num": 40.02, "prePctChangeRate": 0.48, "numChange": -70.03, "rank": 73}}}, "T11.ZHBK": {"categoryCode": "T11.ZHBK", "categoryName": "家用电器", "num": 40.02, "prePctChangeRate": 1.98, "numChange": 20.02, "blocks": ["T11.ZHBK"], "industry": 1, "rank": 43, "block_dict": {"T11.ZHBK": {"blockCode": "T11.ZHBK", "num": 40.02, "prePctChangeRate": 1.98, "numChange": 20.02, "rank": 74}}}, "000022.BKDL": {"categoryCode": "000022.BKDL", "categoryName": "**化工", "num": 40.01, "prePctChangeRate": 1.21, "numChange": -20.02, "blocks": ["885882.DDBK", "885650.DDBK", "885912.DDBK"], "industry": 0, "rank": 44, "block_dict": {"885882.DDBK": {"blockCode": "885882.DDBK", "num": 20.01, "prePctChangeRate": 0.55, "numChange": 10.01, "rank": 157}, "885650.DDBK": {"blockCode": "885650.DDBK", "num": 10.0, "prePctChangeRate": 2.04, "numChange": 10.0, "rank": 277}, "885912.DDBK": {"blockCode": "885912.DDBK", "num": 10.0, "prePctChangeRate": 1.2, "numChange": -0.01, "rank": 278}}}, "000036.BKDL": {"categoryCode": "000036.BKDL", "categoryName": "新材料", "num": 40.0, "prePctChangeRate": 1.67, "numChange": -30.03, "blocks": ["885544.DDBK", "886063.DDBK"], "industry": 0, "rank": 45, "block_dict": {"885544.DDBK": {"blockCode": "885544.DDBK", "num": 40.0, "prePctChangeRate": 1.67, "numChange": -30.03, "rank": 81}, "886063.DDBK": {"blockCode": "886063.DDBK", "num": 10.0, "prePctChangeRate": 1.56, "numChange": -0.01, "rank": 279}}}, "T25.ZHBK": {"categoryCode": "T25.ZHBK", "categoryName": "建筑装饰", "num": 30.02, "prePctChangeRate": 0.94, "numChange": -10.01, "blocks": ["T25.ZHBK"], "industry": 1, "rank": 46, "block_dict": {"T25.ZHBK": {"blockCode": "T25.ZHBK", "num": 30.02, "prePctChangeRate": 0.94, "numChange": -10.01, "rank": 93}}}, "T28.ZHBK": {"categoryCode": "T28.ZHBK", "categoryName": "通信", "num": 30.02, "prePctChangeRate": 1.85, "numChange": -29.99, "blocks": ["T28.ZHBK"], "industry": 1, "rank": 47, "block_dict": {"T28.ZHBK": {"blockCode": "T28.ZHBK", "num": 30.02, "prePctChangeRate": 1.85, "numChange": -29.99, "rank": 94}}}, "T17.ZHBK": {"categoryCode": "T17.ZHBK", "categoryName": "交通运输", "num": 30.02, "prePctChangeRate": 1.12, "numChange": 20.02, "blocks": ["T17.ZHBK"], "industry": 1, "rank": 48, "block_dict": {"T17.ZHBK": {"blockCode": "T17.ZHBK", "num": 30.02, "prePctChangeRate": 1.12, "numChange": 20.02, "rank": 95}}}, "T20.ZHBK": {"categoryCode": "T20.ZHBK", "categoryName": "商贸零售", "num": 20.0, "prePctChangeRate": 0.72, "numChange": 9.99, "blocks": ["T20.ZHBK"], "industry": 1, "rank": 49, "block_dict": {"T20.ZHBK": {"blockCode": "T20.ZHBK", "num": 20.0, "prePctChangeRate": 0.72, "numChange": 9.99, "rank": 179}}}, "T29.ZHBK": {"categoryCode": "T29.ZHBK", "categoryName": "石油石化", "num": 10.01, "prePctChangeRate": 0.38, "numChange": 10.01, "blocks": ["T29.ZHBK"], "industry": 1, "rank": 50, "block_dict": {"T29.ZHBK": {"blockCode": "T29.ZHBK", "num": 10.01, "prePctChangeRate": 0.38, "numChange": 10.01, "rank": 213}}}, "T18.ZHBK": {"categoryCode": "T18.ZHBK", "categoryName": "房地产", "num": 10.01, "prePctChangeRate": 0.46, "numChange": 0.0, "blocks": ["T18.ZHBK"], "industry": 1, "rank": 51, "block_dict": {"T18.ZHBK": {"blockCode": "T18.ZHBK", "num": 10.01, "prePctChangeRate": 0.46, "numChange": 0.0, "rank": 214}}}}

In [52]:
abs2 = []
for c,d in tt.items():
    blocks = d['blocks']
    abs2.extend(blocks)

In [53]:
len(abs2)

279

In [87]:
'883436.DDBK' in abs2

False

In [ ]:
'886079.DDBK'
'883436.DDBK',
'885338.DDBK',

In [77]:
cc = []

In [78]:
for x in abs:
    if x not in abs2:
        cc.append(x)

In [79]:
len(cc)

62

In [80]:
len(abs)

233

In [81]:
len(abs2)

279

In [88]:
dd = []
for x in abs2:
    if x not in abs:
        dd.append(x)

In [ ]:
len(dd)

108

: 

In [1]:
default_strategy_positions = {
    "低吸-高强中低开低吸:强方向前2": 0.25,
    "低吸-高强中低开低吸:方向前1": 0.25,
    "低吸-高强中低开低吸:强方向前1": 0.25,
    "低吸-高强中低开低吸:方向前2": 0.25,
    "低吸-中位孕线低吸:方向2": 0.25,
    "低吸-中位孕线低吸:方向1": 0.25,
    "低吸-低位中强中低开低吸": 0.25,
    "低吸-低位高强低吸:中低频2": 0.25,
    "低吸-低位高强低吸:中低频3": 0.25,
    "低吸-低位高强低吸:中低频4": 0.25,
    "低吸-低位中强低吸:方向低频前2": 0.25,
    "低吸-低位中强低吸:方向低频前1": 0.25,
    "低吸-低位高强中低开低吸:方向低频": 0.25,
    "低吸-低位高强中低开低吸:方向低频2": 0.25,
    "低吸-低位高强中低开低吸:方向低频3": 0.25
}

def get_strategy_position(strategy, sub_strategy=None, default_strategy_positions=default_strategy_positions):
    """
    根据输入的 strategy 和 sub_strategy 获取对应的系数。
    如果只提供 strategy，则返回只匹配 strategy 的 key 对应的 value；
    如果提供了 sub_strategy，则尝试匹配完整的 'strategy:sub_strategy' 格式的 key。

    :param strategy: 策略名称
    :param sub_strategy: 子策略名称，可选
    :return: 对应的系数，如果未找到则返回 None
    """
    if sub_strategy:
        full_key = f"{strategy}:{sub_strategy}"
        if full_key in default_strategy_positions:
            return default_strategy_positions[full_key]
    
    # 尝试匹配只包含 strategy 的 key
    for key in default_strategy_positions:
        if key == strategy:
            return default_strategy_positions[key]
    
    return 0

In [3]:
get_strategy_position("低吸-低位中强中低开低吸", "")

0.25

In [1]:
def modify_string_slice(input_str, index, new_value):
    """
    使用切片修改指定位置的字符值

    :param input_str: 输入的字符串
    :param index: 要修改的位置索引
    :param new_value: 新的值，只能是 '0' 或 '1'
    :return: 修改后的字符串
    """
    if new_value not in ['0', '1']:
        raise ValueError("新值必须是 '0' 或 '1'")
    if index < 0 or index >= len(input_str):
        raise IndexError("索引超出范围")
    
    return input_str[:index] + new_value + input_str[index + 1:]

# 初始化字符串
original_str = '0' * 10
print("初始字符串:", original_str)

# 修改操作
modified_str = modify_string_slice(original_str, 3, '1')
modified_str = modify_string_slice(modified_str, 4, '1')
modified_str = modify_string_slice(modified_str, 5, '1')
modified_str = modify_string_slice(modified_str, 6, '1')
modified_str = modify_string_slice(modified_str, 9, '1')
print("修改后的字符串:", modified_str)

初始字符串: 0000000000
修改后的字符串: 0001111001


In [2]:
a = {}
a[4] = 5

In [ ]:
# 模拟外部的 code_to_limit_up_monitor 字典
code_to_limit_up_monitor = {}

def add_limit_up_monitor(stock_code, row_ids = [], code_to_limit_up_monitor = code_to_limit_up_monitor):
    if not row_ids:
        return
    print("函数内部开始时的 code_to_limit_up_monitor:", code_to_limit_up_monitor)
    # 模拟一些操作
    code_to_limit_up_monitor[stock_code] = "monitor_instance"
    print("函数内部结束时的 code_to_limit_up_monitor:", code_to_limit_up_monitor)

# 调用函数
add_limit_up_monitor("603823.SH", [1])

# 在函数外部修改 code_to_limit_up_monitor
code_to_limit_up_monitor["600000.SH"] = "new_monitor_instance"

# 再次调用函数
add_limit_up_monitor("600519.SH", [2])

函数内部开始时的 code_to_limit_up_monitor: {}
函数内部结束时的 code_to_limit_up_monitor: {'603823.SH': 'monitor_instance'}
函数内部开始时的 code_to_limit_up_monitor: {'603823.SH': 'monitor_instance', '600000.SH': 'new_monitor_instance'}
函数内部结束时的 code_to_limit_up_monitor: {'603823.SH': 'monitor_instance', '600000.SH': 'new_monitor_instance', '600519.SH': 'monitor_instance'}


: 

In [3]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a[:-5]

[1, 2, 3, 4, 5]

In [4]:
sum(a[:-5])

15

In [7]:
a = []
b = '35,12'.split(',')

In [8]:
a.extend(b)

In [ ]:
# 002104

['35', '12']

: 

In [ ]:
import requests

url = "http://api.zhituapi.com/hs/instrument/000566.SH?token=C0B5FFC3-CA8E-4036-B9C0-9E6B7021D9AD"

response = requests.get(url)

data = response.json()

print(data)

{'data': '<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.28.0</center>\r\n</body>\r\n</html>\r\n'}


: 

In [2]:
a = [1,2,3,4,5,6]

In [ ]:
min(a[-3:])

4

: 

In [8]:
import requests
import pandas as pd
import time

def get_reduction_stocks(token, days=7, max_retries=3):
    """
    获取最近N天的减持股公告
    :param token: ZhituAPI凭证
    :param days: 查询最近天数
    :param max_retries: 最大重试次数
    :return: DataFrame格式的减持股数据
    """
    url = "https://api.zhituapi.com/announcement/list"
    end_date = time.strftime("%Y%m%d", time.localtime())
    start_date = time.strftime("%Y%m%d", time.localtime(time.time() - days*86400))
    
    params = {
        "token": token,
        "start_date": start_date,
        "end_date": end_date,
        "page": 1,
        "per_page": 100  # 最大每页数量
    }
    
    # 带重试机制的请求
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()  # 检查HTTP错误
            
            data = response.json()
            if data["code"] != 0:
                raise ValueError(f"API错误: {data['msg']} (错误码: {data['code']})")
            
            # 解析数据
            announcements = data["data"]["list"]
            if not announcements:
                print("未查询到近期公告")
                return pd.DataFrame()
            
            # 筛选减持相关公告
            reduction_keywords = ["减持", "减持计划", "减持实施", "股东减持"]
            reduction_data = []
            
            for ann in announcements:
                title = ann["title"]
                # 检查标题是否含减持关键词
                if any(keyword in title for keyword in reduction_keywords):
                    # 提取关键信息
                    reduction_data.append({
                        "股票代码": ann["symbol"],
                        "股票简称": ann["name"],
                        "公告标题": title,
                        "公告日期": ann["publish_date"],
                        "公告链接": ann["url"],
                        "股东名称": parse_shareholder(title),  # 自定义解析函数
                        "减持数量(万股)": parse_amount(title)  # 自定义解析函数
                    })
            
            return pd.DataFrame(reduction_data)
        
        except requests.exceptions.RequestException as e:
            print(f"请求失败 ({attempt+1}/{max_retries}): {str(e)}")
            time.sleep(2)  # 指数退避重试
        except ValueError as e:
            print(f"数据处理错误: {str(e)}")
            break
    
    return pd.DataFrame()

def parse_shareholder(title):
    """从公告标题解析股东名称（简化示例）"""
    # 实际使用时需要更复杂的文本处理
    if "控股股东" in title: return "控股股东"
    if "实际控制人" in title: return "实际控制人"
    if "高管" in title: return "公司高管"
    return "未指定股东"

def parse_amount(title):
    """从公告标题解析减持数量（简化示例）"""
    # 实际使用时建议使用正则表达式提取精确数值
    if "不超过" in title: return "待定"
    if "万股" in title:
        start = title.find("减持") + 2
        return title[start:start+10]  # 简略截取
    return "未公布"

# 替换为你的实际token（免费申请：https://www.zhituapi.com/gettoken.html）
API_TOKEN = "C0B5FFC3-CA8E-4036-B9C0-9E6B7021D9AD"  

# 获取最近7天减持股
reduction_df = get_reduction_stocks(API_TOKEN, days=7)

if not reduction_df.empty:
    print(f"获取到 {len(reduction_df)} 条减持公告：")
    # 精简显示关键列
    print(reduction_df[["股票代码", "股票简称", "公告日期", "公告标题"]].to_string(index=False))
    
    # 导出到CSV（可选）
    reduction_df.to_csv("reduction_stocks.csv", index=False, encoding='utf_8_sig')
else:
    print("未找到减持股公告")

请求失败 (1/3): 404 Client Error: Not Found for url: https://api.zhituapi.com/announcement/list?token=C0B5FFC3-CA8E-4036-B9C0-9E6B7021D9AD&start_date=20250607&end_date=20250614&page=1&per_page=100
请求失败 (2/3): 404 Client Error: Not Found for url: https://api.zhituapi.com/announcement/list?token=C0B5FFC3-CA8E-4036-B9C0-9E6B7021D9AD&start_date=20250607&end_date=20250614&page=1&per_page=100
请求失败 (3/3): 404 Client Error: Not Found for url: https://api.zhituapi.com/announcement/list?token=C0B5FFC3-CA8E-4036-B9C0-9E6B7021D9AD&start_date=20250607&end_date=20250614&page=1&per_page=100
未找到减持股公告


In [3]:
for i in range(0, 20, 40):
    print(i)

0


In [1]:
a = list(range(20))

In [ ]:
a[10:100]

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

: 

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

def filter_strong_stocks(data: pd.DataFrame, window=5):
    """
    筛选近期强势股票
    :param data: 包含股票数据的DataFrame，需包含字段['code','date','close','volume']
    :param window: 分析窗口(默认5日)
    :return: 强势股DataFrame
    """
    # 按日期排序确保计算正确
    data = data.sort_values('date')
    
    # 基础计算
    data['pct_chg'] = data.groupby('code')['close'].pct_change()
    data['is_up'] = data['pct_chg'] > 0
    
    # 分组计算指标
    grouped = data.groupby('code')
    
    # 近5日涨幅
    last_close = grouped['close'].last()
    pre_close = grouped['close'].nth(-window)
    gain_5d = (last_close - pre_close) / pre_close
    
    # 近3日涨幅
    pre_3d_close = grouped['close'].nth(-3)
    gain_3d = (last_close - pre_3d_close) / pre_3d_close
    
    # 上涨天数占比
    up_ratio = grouped['is_up'].tail(window).groupby('code').mean()
    
    # 5日平均成交量
    vol_mean_5d = grouped['volume'].tail(window).groupby('code').mean()
    
    # 近3日量能均值
    vol_mean_3d = grouped['volume'].tail(3).groupby('code').mean()
    
    # 计算斜率
    def calc_slope(group):
        if len(group) < window:
            return np.nan
        x = np.arange(window)
        y = group['close'].iloc[-window:].values
        slope = linregress(x, y)[0]
        return slope / y.mean()  # 标准化斜率
    
    slopes = grouped.apply(calc_slope)
    
    # 构建结果DataFrame
    result = pd.DataFrame({
        'gain_5d': gain_5d,
        'gain_3d': gain_3d,
        'up_ratio': up_ratio,
        'vol_ratio': vol_mean_3d / vol_mean_5d,
        'slope': slopes
    }).dropna()
    
    # 筛选条件
    strong_mask = (
        (result['gain_5d'] > 0.15) &             # 5日涨幅>15%
        (result['gain_3d'] > 0.05) &              # 3日涨幅>5%
        (result['up_ratio'] >= 0.6) &             # 60%天数上涨
        (result['slope'] > 0) &                   # 趋势向上
        (result['vol_ratio'] > 1)                # 近期放量
    )
    
    return result[strong_mask].sort_values('slope', ascending=False)


stock_data = get_stock_data()  # 需要自行实现数据获取

strong_stocks = filter_strong_stocks(stock_data)
print(f"筛选到{len(strong_stocks)}只强势股：")
print(strong_stocks.head(10))

In [3]:
from scipy.stats import linregress
import numpy as np

In [4]:
import sys
sys.path.append(r"D:\workspace\TradeX\ezMoney")
from date_utils import get_trade_dates_by_end, get_current_date
from common import factors

In [5]:
datadf = factors.get_n_days_data_batch(['603131.SH'], '2025-07-12', 15)[0]

In [ ]:
datadf

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250711,1752163200000,17.78,17.84,17.64,17.81,62638,111215054.0,0.0,15,17.79,0
20250710,1752076800000,17.76,17.89,17.56,17.79,79971,141657359.0,0.0,15,17.90,0
20250709,1751990400000,18.01,18.25,17.80,17.90,101702,182931595.0,0.0,15,18.16,0
20250708,1751904000000,17.95,18.47,17.85,18.16,123215,223432391.0,0.0,15,18.06,0
20250707,1751817600000,18.11,18.14,17.66,18.06,136348,244719722.0,0.0,15,17.88,0
20250704,1751558400000,18.08,18.40,17.87,17.88,244533,441489993.0,0.0,15,18.44,0
20250703,1751472000000,17.56,19.32,17.56,18.44,406412,766161210.0,0.0,15,17.56,0
20250702,1751385600000,17.60,17.78,17.30,17.56,76083,133602054.0,0.0,15,17.55,0
20250701,1751299200000,17.71,17.82,17.45,17.55,57046,100291389.0,0.0,15,17.72,0
20250630,1751212800000,17.44,17.79,17.36,17.72,81086,143391046.0,0.0,15,17.45,0


: 

In [8]:
datadf['close'].shift(1)

20250711      NaN
20250710    17.81
20250709    17.79
20250708    17.90
20250707    18.16
20250704    18.06
20250703    17.88
Name: close, dtype: float64

In [39]:
datadf

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag
20250711,1752163200000,17.78,17.84,17.64,17.81,62638,111215054.0,0.0,15,17.79,0
20250710,1752076800000,17.76,17.89,17.56,17.79,79971,141657359.0,0.0,15,17.90,0
20250709,1751990400000,18.01,18.25,17.80,17.90,101702,182931595.0,0.0,15,18.16,0
20250708,1751904000000,17.95,18.47,17.85,18.16,123215,223432391.0,0.0,15,18.06,0
20250707,1751817600000,18.11,18.14,17.66,18.06,136348,244719722.0,0.0,15,17.88,0
20250704,1751558400000,18.08,18.40,17.87,17.88,244533,441489993.0,0.0,15,18.44,0
20250703,1751472000000,17.56,19.32,17.56,18.44,406412,766161210.0,0.0,15,17.56,0
20250702,1751385600000,17.60,17.78,17.30,17.56,76083,133602054.0,0.0,15,17.55,0
20250701,1751299200000,17.71,17.82,17.45,17.55,57046,100291389.0,0.0,15,17.72,0
20250630,1751212800000,17.44,17.79,17.36,17.72,81086,143391046.0,0.0,15,17.45,0


In [33]:
datadf['close'][::-1]

20250703    18.44
20250704    17.88
20250707    18.06
20250708    18.16
20250709    17.90
20250710    17.79
20250711    17.81
Name: close, dtype: float64

In [42]:
reversed_close = datadf['close'][::-1]
datadf['price_slope_5'] = reversed_close.rolling(window=7).apply(
lambda x: linregress(np.arange(len(x)), x).slope, raw=True
)[::-1]

In [ ]:
datadf

,time,open,high,low,close,volume,amount,settelementPrice,openInterest,preClose,suspendFlag,price_slope_5
20250711,1752163200000,17.78,17.84,17.64,17.81,62638,111215054.0,0.0,15,17.79,0,-0.079643
20250710,1752076800000,17.76,17.89,17.56,17.79,79971,141657359.0,0.0,15,17.90,0,-0.003929
20250709,1751990400000,18.01,18.25,17.80,17.90,101702,182931595.0,0.0,15,18.16,0,0.066786
20250708,1751904000000,17.95,18.47,17.85,18.16,123215,223432391.0,0.0,15,18.06,0,0.095000
20250707,1751817600000,18.11,18.14,17.66,18.06,136348,244719722.0,0.0,15,17.88,0,0.108571
20250704,1751558400000,18.08,18.40,17.87,17.88,244533,441489993.0,0.0,15,18.44,0,0.113214
20250703,1751472000000,17.56,19.32,17.56,18.44,406412,766161210.0,0.0,15,17.56,0,0.114643
20250702,1751385600000,17.60,17.78,17.30,17.56,76083,133602054.0,0.0,15,17.55,0,0.044643
20250701,1751299200000,17.71,17.82,17.45,17.55,57046,100291389.0,0.0,15,17.72,0,0.091071
20250630,1751212800000,17.44,17.79,17.36,17.72,81086,143391046.0,0.0,15,17.45,0,NaN


: 

In [31]:
datadf.drop(columns=['price_slope_5'], inplace=True)

In [ ]:
def calculate_technical_factors(df, window=10):
    """
    计算技术分析因子
    :param df: 包含原始数据的DataFrame
    :param window: 分析窗口大小
    :return: 包含技术因子的DataFrame
    """
    # 基础计算
    df['returns'] = df['close'].pct_change()
    df['prev_close'] = df['close'].shift(1)
    
    # 1. 动量类因子
    # 价格斜率因子 (5日)

    reversed_close = df['close'][::-1]
    df['price_slope_5'] = reversed_close.rolling(window=5).apply(
        lambda x: linregress(np.arange(len(x)), x).slope, raw=True
    )[::-1].reset_index(drop=True)
    
    # 相对强弱因子 (10日)
    up_days = df['close'].diff().apply(lambda x: x if x > 0 else 0).rolling(window=window).sum()
    down_days = df['close'].diff().apply(lambda x: -x if x < 0 else 0).rolling(window=window).sum()
    df['RSI'] = 100 - (100 / (1 + (up_days / down_days)))
    
    # 2. 成交量类因子
    # 量价背离因子
    df['volume_ma_5'] = df['volume'].rolling(window=5).mean()
    df['price_ma_5'] = df['close'].rolling(window=5).mean()
    df['volume_price_divergence'] = df['volume_ma_5'] / df['price_ma_5']
    
    # 量能聚集因子 (10日)
    df['volatility'] = df['high'] - df['low']
    df['volume_cluster'] = df['volume'].rolling(window=window).std() / df['volatility'].rolling(window=window).mean()
    
    # 3. 价格位置因子
    # 价格通道位置 (10日)
    df['high_10'] = df['high'].rolling(window=window).max()
    df['low_10'] = df['low'].rolling(window=window).min()
    df['price_position'] = (df['close'] - df['low_10']) / (df['high_10'] - df['low_10'])
    
    # 均线排列因子 (5日)
    df['ma5'] = df['close'].rolling(window=5).mean()
    df['ma10'] = df['close'].rolling(window=10).mean()
    df['ma_arrangement'] = (df['ma5'] > df['ma10']).astype(int)
    
    # 4. 波动率因子
    # 真实波幅 (ATR)
    df['tr1'] = df['high'] - df['low']
    df['tr2'] = abs(df['high'] - df['prev_close'])
    df['tr3'] = abs(df['low'] - df['prev_close'])
    df['true_range'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)
    df['ATR_5'] = df['true_range'].rolling(window=5).mean()
    
    # 5. 特殊形态因子
    # 炸板识别因子
    df['is_limit_up'] = (df['high'] == df['low']) & (df['close'] > df['prev_close'] * 1.095)
    df['limit_up_open'] = df['is_limit_up'].shift(1)
    df['volume_change'] = df['volume'] / df['volume'].shift(1)
    df['is_blow_up'] = (df['limit_up_open'] & 
                       (df['close'] < df['open'] * 0.97) & 
                       (df['volume_change'] > 1.8))
    
    # 长上影线识别 (射击之星)
    body_size = abs(df['close'] - df['open'])
    upper_shadow = df['high'] - df[['open', 'close']].max(axis=1)
    df['is_shooting_star'] = (upper_shadow > body_size * 1.5) & (df['close'] < df['open'])
    
    # 6. 趋势强度因子
    # ADI 趋势强度
    df['cmf'] = ((2*df['close'] - df['low'] - df['high']) / 
                (df['high'] - df['low'])) * df['volume']
    df['ADI_10'] = df['cmf'].rolling(window=10).sum()
    
    # 7. 量价协同因子
    # OBV 能量潮
    df['obv'] = np.where(df['close'] > df['prev_close'], 
                        df['volume'], 
                        np.where(df['close'] < df['prev_close'], -df['volume'], 0)).cumsum()
    
    return df

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import linregress

def calculate_trend_factors(df):
    """
    计算多维趋势因子，包含不同周期趋势强度和转折信号
    """
    # 基础价格数据准备
    df['prev_close'] = df['close'].shift(1)
    
    # =====================
    # 1. 不同周期斜率因子
    # =====================
    reversed_close = df['close'][::-1]

    df['slope_2'] = reversed_close.rolling(window=2).apply(
        lambda x: linregress(np.arange(2), x).slope, raw=True)[::-1]

    # 3日斜率 (超短期趋势)
    df['slope_3'] = reversed_close.rolling(window=3).apply(
        lambda x: linregress(np.arange(3), x).slope, raw=True)[::-1]
    
    # 5日斜率 (短期趋势)
    df['slope_5'] = reversed_close.rolling(window=5).apply(
        lambda x: linregress(np.arange(5), x).slope, raw=True)[::-1]
    
    # 10日斜率 (中期趋势)
    df['slope_10'] = reversed_close.rolling(window=10).apply(
        lambda x: linregress(np.arange(10), x).slope, raw=True)[::-1]
    
    # ==========================
    # 2. 趋势强度与健康度因子
    # ==========================
    
    # 趋势一致性指标 (2/3/5/10日斜率符号一致性)
    df['trend_consistency'] = (
        np.sign(df['slope_2']) +
        np.sign(df['slope_3']) + 
        np.sign(df['slope_5']) + 
        np.sign(df['slope_10'])
    ) / 4.0
    
    # 趋势强度比 (短期vs中期)
    df['trend_strength_ratio5'] = np.abs(df['slope_5']) / (np.abs(df['slope_10']) + 0.001)

    df['trend_strength_ratio2'] = np.abs(df['slope_2']) / (np.abs(df['slope_3']) + 0.001)

    df['trend_strength_ratio3'] = np.abs(df['slope_3']) / (np.abs(df['slope_5']) + 0.001)
    
    # 量能趋势配合度 (价格斜率与成交量斜率的相关系数)

    reversed_volume = df['volume'][::-1]
    volume_slope_5 = reversed_volume.rolling(window=5).apply(
        lambda x: linregress(np.arange(5), x).slope, raw=True)[::-1]
    
    volume_slope_3 = reversed_volume.rolling(window=3).apply(
        lambda x: linregress(np.arange(3), x).slope, raw=True)[::-1]
    df['volume_trend_corr5'] = df['slope_5'].rolling(window=5).corr(volume_slope_5)

    df['volume_trend_corr3'] = df['slope_3'].rolling(window=3).corr(volume_slope_3)
    
    # ==========================
    # 3. 趋势转折识别因子
    # ==========================
    
    # 斜率变化加速度 (二阶导数)
    df['slope_accel_2'] = df['slope_2'].diff()
    df['slope_accel_3'] = df['slope_3'].diff()
    df['slope_accel_5'] = df['slope_5'].diff()
    
    # 趋势反转信号 (金叉死叉型)
    df['trend_reversal_signal'] = np.where(
        (df['slope_3'] > 0) & (df['slope_3'].shift(1) < 0), 1,  # 由跌转涨
        np.where(
            (df['slope_3'] < 0) & (df['slope_3'].shift(1) > 0), -1,  # 由涨转跌
            0  # 无变化
        )
    )
    
    # 多头排列健康度 (EMA12>EMA26>EMA50)
    df['ema5'] = df['close'].ewm(span=5, adjust=False).mean()
    df['ema10'] = df['close'].ewm(span=10, adjust=False).mean()
    df['ema20'] = df['close'].ewm(span=20, adjust=False).mean()
    df['ma_health'] = (
        (df['ema5'] > df['ema10']).astype(int) + 
        (df['ema10'] > df['ema20']).astype(int)
    )
    
    # ==========================
    # 4. A杀特征识别因子
    # ==========================
    
    # 急跌特征 (3日内最大回撤)
    df['max_drawdown_3'] = (df['close'].rolling(window=3).max() - df['close']) / df['close']
    
    # 高位放量下跌 (A杀核心特征)
    df['high_position'] = df['close'] / df['close'].rolling(window=20).max()
    df['volume_spike'] = df['volume'] / df['volume'].rolling(window=10).mean()
    df['a_sha_signal'] = (
        (df['trend_reversal_signal'] == -1) &  # 趋势刚转跌
        (df['high_position'] > 0.85) &        # 处于近期高位
        (df['volume_spike'] > 1.8) &          # 放量下跌
        (df['max_drawdown_3'] > 0.08)         # 短期大幅回撤
    ).astype(int)
    
    # 趋势破位信号 (跌破关键均线)
    df['break_ema26'] = (df['close'] < df['ema26']) & (df['prev_close'] > df['ema26'])
    df['break_ema50'] = (df['close'] < df['ema50']) & (df['prev_close'] > df['ema50'])
    
    # ==========================
    # 5. 复合趋势健康评分
    # ==========================
    
    # 趋势健康评分 (0-100分)
    df['trend_health_score'] = (
        # 正向因子
        30 * (df['slope_5'] > 0).astype(int) + 
        20 * (df['trend_consistency'] > 0.5).astype(int) + 
        15 * (df['volume_trend_corr'] > 0.3).astype(int) + 
        15 * (df['ma_health'] == 2).astype(int) + 
        
        # 负向因子
        -20 * (df['a_sha_signal'] == 1).astype(int) + 
        -15 * (df['break_ema26'] == 1).astype(int) + 
        -10 * (df['trend_reversal_signal'] == -1).astype(int)
    )
    
    # 确保分数在合理范围
    df['trend_health_score'] = np.clip(df['trend_health_score'], 0, 100)
    
    return df

def analyze_trend_status(df):
    """
    综合判断股票趋势状态
    """
    # 计算所有趋势因子
    df = calculate_trend_factors(df)
    
    # 最近3天趋势状态
    df['trend_status_3d'] = np.where(
        df['slope_3'] > 0.005, 'strong_up',
        np.where(
            df['slope_3'] < -0.005, 'strong_down',
            np.where(
                df['slope_3'] > 0, 'weak_up', 'weak_down'
            )
        )
    )
    
    # 趋势健康度分类
    df['trend_health'] = np.where(
        df['trend_health_score'] >= 70, 'excellent',
        np.where(
            df['trend_health_score'] >= 50, 'good',
            np.where(
                df['trend_health_score'] >= 30, 'caution', 'danger'
            )
        )
    )
    
    # 趋势转折预警
    df['trend_warning'] = np.where(
        df['a_sha_signal'] == 1, 'A杀风险',
        np.where(
            (df['trend_reversal_signal'] == -1) & (df['high_position'] > 0.8), 
            '高位转跌',
            np.where(
                (df['slope_accel_5'] < -0.002) & (df['slope_5'] > 0), 
                '上涨减速',
                '无预警'
            )
        )
    )
    
    return df

# 使用示例
if __name__ == "__main__":
    # 加载数据
    df = pd.read_csv('stock_data.csv')
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time')
    
    # 按股票代码分组处理
    results = []
    for symbol, group in df.groupby('symbol'):
        # 仅保留最近15个交易日数据
        recent_data = group.tail(15).copy()
        
        # 计算趋势因子
        analyzed = analyze_trend_status(recent_data)
        
        # 取最新一天的分析结果
        latest = analyzed.iloc[-1].copy()
        latest['symbol'] = symbol
        results.append(latest)
    
    # 创建分析结果DataFrame
    trend_df = pd.DataFrame(results)
    
    # 关键分析列
    analysis_cols = [
        'symbol', 'time', 'close', 
        'slope_3', 'slope_5', 'slope_10',
        'trend_status_3d', 'trend_health', 'trend_warning',
        'trend_consistency', 'trend_strength_ratio', 'a_sha_signal',
        'trend_health_score'
    ]
    
    # 输出分析结果
    print(trend_df[analysis_cols])